In [ ]:
from re import S
from ctypes import create_unicode_buffer
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import time
import math
from itertools import combinations

from math import ceil
from functools import partial
from sklearn.decomposition import PCA



# ----------------------------------------------
# Multi-scale MMD divergence
# ----------------------------------------------

try:
    import cupy as cp
except Exception:
    cp = None
import numpy as np

M_MMD_DEFAULTS = {
    # kernel mixture
    "kernel": "rbf",        # 'rbf' or 'imq'
    "scales": None,         # list/array of bandwidths (RBF sigmas or IMQ c's). If None, use median heuristic.
    "scale_weights": None,  # list/array of nonnegative weights (same length as scales). If None, uniform.
    # median heuristic for automatic scales (used if scales=None)
    "use_median_heuristic": True,
    "num_scales": 5,        # number of scales in the mixture
    "scale_min": 0.5,       # lower multiplier relative to base (logspace)
    "scale_max": 2.0,       # upper multiplier relative to base (logspace)
    # estimator & output
    "unbiased": True,       # U-statistic (exclude diagonals in Kxx, Kyy)
    "squared": False,       # return MMD^2 if True, else sqrt(max(MMD^2,0))
    # kernel-specific params
    "imq_beta": 0.5,        # k_IMQ(d2) = (1 + d2 / c^2)^(-beta)
    # numerics
    "chunk_size": 4096,     # process pairwise blocks to limit memory
    "return_per_scale": False, # if True, also return dict with per-scale contributions
    # optional per-sample weights (length N_x / N_y). If provided, computes *weighted* MMD.
    "x_weights": None,
    "y_weights": None,
}

def _xp_of(arr):
    """Return array module (np or cp) that matches arr."""
    if cp is not None and isinstance(arr, cp.ndarray):
        return cp
    return np

def _as_xp(x, xp):
    """Ensure x is an xp array."""
    if xp is np:
        return np.asarray(x)
    else:
        return x if isinstance(x, xp.ndarray) else xp.asarray(x)

def _median_pairwise_sqdist(Z, xp, max_samples=1024):
    """Robust median of pairwise squared distances from a subsample."""
    n = Z.shape[0]
    if n <= 1:
        return xp.array(1.0, dtype=Z.dtype)
    idx = xp.random.permutation(n)[:min(n, max_samples)]
    S = Z[idx]
    # pairwise squared distances for the subsample
    S2 = xp.sum(S * S, axis=1, keepdims=True)
    D2 = xp.maximum(S2 + S2.T - 2 * (S @ S.T), 0.0)
    # exclude zeros on the diagonal
    tri = D2[~xp.eye(D2.shape[0], dtype=bool)]
    if tri.size == 0:
        return xp.array(1.0, dtype=Z.dtype)
    return xp.median(tri)

def _kernel_from_d2(d2, kernel, scale, xp, beta):
    """Evaluate kernel from squared distances."""
    if kernel == "rbf":
        # k(d2) = exp(- d2 / (2 sigma^2))
        inv2sig2 = 1.0 / (2.0 * (scale ** 2))
        return xp.exp(-d2 * inv2sig2)
    elif kernel == "imq":
        # k(d2) = (1 + d2 / c^2)^(-beta)
        return (1.0 + d2 / (scale ** 2)) ** (-beta)
    else:
        raise ValueError(f"Unknown kernel '{kernel}' (use 'rbf' or 'imq').")

def _sum_kernel_pairs_XY(X, Y, scales, weights, kernel, xp, beta, chunk, xw=None, yw=None):
    """
    Sum over k(x_i, y_j) for all i,j across a *mixture* of kernels.
    Returns per-scale sums as array [S] and normalization constant (weighted).
    """
    nx, ny = X.shape[0], Y.shape[0]
    S = xp.zeros((len(scales),), dtype=X.dtype)
    # normalization (weighted or unweighted)
    if xw is None and yw is None:
        norm = float(nx) * float(ny)
    else:
        xw = xp.ones(nx, dtype=X.dtype) if xw is None else xw
        yw = xp.ones(ny, dtype=Y.dtype) if yw is None else yw
        norm = float(xp.sum(xw) * xp.sum(yw))

    for i0 in range(0, nx, chunk):
        Xi = X[i0:i0 + chunk]
        # precompute norms for distance
        Xi2 = xp.sum(Xi * Xi, axis=1, keepdims=True)  # (bi,1)
        wi = None if xw is None else xw[i0:i0 + chunk]
        for j0 in range(0, ny, chunk):
            Yj = Y[j0:j0 + chunk]
            Yj2 = xp.sum(Yj * Yj, axis=1, keepdims=True)  # (bj,1)
            # (bi,bj) squared distances
            D2 = xp.maximum(Xi2 + Yj2.T - 2.0 * (Xi @ Yj.T), 0.0)
            # stack per-scale kernels with broadcasting: (S, bi, bj)
            Ks = xp.stack([_kernel_from_d2(D2, kernel, s, xp, beta) for s in scales], axis=0)
            if xw is None and yw is None:
                S += Ks.sum(axis=(1, 2))
            else:
                wj = yw[j0:j0 + chunk]
                W = (wi[:, None] * wj[None, :])  # (bi,bj)
                S += xp.tensordot(Ks, W, axes=((1, 2), (0, 1)))  # sum_{bi,bj} Ks[...,bi,bj]*W[bi,bj]
    # apply mixture weights here (leave per-scale sums unweighted for diagnostics)
    if weights is not None:
        total = float(xp.sum(weights * (S / norm)))
    else:
        total = float(xp.sum(S / norm))
    return S, norm, total

def _sum_kernel_pairs_XX(A, scales, weights, kernel, xp, beta, chunk, unbiased=True, aw=None):
    """
    Sum over k(a_i, a_j) across a *mixture* of kernels for a single set A.
    Returns per-scale sums as array [S] and normalization constant (weighted).
    For unbiased=True, excludes diagonals (i != j) using ordered-pairs normalization n(n-1).
    """
    n = A.shape[0]
    S = xp.zeros((len(scales),), dtype=A.dtype)

    if aw is None:
        if unbiased:
            norm = float(n) * float(max(n - 1, 1))
        else:
            norm = float(n) * float(n)
    else:
        aw = aw.astype(A.dtype, copy=False)
        s1 = xp.sum(aw)
        if unbiased:
            norm = float(s1 * s1 - xp.sum(aw * aw))
        else:
            norm = float(s1 * s1)

    for i0 in range(0, n, chunk):
        Ai = A[i0:i0 + chunk]
        Ai2 = xp.sum(Ai * Ai, axis=1, keepdims=True)
        wi = None if aw is None else aw[i0:i0 + chunk]
        j_start = i0  # upper blocks including diagonal
        for j0 in range(j_start, n, chunk):
            Aj = A[j0:j0 + chunk]
            Aj2 = xp.sum(Aj * Aj, axis=1, keepdims=True)
            D2 = xp.maximum(Ai2 + Aj2.T - 2.0 * (Ai @ Aj.T), 0.0)
            Ks = xp.stack([_kernel_from_d2(D2, kernel, s, xp, beta) for s in scales], axis=0)  # (S,bi,bj)

            if aw is None:
                if i0 == j0:
                    if unbiased:
                        # subtract diagonals; keep off-diagonals both (i,j) and (j,i) inside the block
                        diag = xp.stack([xp.diag(Ks[k]) for k in range(Ks.shape[0])], axis=0).sum(axis=1)
                        S += Ks.sum(axis=(1, 2)) - diag
                    else:
                        S += Ks.sum(axis=(1, 2))
                else:
                    # off-diagonal blocks contribute both (i,j) and (j,i)
                    S += 2.0 * Ks.sum(axis=(1, 2))
            else:
                wj = aw[j0:j0 + chunk]
                W = (wi[:, None] * wj[None, :])  # (bi,bj)
                if i0 == j0:
                    if unbiased:
                        # zero-out diagonal weights before summing
                        # subtract weighted diagonal: sum_k Ks[k]*W with diag set to 0
                        # Equivalent: total sum minus diag(K)*w^2
                        diag = xp.stack([xp.diag(Ks[k]) for k in range(Ks.shape[0])], axis=0)  # (S, b)
                        diag_w2 = (wi * wi)[None, :] * diag  # (S,b)
                        S += xp.tensordot(Ks, W, axes=((1, 2), (0, 1))) - diag_w2.sum(axis=1)
                    else:
                        S += xp.tensordot(Ks, W, axes=((1, 2), (0, 1)))
                else:
                    # both (i,j) and (j,i) pairs
                    S += 2.0 * xp.tensordot(Ks, W, axes=((1, 2), (0, 1)))
    if weights is not None:
        total = float(xp.sum(weights * (S / norm)))
    else:
        total = float(xp.sum(S / norm))
    return S, norm, total

def M_MMD(X, Y, params=None):
    """
    Multi-scale MMD divergence between sample sets X ~ P and Y ~ Q.

    Args:
        X: (N_x, d) numpy or cupy array.
        Y: (N_y, d) numpy or cupy array.
        params: dict overriding M_MMD_DEFAULTS.
            - kernel: 'rbf' or 'imq'
            - scales: list/array of bandwidths (RBF sigmas or IMQ c's). If None, automatic.
            - scale_weights: list/array (same length as scales). Defaults to uniform.
            - use_median_heuristic: if True and scales=None, set base scale from combined median distance.
                * RBF: base sigma = sqrt(0.5 * median(D^2)), then logspace multipliers [scale_min, scale_max]
                * IMQ: base c = sqrt(median(D^2)), then logspace multipliers
            - num_scales, scale_min, scale_max: for automatic scales (logspace).
            - unbiased: U-statistic (exclude diagonals in within terms).
            - squared: return MMD^2 if True; else return sqrt(max(MMD^2, 0)).
            - imq_beta: exponent for IMQ kernel.
            - chunk_size: block size for memory control.
            - x_weights, y_weights: optional per-sample nonnegative weights.
            - return_per_scale: if True, also returns a dict with per-scale pieces.

    Returns:
        mmd_value (float), and optionally (if return_per_scale) a dict with
        'per_scale_mmd2', 'scales', 'scale_weights'.
    """
    # merge params with defaults
    p = dict(M_MMD_DEFAULTS)
    if params is not None:
        p.update(params)

    # choose backend
    xp = _xp_of(X)
    X = _as_xp(X, xp)
    Y = _as_xp(Y, xp)
    X = X.reshape(X.shape[0], -1)
    Y = Y.reshape(Y.shape[0], -1)

    # weights (optional)
    xw = None if p["x_weights"] is None else _as_xp(p["x_weights"], xp).astype(X.dtype, copy=False)
    yw = None if p["y_weights"] is None else _as_xp(p["y_weights"], xp).astype(Y.dtype, copy=False)

    # build scales & weights
    if p["scales"] is None:
        # base scale from combined median distance
        Z = xp.concatenate([X, Y], axis=0)
        med2 = _median_pairwise_sqdist(Z, xp, max_samples=1024)
        if p["kernel"] == "rbf":
            base = xp.sqrt(xp.maximum(0.5 * med2, 1e-12))
        else:  # imq
            base = xp.sqrt(xp.maximum(med2, 1e-12))
        mul = xp.logspace(np.log10(p["scale_min"]), np.log10(p["scale_max"]), p["num_scales"])
        scales = base * mul
    else:
        scales = _as_xp(p["scales"], xp).astype(X.dtype, copy=False)

    if p["scale_weights"] is None:
        scale_weights = xp.ones((len(scales),), dtype=X.dtype) / float(len(scales))
    else:
        w = _as_xp(p["scale_weights"], xp).astype(X.dtype, copy=False)
        w_sum = xp.sum(w)
        scale_weights = w / (w_sum if float(w_sum) > 0 else 1.0)

    # compute per-set & cross sums, with mixture applied after normalization
    Sxx, nxx, mix_xx = _sum_kernel_pairs_XX(X, scales, scale_weights, p["kernel"], xp,
                                            p["imq_beta"], p["chunk_size"], p["unbiased"], xw)
    Syy, nyy, mix_yy = _sum_kernel_pairs_XX(Y, scales, scale_weights, p["kernel"], xp,
                                            p["imq_beta"], p["chunk_size"], p["unbiased"], yw)
    Sxy, nxy, mix_xy = _sum_kernel_pairs_XY(X, Y, scales, scale_weights, p["kernel"], xp,
                                            p["imq_beta"], p["chunk_size"], xw, yw)

    # Full mixture MMD^2
    mmd2 = (mix_xx + mix_yy - 2.0 * mix_xy)
    # numerical guard
    if mmd2 < 0.0:
        mmd2 = 0.0

    if p["squared"]:
        out = float(mmd2)
    else:
        out = float(np.sqrt(mmd2))  # safe cast; cp scalar ok through numpy sqrt of float

    if p["return_per_scale"]:
        # per-scale contributions *before* mixture weighting, but normalized
        per_scale = (Sxx / nxx) + (Syy / nyy) - 2.0 * (Sxy / nxy)
        # clamp small negatives
        per_scale = xp.maximum(per_scale, 0.0)
        return out, {
            "per_scale_mmd2": _as_xp(per_scale, np),  # move to numpy for easy logging
            "scales": _as_xp(scales, np),
            "scale_weights": _as_xp(scale_weights, np),
        }
    return out


# ------------------------------ Single-σ MMD ----------------------------------
def OT_MMD(X, Y, *, sigma=1.0, unbiased=False, chunk=4096):
    """
    Single-scale RBF MMD between X and Y with bandwidth `sigma`.
    Works with NumPy or CuPy arrays; keeps everything on-device if CuPy.
    """
    # choose xp
    xp = cp if (cp is not None and isinstance(X, cp.ndarray)) or \
               (cp is not None and isinstance(Y, cp.ndarray)) else np
    X = xp.asarray(X, dtype=xp.float64).reshape(X.shape[0], -1)
    Y = xp.asarray(Y, dtype=xp.float64).reshape(Y.shape[0], -1)
    n, m = X.shape[0], Y.shape[0]
    inv2s2 = 1.0 / (2.0 * (float(sigma) ** 2))

    def _rbf_sum(A, B):
        SA = xp.sum(A * A, axis=1, keepdims=True)
        SB = xp.sum(B * B, axis=1, keepdims=True)
        D2 = xp.maximum(SA + SB.T - 2.0 * (A @ B.T), 0.0)
        return xp.exp(-inv2s2 * D2).sum()

    if unbiased:
        # U-statistic: exclude self terms in Kxx, Kyy, normalize by n(n-1), m(m-1)
        # Block to limit memory
        def _self_sum(A):
            s = xp.array(0.0, dtype=xp.float64)
            NA = A.shape[0]
            for i0 in range(0, NA, chunk):
                Ai = A[i0:i0 + chunk]
                Ai2 = xp.sum(Ai * Ai, axis=1, keepdims=True)
                for j0 in range(0, NA, chunk):
                    Aj = A[j0:j0 + chunk]
                    Aj2 = xp.sum(Aj * Aj, axis=1, keepdims=True)
                    D2 = xp.maximum(Ai2 + Aj2.T - 2.0 * (Ai @ Aj.T), 0.0)
                    K = xp.exp(-inv2s2 * D2)
                    if i0 == j0:
                        K = K - xp.eye(K.shape[0], dtype=K.dtype)
                    s += K.sum()
            return s

        s_xx = _self_sum(X) / max(n * (n - 1), 1)
        s_yy = _self_sum(Y) / max(m * (m - 1), 1)

    else:
        # Biased estimator: include diagonals, normalize by n^2, m^2
        s_xx = _rbf_sum(X, X) / max(n * n, 1)
        s_yy = _rbf_sum(Y, Y) / max(m * m, 1)

    s_xy = _rbf_sum(X, Y) / max(n * m, 1)
    mmd2 = float((s_xx + s_yy - 2.0 * s_xy))
    return math.sqrt(mmd2) if mmd2 > 0 else 0.0


# Script One, works well, just fits np estimators uses for sampling


def _to_numpy(arr):
    """Return a NumPy view; copy from GPU to CPU if the input is a CuPy array."""
    try:
        import cupy as cp
        if isinstance(arr, cp.ndarray):
            return arr.get()          # move to host
    except ImportError:
        pass                          # no CuPy in this env
    return np.asarray(arr)


def prune_cp_arr(x_cp, p_percent = 1):
    x = cp.asarray(x_cp)
    if x.ndim != 2:
        raise ValueError("expected a 2D CuPy array (N,d)")
    if not (0 <= p_percent <= 100):
        raise ValueError("p_percent must be in [0, 100]")

    N = x.shape[0]
    if N == 0:
        return x, cp.zeros(0, dtype=cp.bool_)

    finite_rows = cp.isfinite(x).all(axis=1)
    idx_finite = cp.where(finite_rows)[0]
    xf = x[finite_rows]
    M = xf.shape[0]

    if M == 0:
        return x[:0], cp.zeros(N, dtype=cp.bool_)

    if p_percent == 0:
        keep_mask = cp.zeros(N, dtype=cp.bool_)
        keep_mask[idx_finite] = True
        return xf, keep_mask
    if p_percent == 100:
        return x[:0], cp.zeros(N, dtype=cp.bool_)

    k = int(cp.ceil(M * (1.0 - p_percent / 100.0)).item())  # rows to keep among finite ones
    if k <= 0:
        return x[:0], cp.zeros(N, dtype=cp.bool_)

    norms = cp.linalg.norm(xf, axis=1)
    keep_local = cp.argpartition(norms, kth=k-1)[:k]  # indices of k smallest norms in xf

    keep_mask = cp.zeros(N, dtype=cp.bool_)
    keep_mask[idx_finite[keep_local]] = True
    return x[keep_mask], keep_mask


import cupy as cp



def sample_gmm_gpu(n_samples, means, stds, weights, *, seed=None):
    xp = cp
    means   = xp.asarray(means,   dtype=xp.float64)
    stds    = xp.asarray(stds,    dtype=xp.float64)
    weights = xp.asarray(weights, dtype=xp.float64).reshape(-1)

    C, D = means.shape
    if stds.ndim == 1 and stds.shape == (C,):
        cov_mode = "isotropic"
    elif stds.ndim == 2 and stds.shape == (C, D):
        cov_mode = "diag"
    elif stds.ndim == 3 and stds.shape == (C, D, D):
        cov_mode = "full"
    else:
        raise ValueError("stds must be (C,), (C,D), or (C,D,D)")

    probs = (weights / float(weights.sum())).ravel()
    rng = xp.random.RandomState(seed) if seed is not None else xp.random
    choices = rng.choice(C, size=int(n_samples), p=cp.asnumpy(probs))
    out = xp.empty((n_samples, D), dtype=xp.float64)

    for i in range(C):
        mask = (choices == i); cnt = int(mask.sum())
        if not cnt: continue
        z = rng.standard_normal((cnt, D), dtype=xp.float64)
        if cov_mode == "isotropic":
            out[mask] = means[i] + z * stds[i]
        elif cov_mode == "diag":
            out[mask] = means[i] + z * stds[i]          # (cnt,D) * (D,)
        else:  # full
            L = xp.linalg.cholesky(stds[i])             # Σ^{1/2}
            out[mask] = means[i] + z @ L.T
    return out



def score_gmm_gpu(x, means, stds, weights, batch_size=4096):
    """
    ∇_x log ∑_c w_c N(x; μ_c, Σ_c)
    stds:
      - (C,)   : isotropic std per comp  (σ_c)          -> Σ_c = σ_c^2 I
      - (C,D)  : diagonal std per comp   (σ_{c,d})      -> Σ_c = diag(σ^2)
      - (C,D,D): full covariance per comp (Σ_c SPD)
    """
    xp = cp
    x       = xp.asarray(x,       dtype=xp.float64)
    means   = xp.asarray(means,   dtype=xp.float64)
    weights = xp.asarray(weights, dtype=xp.float64).reshape(-1)
    stds    = xp.asarray(stds,    dtype=xp.float64)

    N, D = x.shape
    C    = means.shape[0]
    if N == 0:
        return xp.zeros((0, D), dtype=xp.float64)

    const = -0.5 * D * xp.log(2.0 * xp.pi)
    log_w = xp.log(weights + 1e-300)
    out   = xp.zeros_like(x)

    # ----- (1) Isotropic: stds = (C,)
    if stds.ndim == 1:
        inv_vars     = 1.0 / (stds**2)                 # (C,)
        log_det_vars = D * xp.log(stds**2)             # (C,)
        norm_const   = const - 0.5 * log_det_vars      # (C,)

        for i in range(0, N, batch_size):
            xb   = x[i:i+batch_size]                   # (B,D)
            diff = means[None, :, :] - xb[:, None, :]  # (B,C,D)
            quad = (diff**2).sum(axis=-1) * inv_vars[None, :]         # (B,C)

            logp = log_w[None, :] + norm_const[None, :] - 0.5 * quad  # (B,C)
            m    = xp.max(logp, axis=1, keepdims=True)
            resp = xp.exp(logp - m); resp /= resp.sum(axis=1, keepdims=True) + 1e-300

            contrib = (resp[:, :, None] * diff) * inv_vars[None, :, None]   # (B,C,D)
            out[i:i+batch_size] = contrib.sum(axis=1)

        return out

    # ----- (2) Diagonal: stds = (C,D)
    if stds.ndim == 2 and stds.shape[1] == D:
        inv_vars     = 1.0 / (stds**2)                 # (C,D)
        log_det_vars = xp.log(stds**2).sum(axis=1)     # (C,)
        norm_const   = const - 0.5 * log_det_vars      # (C,)

        for i in range(0, N, batch_size):
            xb   = x[i:i+batch_size]                   # (B,D)
            diff = means[None, :, :] - xb[:, None, :]  # (B,C,D)
            quad = ((diff**2) * inv_vars[None, :, :]).sum(axis=-1)          # (B,C)

            logp = log_w[None, :] + norm_const[None, :] - 0.5 * quad        # (B,C)
            m    = xp.max(logp, axis=1, keepdims=True)
            resp = xp.exp(logp - m); resp /= resp.sum(axis=1, keepdims=True) + 1e-300

            contrib = (resp[:, :, None] * diff) * inv_vars[None, :, :]      # (B,C,D)
            out[i:i+batch_size] = contrib.sum(axis=1)

        return out

    # ----- (3) Full: stds = (C,D,D)  (here 'stds' is Σ_c)
    if stds.ndim == 3 and stds.shape[1] == D and stds.shape[2] == D:
        # Cholesky once per component
        Ls      = xp.linalg.cholesky(stds)                                     # (C,D,D), Σ = L L^T
        log_det = 2.0 * xp.log(xp.diagonal(Ls, axis1=1, axis2=2)).sum(axis=1)  # (C,)
        norm_const = const - 0.5 * log_det                                     # (C,)

        for i in range(0, N, batch_size):
            xb    = x[i:i+batch_size]                         # (B,D)
            B     = xb.shape[0]
            logp  = xp.empty((B, C), dtype=xp.float64)
            grads = xp.empty((B, C, D), dtype=xp.float64)

            for c in range(C):
                L = Ls[c]                                      # (D,D)

                # quad term: (x - μ)^T Σ^{-1} (x - μ) = || L^{-1}(x-μ) ||^2
                diff_x = xb - means[c][None, :]                # (B,D)
                v = xp.linalg.solve(L, diff_x.T)               # (D,B)
                quad = xp.sum(v * v, axis=0)                   # (B,)
                logp[:, c] = log_w[c] + norm_const[c] - 0.5 * quad

                # gradient per comp: Σ^{-1}(μ - x) = L^{-T} L^{-1} (μ - x)
                diff_m = means[c][None, :] - xb                # (B,D)
                y = xp.linalg.solve(L, diff_m.T)               # (D,B)
                g = xp.linalg.solve(L.T, y).T                  # (B,D)
                grads[:, c, :] = g

            m    = xp.max(logp, axis=1, keepdims=True)
            resp = xp.exp(logp - m); resp /= resp.sum(axis=1, keepdims=True) + 1e-300

            out[i:i+batch_size] = xp.einsum("bc,bcd->bd", resp, grads)

        return out

    raise ValueError("stds must be (C,), (C,D), or (C,D,D) (the last is full covariances).")




def score_div_gmm(x, means, covs, weights, batch_size=4096):

    xp = cp
    x       = xp.asarray(x, dtype=xp.float64)
    means   = xp.asarray(means, dtype=xp.float64)
    covs    = xp.asarray(covs,  dtype=xp.float64)
    weights = xp.asarray(weights, dtype=xp.float64).reshape(-1)

    N, D = x.shape
    C    = means.shape[0]
    if N == 0:
        return xp.zeros_like(x), xp.zeros((0,), dtype=xp.float64)

    # --- weight normalization & constants
    wsum = xp.sum(weights)
    if not xp.isfinite(wsum) or wsum <= 0:
        raise ValueError("Mixture weights must sum to a positive value.")
    log_w = xp.log(weights / wsum + 1e-300)
    const = -0.5 * D * xp.log(2.0 * xp.pi)

    # --- covariance handling
    kind = None
    if covs.ndim == 1 and covs.shape[0] == C:             # isotropic stds
        kind     = "iso"
        inv_vars = 1.0 / (covs**2)                         # (C,)
        log_det  = D * xp.log(covs**2)                     # (C,)
        trA      = D * inv_vars                            # (C,)
    elif covs.ndim == 2 and covs.shape == (C, D):          # diagonal stds
        kind     = "diag"
        inv_vars = 1.0 / (covs**2)                         # (C,D)
        log_det  = xp.log(covs**2).sum(axis=1)             # (C,)
        trA      = inv_vars.sum(axis=1)                    # (C,)
    elif covs.ndim == 3 and covs.shape[1:] == (D, D):      # full Σ
        kind = "full"
        Ls, log_det, trA = [], xp.empty(C), xp.empty(C)
        I = xp.eye(D, dtype=xp.float64)
        for c in range(C):
            L = xp.linalg.cholesky(covs[c])
            Ls.append(L)
            log_det[c] = 2.0 * xp.sum(xp.log(xp.diag(L)))
            Z = xp.linalg.solve(L, I)                      # L^{-1}
            trA[c] = xp.sum(Z * Z)                         # ||L^{-1}||_F^2 = tr(Σ^{-1})
    else:
        raise ValueError("covs must be (C,), (C,D) with stds, or (C,D,D) with full covariances.")

    # --- outputs
    score = xp.empty_like(x)
    div   = xp.empty((N,), dtype=xp.float64)

    for i in range(0, N, batch_size):
        xb = x[i:i+batch_size]                             # (B,D)
        B  = xb.shape[0]

        # per-component log-densities and a_c = A_c (mu_c - x)
        if kind in ("iso", "diag"):
            diff = means[None, :, :] - xb[:, None, :]      # (B,C,D)
            if kind == "iso":
                quad = (diff**2).sum(-1) * inv_vars[None, :]          # (B,C)
                a    = diff * inv_vars[None, :, None]                  # (B,C,D)
            else:
                quad = ((diff**2) * inv_vars[None, :, :]).sum(-1)      # (B,C)
                a    = diff * inv_vars[None, :, :]                     # (B,C,D)
        else:
            quad = xp.empty((B, C), dtype=xp.float64)
            a    = xp.empty((B, C, D), dtype=xp.float64)
            for c in range(C):
                diff = (means[c][None, :] - xb)                        # (B,D)
                # a_c = Σ_c^{-1} diff via two triangular solves
                z = xp.linalg.solve(Ls[c], diff.T)                     # (D,B)
                u = xp.linalg.solve(Ls[c].T, z)                        # (D,B)
                a_c = u.T                                              # (B,D)
                a[:, c, :] = a_c
                quad[:, c] = xp.sum(diff * a_c, axis=1)                # diff^T Σ^{-1} diff

        log_comp = log_w[None, :] + const - 0.5*log_det[None, :] - 0.5*quad  # (B,C)
        m   = xp.max(log_comp, axis=1, keepdims=True)
        resp = xp.exp(log_comp - m)
        resp /= (resp.sum(axis=1, keepdims=True) + 1e-300)             # (B,C)
        s = xp.sum(resp[:, :, None] * a, axis=1)                        # (B,D)

        a_norm2 = xp.sum(a * a, axis=2)                                 # (B,C)
        s_dot_a = xp.sum(s[:, None, :] * a, axis=2)                     # (B,C)
        div_b   = xp.sum(resp * (a_norm2 - s_dot_a - trA[None, :]), axis=1)  # (B,)

        score[i:i+B] = s
        div[i:i+B]   = div_b

    return score, div


def hyvarinen_to_target(X, score_fn, div_fn, batch_size=65536):
    """
    Hyvärinen-to-target loss: E[ 0.5||s(x)||^2 + div s(x) ].
    `score_fn` returns (N,D), `div_fn` returns (N,).
    """
    xp = cp
    X  = xp.asarray(X, dtype=xp.float64)
    N  = X.shape[0]
    acc = 0.0
    cnt = 0
    for i in range(0, N, int(batch_size)):
        xb = X[i:i+batch_size]
        s  = score_fn(xb)
        dv = div_fn(xb)
        acc += float(xp.mean(0.5 * xp.sum(s*s, axis=1) + dv))
        cnt += 1
    return acc / max(cnt, 1)




def density_gmm_gpu(x, means, stds, weights, mode="absolute", iso_std=1.0, batch_size=4096):
    xp = cp
    x      = x.astype(xp.float64, copy=False)
    means  = means.astype(xp.float64, copy=False)
    stds   = stds.astype(xp.float64, copy=False)
    weights= weights.astype(xp.float64, copy=False)

    N, D = x.shape
    if N == 0:
        return xp.zeros((0,), dtype=xp.float64)

    # Normalize mixture weights safely
    wsum = weights.sum()
    if wsum <= 0:
        raise ValueError("Mixture weights must sum to a positive value.")
    w = weights / wsum
    log_w = xp.log(w + 1e-40)  # [C]

    # Handle variances and normalization constants
    if stds.ndim == 1:
        # Isotropic per-component: Σ_c = (std_c^2) * I
        inv_vars = 1.0 / (stds**2)                 # [C]
        log_det_vars = D * xp.log(stds**2)         # [C]  (since det = (std^2)^D)
    elif stds.ndim == 2 and stds.shape[1] == D:
        # Diagonal per-component: Σ_c = diag(std_{c,1}^2, ..., std_{c,D}^2)
        inv_vars = 1.0 / (stds**2)                 # [C, D]
        log_det_vars = xp.log(stds**2).sum(axis=1) # [C]  (since det = prod_d std_{c,d}^2)
    else:
        raise ValueError("`stds` must have shape (C,) or (C, D).")

    const = -0.5 * D * xp.log(2.0 * xp.pi)         # scalar
    norm_const = const - 0.5 * log_det_vars        # [C]

    out = xp.empty((N,), dtype=xp.float64)

    for i in range(0, N, batch_size):
        xb = x[i:i+batch_size]                     # [B, D]
        diff = xb[:, None, :] - means[None, :, :]  # [B, C, D]

        if inv_vars.ndim == 1:
            # quadratic term = sum_d (diff^2) * (1/std_c^2)
            d2 = (diff**2).sum(axis=-1)            # [B, C]
            quad = d2 * inv_vars[None, :]          # [B, C]
        else:
            # quadratic term = sum_d (diff^2 / std_{c,d}^2)
            quad = ((diff**2) * inv_vars[None, :, :]).sum(axis=-1)  # [B, C]

        # log component densities (including weights)
        log_comp = log_w[None, :] + norm_const[None, :] - 0.5 * quad  # [B, C]

        # log-sum-exp over components for numerical stability
        m = xp.max(log_comp, axis=1, keepdims=True)                   # [B, 1]
        log_mix = xp.squeeze(m, 1) + xp.log(xp.exp(log_comp - m).sum(axis=1))  # [B]

        if mode == "absolute":
            out[i:i+batch_size] = xp.exp(log_mix)                     # p_mix(x)
        elif mode == "relative":
            s2 = float(iso_std) ** 2
            # log phi_iso(x) for N(0, s2 I)
            log_iso = -0.5 * D * xp.log(2.0 * xp.pi * s2) - 0.5 * (xb**2).sum(axis=1) / s2  # [B]
            out[i:i+batch_size] = xp.exp(log_mix - log_iso)           # p_mix(x)/phi_iso(x)
        else:
            raise ValueError("mode must be 'absolute' or 'relative'.")

    return out
import cupy as cp


def get_ou_evolved_gmm_params(t, means0, stds0, w0):
    xp = cp
    means0 = xp.asarray(means0)
    e = xp.exp(-t)
    e2 = e * e

    # Evolved means
    m_t = e * means0

    S0 = xp.asarray(stds0)
    K = means0.shape[0]
    D = means0.shape[1]

    # --- Full covariance cases ---
    if (S0.ndim == 3 and S0.shape[-1] == S0.shape[-2]) or (S0.ndim == 2 and S0.shape[0] == S0.shape[1]):
        # Ensure shape (K, D, D)
        if S0.ndim == 2:  # (D, D) for all components
            S0 = xp.broadcast_to(S0[None, :, :], (K, D, D))
        # Σ_t = e^{-2t} Σ_0 + (1 - e^{-2t}) I
        eye = xp.eye(D, dtype=S0.dtype)[None, :, :]  # (1, D, D)
        Sigma_t = e2 * S0 + (1.0 - e2) * eye
        return m_t, Sigma_t, w0

    # --- Diagonal / isotropic std cases ---
    # Treat S0 as stds; compute variances elementwise with broadcasting
    v0 = S0 * S0
    v_t = e2 * v0 + (1.0 - e2)  # adds isotropic I term
    std_t = xp.sqrt(v_t)
    return m_t, std_t, w0


def OU_evolve_samples(y_cp, t_scalar):
    import cupy as cp
    y = cp.asarray(y_cp)
    t = float(t_scalar)
    et = cp.exp(-t)
    sigma = cp.sqrt(cp.maximum(1.0 - et*et, 0.0))
    z = cp.random.standard_normal(y.shape, dtype=y.dtype)
    return et * y + sigma * z


def samples_to_sampler(samples):
    samples = np.asarray(samples)
    N = samples.shape[0]
    def sampler(n):
        if n <= N:
            return samples[:n].copy()
        k = n // N; r = n % N
        out = [samples for _ in range(k)]
        if r: out.append(samples[:r])
        return np.concatenate(out, axis=0).copy()
    return sampler


import numpy as np
try:
    import cupy as cp
except Exception:
    cp = None

import numpy as np
try:
    import cupy as cp
except Exception:
    cp = None



def normalize_gmm_params(means, stds, weights, *, mode="global", eps=1e-12):
    """
    Center & scale a GMM so the mixture mean is 0 and the marginal variance is 1.

    Inputs
    ------
    means   : (C, D)
    stds    : (C,)  or (C, D)     # isotropic or diagonal per component (std, not var)
    weights : (C,)
    mode    : "global" -> one scalar scale s so average marginal variance == 1
              "per_dim" -> vector scale s_d so EACH marginal variance == 1

    Returns
    -------
    means_n : (C, D)
    stds_n  : (C,)  if input was (C,) and mode="global"
              (C,D) if mode="per_dim" or input was (C,D)
    weights_n : (C,)   (re-normalized to sum to 1)
    """
    # pick backend
    try:
        import cupy as cp
        xp = cp if isinstance(means, cp.ndarray) else __import__("numpy")
    except Exception:
        import numpy as np
        xp = np

    m = xp.asarray(means,   dtype=xp.float64)
    s = xp.asarray(stds,    dtype=xp.float64)
    w = xp.asarray(weights, dtype=xp.float64)

    # normalize weights
    w = xp.clip(w, 0, xp.inf)
    ws = w.sum()
    if not xp.isfinite(ws) or ws <= 0:
        raise ValueError("weights must sum to a positive value.")
    w = w / ws

    C, D = m.shape

    # per-dimension variance contributions
    if s.ndim == 1:          # isotropic per component
        var_terms = (s**2)[:, None]                # (C,1) -> broadcast to (C,D)
    elif s.ndim == 2 and s.shape[1] == D:
        var_terms = s**2                            # (C,D)
    else:
        raise ValueError("stds must have shape (C,) or (C,D).")

    # mixture stats
    mu_mix  = w @ m                                  # (D,)
    Ex2     = (w[:, None] * (var_terms + m**2)).sum(axis=0)  # (D,)
    var_mix = xp.maximum(Ex2 - mu_mix**2, eps)       # (D,)

    if mode == "global":
        scale = xp.sqrt(var_mix.mean())              # scalar s
        m_n = (m - mu_mix[None, :]) / scale
        s_n = s / scale                              # keeps shape of s
    elif mode == "per_dim":
        scale = xp.sqrt(var_mix)                     # (D,)
        m_n = (m - mu_mix[None, :]) / scale[None, :]
        if s.ndim == 1:
            s_n = s[:, None] / scale[None, :]        # becomes diagonal (C,D)
        else:
            s_n = s / scale[None, :]
    else:
        raise ValueError('mode must be "global" or "per_dim".')

    return (m_n.astype(means.dtype, copy=False),
            s_n.astype(stds.dtype,  copy=False),
            w.astype(weights.dtype, copy=False))




def get_gmm_funcs(
    num_c: int = 200,
    k_dim: int = 5,
    variant: str = "helix",
    comp_std: float = 0.10,
    overall_scale: float = 1.0,
    rs=None,
    m_dim: int = 3,
    powerlaw_weights: bool = True,
    seed: int = 0,
    stds_specified = None,
    weights_specified = None,
    preset_params = None,
    normalize = False,
    size = 1,
    embedding_mode: str = "linear"  # NEW
):
    # ---- RNG (seeded if requested) -----------------------------------------
    if rs is not None and not isinstance(rs, np.random.RandomState):
        raise TypeError("rs must be a numpy.random.RandomState")
    rng = rs if rs is not None else (np.random.RandomState(seed) if seed is not None
                                     else np.random.RandomState())

    xp = cp
    m_embed = m_dim  # intrinsic dimension placeholder; updated by some variants

    # -------------------------- helpers (NEW) -------------------------------
    def _qr_tall(k, m):
        """Q in R^{k×m} with orthonormal columns (k >= m)."""
        Q, _ = np.linalg.qr(rng.randn(k, m))
        return Q[:, :m]

    def _qr_wide(m, k):
        """A in R^{m×k} with orthonormal columns (k <= m)."""
        G = rng.randn(m, k)
        # Orthonormalize columns of G via SVD
        U, _, Vt = np.linalg.svd(G, full_matrices=False)
        return (U @ Vt)  # shape (m×k), columns orthonormal

    def _orthonormal_complement(Q, k, s):
        """Given Q (k×m) with orthonormal cols, return Q_perp (k×s) orthonormal & Q^T Q_perp = 0."""
        if s <= 0:
            return None
        G = rng.randn(k, s)
        G = G - Q @ (Q.T @ G)
        Qp, _ = np.linalg.qr(G)
        return Qp[:, :s]

    def _embed_linear(X_intr):
        # Preserve original behavior (branch uses m_dim, not m_embed)
        if k_dim > m_dim:
            Q = _qr_tall(k_dim, X_intr.shape[1])  # k×m_embed
            Z = X_intr @ Q.T                      # num_c×k
            covs = np.array([ (comp_std**2) * (Q @ Q.T) + (1e-4*comp_std)**2*np.eye(k_dim)
                              for _ in range(num_c) ])
        else:
            Z = X_intr[:, :k_dim]
            covs = np.array([ (comp_std**2) * np.eye(k_dim) for _ in range(num_c) ])
        return Z, covs

    def _embed_sine_wiggle(X_intr, amp=0.25, freq=1.0):
        m = X_intr.shape[1]
        # base linear map into k-dim
        if k_dim >= m:
            Q = _qr_tall(k_dim, m)               # k×m
            Zlin = X_intr @ Q.T                  # num_c×k
            # add wiggle in orthogonal directions
            s = min(max(1, m), max(0, k_dim - m))
            Q_perp = _orthonormal_complement(Q, k_dim, s)
            if Q_perp is not None:
                R = rng.randn(s, m) * freq
                b = rng.uniform(0.0, 2*np.pi, size=(s,))
                H = np.sin(X_intr @ R.T + b)     # num_c×s
                Zwig = H @ Q_perp.T              # num_c×k
                Z = Zlin + amp * Zwig
            else:
                Z = Zlin
        else:
            # reduce then wiggle using internal orthonormal columns
            A = _qr_wide(m, k_dim)               # m×k
            Zlin = X_intr @ A                    # num_c×k
            # small nonlinear bump inside the current k-subspace
            R = rng.randn(k_dim, m) * freq
            b = rng.uniform(0.0, 2*np.pi, size=(k_dim,))
            H = np.sin(X_intr @ R.T + b)         # num_c×k
            Z = Zlin + amp * H
        covs = np.array([ (comp_std**2) * np.eye(k_dim) for _ in range(num_c) ])
        return Z, covs

    def _embed_rff(X_intr, freq=1.0):
        m = X_intr.shape[1]
        M = max(1, k_dim // 2)
        W = rng.randn(M, m) * freq
        b = rng.uniform(0.0, 2*np.pi, size=(M,))
        C = np.cos(X_intr @ W.T + b)
        S = np.sin(X_intr @ W.T + b)
        Z = np.concatenate([C, S], axis=1)
        if Z.shape[1] < k_dim:
            pad = X_intr @ rng.randn(m, k_dim - Z.shape[1])
            Z = np.concatenate([Z, pad], axis=1)
        else:
            Z = Z[:, :k_dim]
        covs = np.array([ (comp_std**2) * np.eye(k_dim) for _ in range(num_c) ])
        return Z, covs

    def _embed_radial(X_intr):
        r = np.linalg.norm(X_intr, axis=1, keepdims=True)
        feats = [X_intr, r, r**2, np.sin(r), np.cos(r)]
        Z = np.concatenate(feats, axis=1)
        while Z.shape[1] < k_dim:
            w = rng.randn(X_intr.shape[1])
            Z = np.hstack([Z, (X_intr @ w[:, None])**2])
        Z = Z[:, :k_dim]
        covs = np.array([ (comp_std**2) * np.eye(k_dim) for _ in range(num_c) ])
        return Z, covs

    # --------------------------- custom branch ------------------------------
    if variant == "custom":
        if preset_params is not None:
            means_np, stds_np, weights_np = preset_params
        else:
            # draw random means in R^k_dim
            means_np = rng.randn(num_c, k_dim) * overall_scale
            # isotropic component stds
            stds_np  = np.full(num_c, comp_std, dtype=float)
            # random non-uniform weights
            weights_np = rng.rand(num_c)
            weights_np /= weights_np.sum()
        # move to GPU
        means   = xp.asarray(means_np, dtype=xp.float64)
        stds    = xp.asarray(stds_np,   dtype=xp.float64)
        weights = xp.asarray(weights_np, dtype=xp.float64)

        def sampler(n, seed=None, debug=False):
            return sample_gmm_gpu(n, means, stds, weights, seed=seed)

        def score_func(x):
            return score_gmm_gpu(x, means, stds, weights)

        def density_func(x):
            return density_gmm_gpu(x, means, stds, weights, mode='absolute')

        def score_div_func(x):
            _, d = score_div_gmm(x, means, stds, weights)
            return d

        params = (means, stds, weights)
        return params, sampler, score_func, density_func, score_div_func

    # ---------- 1) build intrinsic manifold in R^{m_embed} -----------------
    if variant == "helix":
        t = np.linspace(0, 4*np.pi, num_c)
        r = 1.0 + 0.25 * np.sin(3*t)
        xyz = np.stack([r*np.cos(t), r*np.sin(t), 0.4*t], axis=1)
        X_intr = xyz; m_embed = 3

    elif variant == "concentric":
        shells    = 3
        shell_id  = np.repeat(np.arange(shells),
                              int(np.ceil(num_c / shells)))[:num_c]
        radii     = 0.7 + 0.7 * shell_id
        vecs      = rng.randn(num_c, m_dim)
        vecs     /= np.linalg.norm(vecs, axis=1, keepdims=True)
        X_intr    = radii[:, None] * vecs
        m_embed   = m_dim

    elif variant == "sparse":
        xyz       = rng.uniform(-2.5, 2.5, size=(num_c, m_dim))
        if m_dim >= 2:
            xyz[:, 1] *= 0.3
        if m_dim >= 3:
            xyz[:, 2] *= 0.1
        if m_dim > 3:
            xyz[:, 3:] *= 0.05
        X_intr = xyz; m_embed = m_dim

    elif variant == "knotted_torus":
        t = np.linspace(0, 2*np.pi, num_c)
        r = 0.1
        knot_x = (2 + np.cos(5*t)) * np.cos(3*t)
        knot_y = (2 + np.cos(5*t)) * np.sin(3*t)
        knot_z = np.sin(5*t)
        base = np.stack([knot_x, knot_y, knot_z], axis=1)

        tan = np.gradient(base, axis=0)
        tan /= np.linalg.norm(tan, axis=1, keepdims=True) + 1e-12
        rnd = rng.randn(num_c, 3)
        n1 = rnd - (rnd * tan).sum(1, keepdims=True) * tan
        n1 /= np.linalg.norm(n1, axis=1, keepdims=True) + 1e-12
        n2 = np.cross(tan, n1)
        n2 /= np.linalg.norm(n2, axis=1, keepdims=True) + 1e-12

        theta  = rng.uniform(0, 2*np.pi, num_c)
        tube_r = r * (0.7 + 0.3 * np.sin(6*t + 0.5))
        xyz = base + tube_r[:, None] * (np.cos(theta)[:, None]*n1 + np.sin(theta)[:, None]*n2)
        xyz += 0.05 * rng.randn(*xyz.shape) * (0.5 + 0.5 * np.sin(8*t)[:, None])
        X_intr = xyz; m_embed = 3

    elif variant == "crossing_torus":
        # ---------------- knobs that amplify Tweedie/DSM failures ----------------
        r        = 0.08          # tube radius (keep thin to keep strands distinct)
        sig_t    = 0.10 * r      # along-tangent elongation (anisotropy ↑)
        sig_n    = 0.02 * r      # normal-plane jitter (keep small)
        pull_frac = 0.25         # how hard to snap near-crossing pairs together
        keep_frac = 1/6          # fraction of points to snap (densify crossings)
        p1, q1   = 5,  3         # knot 1 winding numbers
        p2, q2   = 7, -3         # knot 2 (opposite toroidal winding -> transversality)
        phase2   = np.pi / 7     # small phase shift -> many near-crossings
        # -------------------------------------------------------------------------

        def torus_knot(p, q, t):
            x = (2.0 + np.cos(p * t)) * np.cos(q * t)
            y = (2.0 + np.cos(p * t)) * np.sin(q * t)
            z = np.sin(p * t)
            return np.stack([x, y, z], axis=1)

        def make_tube(base, t):
            tan = np.gradient(base, axis=0)
            tan /= np.linalg.norm(tan, axis=1, keepdims=True) + 1e-12

            rnd = rng.randn(base.shape[0], 3)
            n1 = rnd - (rnd * tan).sum(1, keepdims=True) * tan
            n1 /= np.linalg.norm(n1, axis=1, keepdims=True) + 1e-12
            n2 = np.cross(tan, n1)
            n2 /= np.linalg.norm(n2, axis=1, keepdims=True) + 1e-12

            theta  = rng.uniform(0, 2*np.pi, base.shape[0])
            # slight modulation keeps visual texture but equalizes density overall
            tube_r = r * (0.7 + 0.3 * np.cos(6 * t))

            xyz = base + tube_r[:, None] * (
                np.cos(theta)[:, None] * n1 + np.sin(theta)[:, None] * n2
            )
            # anisotropic jitter: long along tangent, tiny in normals
            xyz += sig_t * rng.randn(base.shape[0], 1) * tan
            xyz += sig_n * rng.randn(base.shape[0], 1) * n1
            xyz += sig_n * rng.randn(base.shape[0], 1) * n2
            return xyz

        # split components between two strands
        k  = num_c // 2
        t1 = np.linspace(0, 2*np.pi, k, endpoint=False)
        t2 = np.linspace(0, 2*np.pi, k, endpoint=False)

        base1 = torus_knot(p1, q1, t1)
        base2 = torus_knot(p2, q2, t2 + phase2)

        xyz1 = make_tube(base1, t1)
        xyz2 = make_tube(base2, t2)

        # --- concentrate mass where strands approach: snap closest pairs together ---
        # (This makes the posterior most bimodal -> Tweedie averaging worst.)
        # For large k, this is ~k^2 memory; if needed, subsample before computing dmat.
        dmat = np.linalg.norm(xyz1[:, None, :] - xyz2[None, :, :], axis=2)
        i_close = np.argpartition(dmat.min(axis=1), int(k * keep_frac))[: int(k * keep_frac)]
        j_close = np.argmin(dmat[i_close], axis=1)

        mid = 0.5 * (xyz1[i_close] + xyz2[j_close])
        xyz1[i_close] = mid + (1.0 - pull_frac) * (xyz1[i_close] - mid)
        xyz2[j_close] = mid + (1.0 - pull_frac) * (xyz2[j_close] - mid)
        # ----------------------------------------------------------------------------

        xyz = np.concatenate([xyz1, xyz2], axis=0)

        # if num_c is odd, pad one more point from strand 1
        if xyz.shape[0] < num_c:
            xyz = np.concatenate([xyz, make_tube(base1[:1], t1[:1])], axis=0)

        X_intr  = xyz
        m_embed = 3
    else:
        raise ValueError("unknown variant")

    # ---------- 2) embed into R^{k_dim} per embedding_mode -----------------
    if embedding_mode == "linear":
        means_np, covs = _embed_linear(X_intr)
    elif embedding_mode == "sine_wiggle":
        means_np, covs = _embed_sine_wiggle(X_intr, amp=0.25, freq=1.0)
    elif embedding_mode == "rff":
        means_np, covs = _embed_rff(X_intr, freq=1.0)
    elif embedding_mode == "radial":
        means_np, covs = _embed_radial(X_intr)
    else:
        raise ValueError(f"unknown embedding_mode: {embedding_mode}")

    # ---------- 3) center & scale ------------------------------------------
    means_np *= overall_scale / np.max(np.abs(means_np))
    means  = xp.asarray(means_np, dtype=xp.float64)

    # ---------- 4) stds/weights (respect variant-provided if any) ----------
    if not stds_specified:
        stds_np    = np.full(num_c, comp_std, dtype=float)
    if not weights_specified:
        weights_np = np.full(num_c, 1.0/num_c, dtype=float)

    stds    = xp.asarray(stds_np, dtype=xp.float64)
    weights = xp.asarray(weights_np, dtype=xp.float64)

    # Optional normalization to a target size box
    if normalize:
        means, stds, weights = normalize_gmm_params(means, stds, weights)
        means, stds = size * means, size * stds

    # ---------- 5) public API: sampler/score/density -----------------------
    def sampler(n, seed=None):
        return sample_gmm_gpu(n, means, stds, weights, seed=seed)

    def score_func(x):
        return score_gmm_gpu(x, means, stds, weights)

    def density_func(x):
        return density_gmm_gpu(x, means, stds, weights, mode='absolute')

    def score_div_func(x):
        _, d = score_div_gmm(x, means, stds, weights)
        return d

    params = (means, stds, weights)  # covs computed above if you need it internally
    return params, sampler, score_func, density_func, score_div_func


def _rand_orth(d, k, seed=None):
    rs = cp.random.RandomState(seed) if seed is not None else cp.random
    G = rs.standard_normal((d, k), dtype=cp.float64)
    # thin QR, make orthonormal columns
    Q, _ = cp.linalg.qr(G, mode='reduced')
    return Q  # d x k


def embed_base(dist, d: int, *, sigma_perp=0.0, rotate=True, seed=None):
    """
    dist: object with 2 callables
        - dist['sample_k'](n, seed) -> cp.ndarray [n,k]
        - dist['score_k'](Z)        -> cp.ndarray [n,k]
    Returns (sampler_func, score_func) on R^d.

    Embedding: x = Q_k z  +  Q_perp eps,  eps ~ N(0, sigma_perp^2 I_{d-k})
    Score:     s(x) = Q_k s_z  +  Q_perp s_eps,  s_eps = -(1/sigma_perp^2) eps
    """
    k = dist['sample_k'](1, seed=seed).shape[1]
    assert 1 <= k <= d
    if rotate:
        Qk = _rand_orth(d, k, seed=seed)
        # Build an orthonormal complement (cheap Gram-Schmidt on random)
        Qp = _rand_orth(d, d, seed=(None if seed is None else seed+1))
        # Re-orth so [Qk | Qperp] spans R^d with Qk preserved
        # Householder trick: project Qp off Qk
        Qp = Qp - Qk @ (Qk.T @ Qp)
        # Orthonormalize columns and take first d-k
        Qp, _ = cp.linalg.qr(Qp, mode='reduced')
        Qp = Qp[:, :d-k] if d>k else cp.zeros((d,0), dtype=cp.float64)
    else:
        Qk = cp.pad(cp.eye(k, dtype=cp.float64), ((0, d-k),(0,0)))  # top k of I_d
        Qp = cp.pad(cp.eye(d-k, dtype=cp.float64), ((k,0),(0,0)))

    sigma_perp = float(sigma_perp)
    inv_perp2  = 0.0 if sigma_perp==0.0 else 1.0/(sigma_perp**2)

    def sampler_func(n, seed=None):
        Z = dist['sample_k'](int(n), seed=seed)                  # [n,k]
        if d == k:
            return (Qk @ Z.T).T
        rs = cp.random.RandomState(seed) if seed is not None else cp.random
        eps = rs.standard_normal((int(n), d-k), dtype=cp.float64) * sigma_perp
        X = (Qk @ Z.T).T + (Qp @ eps.T).T
        return X

    def score_func(X):
        X = cp.asarray(X, dtype=cp.float64)
        Z   = (Qk.T @ X.T).T                                     # [n,k]
        s_k = dist['score_k'](Z)                                 # [n,k]
        if d == k:
            return (Qk @ s_k.T).T
        eps = (Qp.T @ X.T).T                                     # [n,d-k]
        s_perp = -inv_perp2 * eps
        S = (Qk @ s_k.T).T + (Qp @ s_perp.T).T
        return S

    return sampler_func, score_func


def make_banana_2d(beta=0.25, a=1.0, sigma1=1.0, sigma2=0.1):
    def sample_k(n, seed=None):
        rs = cp.random.RandomState(seed) if seed is not None else cp.random
        z1 = rs.standard_normal(int(n)) * sigma1
        u  = rs.standard_normal(int(n)) * sigma2
        z2 = u - beta*(z1**2 - a)
        return cp.stack([z1, z2], axis=1).astype(cp.float64)

    def score_k(Z):
        Z = cp.asarray(Z, dtype=cp.float64)
        z1, z2 = Z[:,0], Z[:,1]
        y = z2 + beta*(z1*z1 - a)
        s1 = - z1/(sigma1**2) - (2*beta*z1*y)/(sigma2**2)
        s2 = - y/(sigma2**2)
        return cp.stack([s1, s2], axis=1)
    return {'sample_k': sample_k, 'score_k': score_k}


def make_ring_2d(R=3.0, sigma_r=0.1, with_jacobian=True):
    def sample_k(n, seed=None):
        rs = cp.random.RandomState(seed) if seed is not None else cp.random
        theta = rs.uniform(0, 2*cp.pi, int(n))
        r = R + rs.standard_normal(int(n)) * sigma_r
        z = cp.stack([r*cp.cos(theta), r*cp.sin(theta)], axis=1)
        return z.astype(cp.float64)

    def score_k(Z):
        Z = cp.asarray(Z, dtype=cp.float64)
        r = cp.linalg.norm(Z, axis=1) + 1e-12
        coef = -(r - R)/(sigma_r**2)
        if with_jacobian:
            coef = coef + 1.0/r
        return (coef[:,None] * (Z / r[:,None])).astype(cp.float64)

    return {'sample_k': sample_k, 'score_k': score_k}



def _xp_random(seed=None, use_cupy=True):
    if use_cupy and cp is not None:
        if seed is not None: cp.random.seed(seed)
        return cp
    rng = np.random.RandomState(seed)
    class NPShim:
        random = rng
        def asarray(self, x, dtype=None): return np.asarray(x, dtype=dtype)
        def zeros(self, shape, dtype=None): return np.zeros(shape, dtype=dtype)
        def ones(self, shape, dtype=None): return np.ones(shape, dtype=dtype)
        def exp(self, x): return np.exp(x)
        def sin(self, x): return np.sin(x)
        def cos(self, x): return np.cos(x)
        def dot(self, a,b): return a @ b
        def maximum(self, a,b): return np.maximum(a,b)
        def minimum(self, a,b): return np.minimum(a,b)
        def sqrt(self, x): return np.sqrt(x)
        def where(self, cond, a, b): return np.where(cond, a, b)
        def eye(self, n): return np.eye(n)
        def stack(self, arrs, axis=0): return np.stack(arrs, axis=axis)
        def concatenate(self, arrs, axis=0): return np.concatenate(arrs, axis=axis)
        def norm(self, x, axis=None): return np.linalg.norm(x, axis=axis)
    return NPShim()

def make_corrugated_gaussian(k=6, r=None, omega=80.0, c=0.6, use_cupy=True, seed=0):
    """
    Toy distribution in R^k with 'r' sinusoidal corrugations along orthonormal directions.
    Returns a dict with 'sample_k' and 'score_k' just like the banana/ring toys.

    log p(x) = -1/2 ||x||^2  - (c/omega) * sum_{j=1}^r sin(omega v_j^T x)
    ⇒ score(x) = -x - c * sum_j cos(omega v_j^T x) v_j
    The likelihood ratio to N(0,I) is bounded by exp(± r*c/omega) (so no huge spikes if r*c/omega is modest).
    """
    # pick backend RNG/array module (your helper keeps cp when available)
    xp = _xp_random(seed=seed, use_cupy=use_cupy)
    if r is None:
        r = min(3, k)  # default: a few ripples but <= k
    assert 1 <= r <= k, "require 1 <= r <= k"

    # Build V ∈ R^{k×r} with orthonormal columns in the SAME array library
    # Generate Gaussian (k×r), QR -> V
    if xp is cp:
        W = xp.random.randn(k, r)
        Q, _ = xp.linalg.qr(W, mode='reduced')   # (k,r)
        V = Q
    else:
        W = np.random.RandomState(seed).randn(k, r)
        Q, _ = np.linalg.qr(W)                   # (k,r)
        V = xp.asarray(Q)

    omega = float(omega); c = float(c)
    rho = c / omega   # appears in the target potential, controls closeness to Gaussian

    def _proj(X):  # (n,k) -> (n,r)
        return X @ V

    # exact score in R^k
    def score_k(Z):
        Z = xp.asarray(Z, dtype=xp.float64)
        U = _proj(Z)                              # (n,r)
        C = xp.cos(omega * U)                     # (n,r)
        return (-Z - (c * C) @ V.T).astype(xp.float64)

    # rejection sample from N(0,I) with envelope exp(r * rho)
    # acceptance ≥ exp(-r*rho) and no host/device copying
    def sample_k(n, seed=None, batch=8192):
        xp_loc = _xp_random(seed=seed, use_cupy=use_cupy)
        out = []
        need = int(n)
        logC = r * rho
        while need > 0:
            m = max(batch, need)
            Z = xp_loc.random.randn(m, k).astype(xp.float64)
            U = Z @ V                              # (m,r)
            w = xp_loc.exp(-rho * xp_loc.sin(omega * U)).prod(axis=1)
            acc = w / xp_loc.exp(logC)
            u = xp_loc.random.rand(m)
            keep = u < acc
            if keep.sum() > 0:
                out.append(Z[keep])
                need -= int(keep.sum())
        return xp.concatenate(out, axis=0)[:n]

    return {'sample_k': sample_k, 'score_k': score_k}



def make_funnel(m):
    d = m + 1
    def sample_k(n, seed=None):
        rs = cp.random.RandomState(seed) if seed is not None else cp.random
        z  = rs.standard_normal(int(n))
        x  = rs.standard_normal((int(n), m)) * cp.exp(z)[:,None]
        return cp.concatenate([x, z[:,None]], axis=1).astype(cp.float64)

    def score_k(Z):
        Z = cp.asarray(Z, dtype=cp.float64)
        x, z = Z[:, :m], Z[:, m]
        ezm2 = cp.exp(-2.0*z)
        s_x  = - (ezm2[:,None] * x)
        s_z  = - z + ezm2 * cp.sum(x*x, axis=1) - m
        return cp.concatenate([s_x, s_z[:,None]], axis=1)
    return {'sample_k': sample_k, 'score_k': score_k}



def make_student_t(k, nu=5.0, Sigma=None, seed=None):
    rs = cp.random.RandomState(seed) if seed is not None else cp.random
    if Sigma is None:
        # strong anisotropy
        scales = cp.asarray(cp.logspace(np.log10(0.1), np.log10(5.0), num=k), dtype=cp.float64)
        L = cp.diag(scales)
        Sinv = cp.diag(1.0/(scales**2))
    else:
        S = cp.asarray(Sigma, dtype=cp.float64)
        L = cp.linalg.cholesky(S)
        Sinv = cp.linalg.inv(S)

    def sample_k(n, seed=None):
        rng = cp.random.RandomState(seed) if seed is not None else cp.random
        eps = rng.standard_normal((int(n), k), dtype=cp.float64)
        g   = rng.chisquare(df=nu, size=int(n)).astype(cp.float64)
        x   = (eps @ L.T) / cp.sqrt(g/nu)[:,None]
        return x

    def score_k(Z):
        Z = cp.asarray(Z, dtype=cp.float64)
        Mah = cp.einsum('nd,dd,nd->n', Z, Sinv, Z)           # x^T S^{-1} x
        coef = - (nu + k) / (nu + Mah)
        return coef[:,None] * (Z @ Sinv.T)
    return {'sample_k': sample_k, 'score_k': score_k}



def make_k_rotated_banana(K=6, beta=0.25, a=1.0, sigma1=1.0, sigma2=0.1):
    base = make_banana_2d(beta, a, sigma1, sigma2)
    thetas = cp.linspace(0, 2*cp.pi, K, endpoint=False)
    Rks = cp.stack([cp.stack([cp.cos(t), -cp.sin(t)], axis=0)
                    for t in thetas], axis=0)  # [K,2]
    Rks = cp.stack([cp.stack([Rks[i], cp.stack([cp.sin(thetas[i]), cp.cos(thetas[i])])],axis=0)
                    for i in range(K)], axis=0)  # [K,2,2]

    def sample_k(n, seed=None):
        rs = cp.random.RandomState(seed) if seed is not None else cp.random
        idx = rs.randint(0, K, size=int(n))
        Z   = base['sample_k'](int(n), seed=None)
        # rotate each sample by chosen R_k
        X = cp.einsum('nij,nj->ni', Rks[idx], Z)
        return X

    def _logp_k(Z):
        # compute log p_k for each k by un-rotating into banana frame
        # log p up to constant from base form
        Z = cp.asarray(Z, dtype=cp.float64)
        Y = cp.einsum('kij,nj->nki', Rks.transpose(0,2,1), Z)  # [n,K,2]
        z1, z2 = Y[...,0], Y[...,1]
        y = z2 + beta*(z1*z1 - a)
        lp = -0.5*(z1*z1/(sigma1**2) + y*y/(sigma2**2))  # [n,K]
        return lp

    def score_k(Z):
        Z = cp.asarray(Z, dtype=cp.float64)
        lp = _logp_k(Z)                       # [n,K]
        m  = lp.max(axis=1, keepdims=True)
        W  = cp.exp(lp - m); W /= W.sum(1, keepdims=True) + 1e-30
        # component scores in x-space: rotate base score
        Y = cp.einsum('kij,nj->nki', Rks.transpose(0,2,1), Z)         # un-rotate
        sb = base['score_k'](Y.reshape(-1,2)).reshape(Z.shape[0], K, 2)
        sx = cp.einsum('kij,nkj->nki', Rks, sb)                       # rotate back
        s  = cp.einsum('nk,nki->ni', W, sx)                           # mixture score
        return s

    return {'sample_k': sample_k, 'score_k': score_k}



def make_folded_2d(c=1.0):
    def sample_k(n, seed=None):
        rs = cp.random.RandomState(seed) if seed is not None else cp.random
        u1 = rs.standard_normal(int(n))
        u2 = rs.standard_normal(int(n))
        z1 = u1
        z2 = u2 + c*cp.abs(u1)
        return cp.stack([z1, z2], axis=1).astype(cp.float64)

    def score_k(Z):
        Z = cp.asarray(Z, dtype=cp.float64)
        z1, z2 = Z[:,0], Z[:,1]
        y = z2 - c*cp.abs(z1)
        s2 = -y
        s1 = -z1 - y*c*cp.sign(z1)
        return cp.stack([s1, s2], axis=1)
    return {'sample_k': sample_k, 'score_k': score_k}



def get_spiky_gmm_funcs(
    num_c=200, k_dim=6, base_std=0.15, spike_std=0.001, spike_frac=0.1, seed=0
):
    rng = np.random.RandomState(seed)
    C_spike = max(1, int(num_c * spike_frac))
    C_base  = num_c - C_spike

    means_base = rng.randn(C_base, k_dim)
    means_spk  = rng.randn(C_spike, k_dim)*0.5 + 3.0*rng.choice([-1,1], size=(C_spike,k_dim))
    means = np.concatenate([means_base, means_spk], 0)

    stds  = np.concatenate([np.full(C_base, base_std), np.full(C_spike, spike_std)])
    w     = np.full(num_c, 1.0/num_c)

    means = cp.asarray(means, dtype=cp.float64)
    stds  = cp.asarray(stds,  dtype=cp.float64)
    w     = cp.asarray(w,     dtype=cp.float64)

    def sampler(n, seed=None):
        return sample_gmm_gpu(n, means, stds, w, seed=seed)
    def score(x): return score_gmm_gpu(x, means, stds, w)
    return sampler, score


def make_pathological_gmm(
    C=500, d=39, sep=5.0, cov_scale=0.05, seed=0):
    rng = np.random.RandomState(seed)
    # place means on a d‐dim hypercube corners or random large coordinates
    means = rng.randn(C, d) * sep
    # tiny isotropic covariances
    stds = np.full(C, cov_scale)
    # nonuniform weights
    weights = rng.rand(C)
    weights /= weights.sum()
    # wrap for existing API
    params = (means, stds, weights)
    sampler, score = get_gmm_funcs(
        num_c=C, k_dim=d, variant='custom', comp_std=cov_scale,
        overall_scale=sep, rs=rng)
    return params, sampler, score


def resample_by_alpha(X, alpha, N, return_indices=False, rng=None):
    xp = np
    is_cupy = type(X).__module__.split('.')[0] == 'cupy' or type(alpha).__module__.split('.')[0] == 'cupy'
    if is_cupy:
        import cupy as cp
        xp = cp

    X = xp.asarray(X)
    alpha = xp.asarray(alpha).reshape(-1)
    M = alpha.size
    if X.shape[0] != M:
        raise ValueError(f"Length mismatch: len(alpha)={M} but X.shape[0]={X.shape[0]}")

    # Normalize weights safely
    w = xp.clip(alpha, 0, xp.inf).astype(xp.float64)
    w_sum = w.sum()
    if not xp.isfinite(w_sum) or w_sum <= 0:
        # Fallback: uniform sampling if weights are invalid or all zero
        p = xp.full(M, 1.0 / M, dtype=xp.float64)
    else:
        p = w / w_sum

    # Draw indices
    if rng is None:
        idx = xp.random.choice(M, size=int(N), replace=True, p=p)
    else:
        # rng must have a .choice method (NumPy/CuPy Generator)
        idx = rng.choice(M, size=int(N), replace=True, p=p)

    Y = X[idx]
    return (Y, idx) if return_indices else Y

# --- Rank-k linear likelihood on GPU (CuPy) ---



def _orthonormal_rows_A(D, k, seed=None):
    """
    Build A with k orthonormal rows in R^{k x D}.
    Uses QR on a (D x k) Gaussian and transposes.
    """
    rs = cp.random.RandomState(int(seed)) if (seed is not None and seed != 'rand') else cp.random
    G = rs.standard_normal((D, k), dtype=cp.float64)      # (D,k)
    Q, _ = cp.linalg.qr(G, mode='reduced')                # (D,k) with orthonormal columns
    A = Q.T                                               # (k,D) with orthonormal rows
    return A.astype(cp.float64, copy=False)

def make_rank_k_likelihood(
    D, rank_k, obs_sigma,
    sampler_func=None,            # optional: to synthesize y as A x* + eps
    y_obs=None,                   # optional: provide your own observed y in R^k
    seed=None
):
    """
    Build rank-k linear Gaussian likelihood:
        log p(y | x) = - (1/(2 sigma^2)) || y - A x ||^2
        grad_x log p  = (1/sigma^2) A^T ( y - A x )

    Returns:
      A              : (k, D) CuPy array
      y_obs_k        : (k,)   CuPy array
      likelyhood_func: (N,D) -> (N,)  log-lik up to additive const
      log_likelyhood_func: alias to likelyhood_func
      loglik_grad_fn : (N,D) -> (N,D)
    """
    k = int(rank_k)
    assert 1 <= k <= D, f"rank_k must be in [1, {D}]"
    A = _orthonormal_rows_A(D, k, seed=seed)             # (k,D)
    inv_var = cp.asarray(1.0 / (obs_sigma * obs_sigma), dtype=cp.float64)
    alpha = 0.5 * inv_var                                 # = 1/(2 sigma^2)

    if y_obs is None:
        if sampler_func is None:
            raise ValueError("Provide sampler_func to synthesize y_obs or pass y_obs directly.")
        x_star = sampler_func(1, seed=seed).reshape(-1)   # (D,)
        eps = cp.random.normal(0.0, obs_sigma, size=(k,)).astype(cp.float64)
        y_obs_k = (A @ x_star.astype(cp.float64)) + eps   # (k,)
    else:
        y_obs_k = cp.asarray(y_obs, dtype=cp.float64).reshape(k)
        # if user passed a D-dim y, project it:
        if y_obs_k.size == D:
            y_obs_k = (A @ y_obs_k)

    def likelyhood_func(x_ref):
        """
        x_ref: (N,D) -> returns log-likelihood up to constant: -(1/(2σ^2)) ||y - A x||^2
        """
        X = cp.asarray(x_ref, dtype=cp.float64)
        Ax = X @ A.T                                      # (N,k)
        resid = y_obs_k[None, :] - Ax                     # (N,k)
        return -alpha * cp.sum(resid * resid, axis=1)     # (N,)

    def loglik_grad_fn(x_ref):
        """
        x_ref: (N,D) -> gradient wrt x: (1/σ^2) A^T (y - A x)
        """
        X = cp.asarray(x_ref, dtype=cp.float64)
        Ax = X @ A.T                                      # (N,k)
        resid = y_obs_k[None, :] - Ax                     # (N,k)
        return (inv_var * (resid @ A))                    # (N,D)

    # keep your API surface:
    def log_likelyhood_func(x_ref):
        # already a log-likelihood; return it directly (do NOT take cp.log again)
        return likelyhood_func(x_ref)

    return A, y_obs_k, likelyhood_func, log_likelyhood_func, loglik_grad_fn


def calculate_true_score_at_t(y, t, means0, stds0, w0, batch_size):
    m, s, w = get_ou_evolved_gmm_params(t, means0, stds0, w0)
    return score_gmm_gpu(y, m, s, w, batch_size)


def ou_logweights_with_lik(x, t, x_ref, loglik_ref=None):
    """
    x:      [B, d] current particles (y_t in your notation)
    x_ref:  [N, d] reference particles ~ p(x0)
    t:      float
    loglik_ref: [N] log p(y | x_ref) (any constant offset OK)

    returns: normalized weights wbar [B, N], plus cached scalars
    """
    xp = cp
    et  = xp.exp(-t)
    sig2 = 1.0 - xp.exp(-2*t)             # sigma_t^2

    # pairwise squared distances ||x - e^{-t} x_i||^2
    diff = x[:, None, :] - et * x_ref[None, :, :]
    sq   = xp.sum(diff*diff, axis=-1)     # [B, N]

    logw = -0.5 * sq / sig2               # OU kernel (const cancels)
    if loglik_ref is not None:
        logw = logw + loglik_ref[None, :] # add the likelihood term

    # stabilize & normalize
    m = xp.max(logw, axis=1, keepdims=True)
    w = xp.exp(logw - m)
    wbar = w / xp.sum(w, axis=1, keepdims=True)  # [B, N]
    return wbar, et, sig2


def gaussian_obs_loglik(x_ref, y, alpha):
    diff = x_ref - y[None, :]
    return -alpha * cp.sum(diff*diff, axis=1)     # [N]


# --- helper (drop near your other utils) ------------------------------------
def _resolve_loglik(x_ref, xp, loglik_ref=None, loglik_fn=None):
    """Return xp.ndarray of shape [N] with per-ref log-likelihoods, or None."""
    if loglik_fn is not None:
        ll = loglik_fn(x_ref)
    else:
        ll = loglik_ref
    if ll is None:
        return None
    ll = xp.asarray(ll).reshape(-1)
    assert ll.shape[0] == x_ref.shape[0], "loglik_ref must have shape [N]"
    return ll


def kss_score_t(y_batch, t, x_ref, s_ref,
                loglik_ref=None, loglik_fn=None,
                w_correct=1.0,
                grad_loglik_ref=None, grad_loglik_fn=None):
    xp = cp
    ll = _resolve_loglik(x_ref, xp, loglik_ref, loglik_fn)

    # build posterior-at-0 reference score if gradient supplied
    if grad_loglik_ref is None and grad_loglik_fn is not None:
        grad_loglik_ref = grad_loglik_fn(x_ref)           # [N,d]

    s_ref_eff = s_ref if grad_loglik_ref is None else (s_ref + grad_loglik_ref)

    var = 1.0 - xp.exp(-2.0*t); inv_v = 1.0/var; et = xp.exp(-t)
    diff   = y_batch[:, None, :] - et * x_ref[None, :, :]
    logits = -0.5 * inv_v * xp.sum(diff**2, axis=-1)
    if ll is not None:
        logits = logits + ll[None, :]

    m = xp.max(logits, axis=1, keepdims=True)
    w = xp.exp(logits - m)
    w *= w_correct
    w = w / (xp.sum(w, axis=1, keepdims=True) + 1e-30)

    return xp.exp(t) * (w @ s_ref_eff)




def tweedie_score_t(y, t, x_ref, loglik_ref=None, loglik_fn=None , w_correct = 1.0):
    xp = cp
    if t < 1e-12:
        raise ValueError("t must be >0 for Tweedie estimator.")
    ll = _resolve_loglik(x_ref, xp, loglik_ref, loglik_fn)

    B, D = y.shape
    var   = 1.0 - xp.exp(-2.0 * t)
    inv_v = 1.0 / var
    et    = xp.exp(-t)

    mu_i  = et * x_ref                                  # (N,D)
    diff  = y[:, None, :] - mu_i[None, :, :]            # (B,N,D)

    logits = -0.5 * inv_v * xp.sum(diff**2, axis=-1)    # (B,N)
    if ll is not None:
        logits = logits + ll[None, :]                   # add log-lik

    m = xp.max(logits, axis=1, keepdims=True)

    w = xp.exp(logits - m)
    w *= w_correct
    w = w / (xp.sum(w, axis=1, keepdims=True) + 1e-30)  # (B,N)


    score = - xp.einsum('bn, bnd -> bd', w, diff) * inv_v
    return score

def proxy_tweedie_score_t(y, t, mu_ref, loglik_ref=None, loglik_fn=None, w_correct=1.0):
    """
    Tweedie NP score but centered on proxy means mu_i (from local Gaussian fits),
    i.e. weights and conditional mean are computed w.r.t. e^{-t} mu_i.
    """
    xp = cp
    if t < 1e-12:
        raise ValueError("t must be >0 for Tweedie estimator.")
    Z = xp.asarray(mu_ref, dtype=xp.float64)  # basis = mu_i

    # optional likelihood values evaluated at mu_i
    if loglik_fn is not None:
        ll = loglik_fn(Z)
    else:
        ll = loglik_ref
    if ll is not None:
        ll = xp.asarray(ll).reshape(-1)

    B, D = y.shape
    var   = 1.0 - xp.exp(-2.0 * t)
    inv_v = 1.0 / var
    et    = xp.exp(-t)

    mu_i_t = et * Z                                      # (N,D)
    diff   = y[:, None, :] - mu_i_t[None, :, :]          # (B,N,D)
    logits = -0.5 * inv_v * xp.sum(diff**2, axis=-1)     # (B,N)
    if ll is not None:
        logits = logits + ll[None, :]

    m = xp.max(logits, axis=1, keepdims=True)
    w = xp.exp(logits - m)
    w *= w_correct
    w = w / (xp.sum(w, axis=1, keepdims=True) + 1e-30)   # (B,N)

    score = - xp.einsum('bn,bnd->bd', w, diff) * inv_v
    return score


def snis_blend(y_batch, x_ref, s0_ref, t, *,
               xp=None, eps=1e-12, return_details=False,
               loglik_ref=None, loglik_fn=None, w_correct=1.0,
               grad_loglik_ref=None, grad_loglik_fn=None,
               twd_basis=None):   # <-- NEW
    if xp is None:
        try: import cupy as _cp; xp = _cp
        except: import numpy as _np; xp = _np

    # Use proxy basis for Tweedie if provided; else fall back to x_ref
    Z_ref = x_ref if twd_basis is None else xp.asarray(twd_basis, dtype=xp.float64)

    # likelihood for the basis used in Tweedie weights
    ll = _resolve_loglik(Z_ref, xp, loglik_ref, loglik_fn)

    if grad_loglik_ref is None and grad_loglik_fn is not None:
        grad_loglik_ref = grad_loglik_fn(x_ref)  # still eval grad at true anchors

    s_ref_eff = s0_ref if grad_loglik_ref is None else (s0_ref + grad_loglik_ref)

    B, d = y_batch.shape
    var  = xp.maximum(1.0 - xp.exp(-2.0*t), eps)
    inv_v = 1.0/var
    et    = xp.exp(-t)

    # weights built against Z_ref (proxy means if provided)
    diff   = y_batch[:, None, :] - et * Z_ref[None, :, :]
    logits = -0.5 * inv_v * xp.sum(diff*diff, axis=-1)
    if ll is not None:
        logits = logits + ll[None, :]

    m  = xp.max(logits, axis=1, keepdims=True)
    w  = xp.exp(logits - m)
    w *= w_correct
    w /= xp.sum(w, axis=1, keepdims=True) + eps

    w2 = w * w
    S0 = xp.sum(w2, axis=1)

    # scores (KSS uses same weights w; Tweedie uses Z_ref)
    s_kss = xp.exp(t) * (w @ s_ref_eff)
    mu_x  = w @ Z_ref
    s_twd = -(inv_v) * (y_batch - et * mu_x)

    den_sn = xp.maximum(1.0 - S0, eps)

    # --- Vk ---
    a_i = xp.exp(t) * s_ref_eff
    a_norm2 = xp.sum(a_i * a_i, axis=1)
    S1a = w2 @ a_norm2
    S2a = w2 @ a_i
    mu_a = s_kss
    mu_a_norm2 = xp.sum(mu_a * mu_a, axis=1)
    num_Vk = S1a - 2.0 * xp.sum(mu_a * S2a, axis=1) + mu_a_norm2 * S0
    Vk = num_Vk / den_sn

    # --- Vt (now uses Z_ref everywhere) ---
    y_norm2 = xp.sum(y_batch*y_batch, axis=1)
    z_norm2 = xp.sum(Z_ref*Z_ref, axis=1)
    w2_z    = w2 @ Z_ref
    y_dot_zW = xp.sum(y_batch * w2_z, axis=1)
    S1b = (y_norm2 * S0 - 2.0 * et * y_dot_zW + (et**2) * (w2 @ z_norm2)) * (inv_v**2)
    S2b = -(inv_v) * (y_batch * S0[:, None] - et * w2_z)
    mu_b = s_twd
    mu_b_norm2 = xp.sum(mu_b * mu_b, axis=1)
    num_Vt = S1b - 2.0 * xp.sum(mu_b * S2b, axis=1) + mu_b_norm2 * S0
    Vt = num_Vt / den_sn

    # --- C (cross, use Z_ref in the a⋅z term) ---
    a_dot_z = xp.sum(a_i * Z_ref, axis=1)      # [N]
    Wa = w2 @ a_i
    term1 = xp.sum(Wa * y_batch, axis=1)
    term2 = w2 @ a_dot_z
    Sab = -(inv_v) * (term1 - et * term2)
    num_C = (Sab
             - xp.sum(mu_a * S2b, axis=1)
             - xp.sum(mu_b * S2a, axis=1)
             + xp.sum(mu_a * mu_b, axis=1) * S0)
    C = num_C / den_sn

    denom = xp.maximum(Vk + Vt - 2.0 * C, eps)
    lam = (Vk - C) / denom
    lam = xp.clip(lam, 0.0, 1.0)
    lam = xp.nan_to_num(lam, nan=0.5, posinf=1.0, neginf=0.0)

    if return_details:
        ess = 1.0 / (S0 + eps)
        return lam, dict(w=w, ess=ess, s_kss=s_kss, s_twd=s_twd, Vk=Vk, Vt=Vt, C=C)
    return lam



'''
def s_blend(x, t, x_ref, s_ref, loglik_ref=None, loglik_fn=None,
            loglik_grad_fn = None, w_correct = 1.0):
    s_tweedie = tweedie_score_t(x, t, x_ref,
        loglik_ref=loglik_ref, loglik_fn=loglik_fn, w_correct=w_correct)
    s_kss = kss_score_t(x, t, x_ref, s_ref, grad_loglik_fn = loglik_grad_fn,
        loglik_ref=loglik_ref, loglik_fn=loglik_fn, w_correct=w_correct)
    wt = snis_blend(x, x_ref, s_ref, t, grad_loglik_fn = loglik_grad_fn,
        loglik_ref=loglik_ref, w_correct=w_correct)
    return (1.0 - wt)[:, None] * s_kss + wt[:, None] * s_tweedie
'''


def s_blend(x, t, x_ref, s_ref, loglik_ref=None, loglik_fn=None,
            loglik_grad_fn=None, w_correct=1.0,
            twd_basis=None):   # <-- NEW (None = use x_ref; array = use mu_i)
    if twd_basis is None:
        # original path (real Tweedie + KSS)
        s_tweedie = tweedie_score_t(x, t, x_ref,
            loglik_ref=loglik_ref, loglik_fn=loglik_fn, w_correct=w_correct)
        s_kss = kss_score_t(x, t, x_ref, s_ref, grad_loglik_fn=loglik_grad_fn,
            loglik_ref=loglik_ref, loglik_fn=loglik_fn, w_correct=w_correct)
        wt = snis_blend(x, x_ref, s_ref, t, grad_loglik_fn=loglik_grad_fn,
            loglik_ref=loglik_ref, loglik_fn=loglik_fn, w_correct=w_correct)
        return (1.0 - wt)[:, None] * s_kss + wt[:, None] * s_tweedie
    else:
        # proxy path: get λ, s_kss, s_twd consistently from the same weights
        lam, det = snis_blend(x, x_ref, s_ref, t, return_details=True,
                              grad_loglik_fn=loglik_grad_fn,
                              loglik_ref=loglik_ref, loglik_fn=loglik_fn,
                              w_correct=w_correct, twd_basis=twd_basis)
        s_kss = det['s_kss']
        s_twd = det['s_twd']
        return (1.0 - lam)[:, None] * s_kss + lam[:, None] * s_twd



def gmm_posterior_params(
    y, pis, mus, Sigmas, *, A=None, b=None, R=None, alpha=None, out="full"
):
    """
    Prior:
        x ~ sum_k pi_k N(mu_k, Σ_k)
        Σ_k may be isotropic: Sigmas.shape == (K,)
              or diagonal :   Sigmas.shape == (K, D)

    Likelihood:
        y ~ N(A x + b, R)     (general case)
        or, if A,b,R omitted and alpha given: identity with sigma^2 = 1/(2*alpha).

    Returns
    -------
    if out == "full":
        w_post, means_post, covs_post        # covs_post: (K, D, D)
    elif out == "diag":
        w_post, means_post, stds_post        # stds_post: (K, D) (sqrt of diag cov)
    """
    # --- backend ---
    try:
        import cupy as cp
        xp = cp.get_array_module(mus)
    except Exception:
        import numpy as np
        xp = np
    import numpy as _np

    K, D = mus.shape
    pis = xp.asarray(pis, dtype=mus.dtype).reshape(K)
    mus = xp.asarray(mus, dtype=mus.dtype).reshape(K, D)

    # Likelihood set-up
    if A is None and alpha is not None:
        sigma2 = 1.0 / (2.0 * alpha)
        A = xp.eye(D, dtype=mus.dtype)
        b = xp.zeros(D, dtype=mus.dtype)
        R = sigma2 * xp.eye(D, dtype=mus.dtype)
    elif A is None:
        raise ValueError("Provide (A,b,R) or alpha.")
    A = xp.asarray(A, dtype=mus.dtype)
    k = int(A.shape[0])

    y = xp.asarray(y, dtype=mus.dtype).reshape(k)
    b = xp.zeros(k, dtype=mus.dtype) if b is None else xp.asarray(b, dtype=mus.dtype).reshape(k)
    if R is None and alpha is not None:
        sigma2 = 1.0 / (2.0 * alpha)
        R = sigma2 * xp.eye(k, dtype=mus.dtype)
    elif R is None:
        raise ValueError("R must be provided when using general A without alpha.")
    R = xp.asarray(R, dtype=mus.dtype).reshape(k, k)

    # Prior variances (support isotropic or diagonal)
    Sigmas = xp.asarray(Sigmas, dtype=mus.dtype)
    if Sigmas.ndim == 1 and Sigmas.shape[0] == K:
        s2 = (Sigmas**2)[:, None] * xp.ones((1, D), dtype=mus.dtype)     # (K, D)
    elif Sigmas.ndim == 2 and Sigmas.shape == (K, D):
        s2 = Sigmas**2                                                  # (K, D)
    else:
        raise ValueError("Sigmas must have shape (K,) or (K,D) (isotropic or diagonal).")
    inv_s2 = 1.0 / xp.maximum(s2, 1e-300)                                # (K, D)

    # Data terms: J = A^T R^{-1} A, h = A^T R^{-1} (y - b)
    Rinv_A = xp.linalg.solve(R, A)               # (k, D)
    J = A.T @ Rinv_A                             # (D, D)
    h = A.T @ xp.linalg.solve(R, (y - b))        # (D,)

    means_post = xp.empty((K, D), dtype=mus.dtype)
    stds_post  = xp.empty((K, D), dtype=mus.dtype) if out == "diag" else None
    covs_post  = xp.empty((K, D, D), dtype=mus.dtype) if out == "full" else None
    logw       = xp.empty(K, dtype=mus.dtype)

    TWO_PI = xp.asarray(2.0 * _np.pi, dtype=mus.dtype)
    y_center = y[None, :] - (A @ mus.T).T - b[None, :]                   # (K, k)
    I_D = xp.eye(D, dtype=mus.dtype)

    for i in range(K):
        # Posterior precision: P_i = J + diag(inv_s2[i])
        P_i = J + xp.diag(inv_s2[i])

        # Mean: m_i = P_i^{-1} (diag(inv_s2[i]) mu_i + h) via Cholesky
        L = xp.linalg.cholesky(P_i)                                      # lower
        rhs = inv_s2[i] * mus[i] + h
        z = xp.linalg.solve(L, rhs)
        m_i = xp.linalg.solve(L.T, z)
        means_post[i] = m_i

        # Covariance: S_i = P_i^{-1}; compute from Cholesky
        # L L^T = P_i => P_i^{-1} = L^{-T} L^{-1}
        Z = xp.linalg.solve(L, I_D)                                      # L^{-1}
        if out == "full":
            covs_post[i] = Z.T @ Z
        else:  # "diag"
            diag_S = xp.sum(Z * Z, axis=0)
            stds_post[i] = xp.sqrt(xp.maximum(diag_S, 0.0))

        # Evidence weight for component i: N(y; A mu_i + b, A Σ_i A^T + R)
        AiSigmaAT = (A * s2[i][None, :]) @ A.T                           # A diag(s2_i) A^T
        C_i = AiSigmaAT + R                                              # (k, k)
        Lc = xp.linalg.cholesky(C_i)
        v  = y_center[i]
        zc = xp.linalg.solve(Lc, v)
        #quad = zc @ xp.linalg.solve(Lc.T, zc)
        quad = zc @ zc
        log_det = 2.0 * xp.sum(xp.log(xp.diag(Lc)))
        log_like = -0.5 * (k * xp.log(TWO_PI) + log_det + quad)
        logw[i] = xp.log(pis[i] + 1e-300) + log_like

    # normalize mixture weights
    mlog = xp.max(logw)
    w_post = xp.exp(logw - mlog)
    w_post /= xp.sum(w_post) + 1e-300

    if out == "full":
        return w_post, means_post, covs_post
    else:
        return w_post, means_post, stds_post





# ─── 2. SLICED-WASSERSTEIN-2 HELPER ────────────────────────────────────────────
def sliced_wasserstein2(a, b, n_proj: int = 256, max_pts: int = 5000):
    xp = cp                                             # keep GPU

    # ----- subsample to equal size -----------------------------------------
    n = min(max_pts, a.shape[0], b.shape[0])
    ia = xp.random.choice(a.shape[0], n, replace=False)
    ib = xp.random.choice(b.shape[0], n, replace=False)
    a_, b_ = a[ia], b[ib]

    # ----- random directions in R^D ----------------------------------------
    D = a_.shape[1]
    dirs = xp.random.randn(n_proj, D, dtype=xp.float64)
    dirs /= xp.linalg.norm(dirs, axis=1, keepdims=True) + 1e-12   # unit vectors

    # ----- project, sort, average squared diffs ----------------------------
    pa = xp.sort(a_ @ dirs.T, axis=0)
    pb = xp.sort(b_ @ dirs.T, axis=0)
    w2_sq = xp.mean((pa - pb)**2)

    return float(xp.sqrt(w2_sq).get())          # move single value to host



def _median_pairwise_dist(X):
    # median ||x_i - x_j|| over i<j (approx via full matrix; subsample with max_pts)
    x2 = cp.sum(X*X, axis=1, keepdims=True)
    D2 = cp.maximum(x2 + x2.T - 2.0*(X @ X.T), 0.0)
    i = cp.triu_indices(D2.shape[0], 1)
    d = cp.sqrt(D2[i])
    return float(cp.median(d))

def _median_knn_dist(X, k=5):
    # median distance to the k-th NN (leave-one-out via inf diagonal)
    x2 = cp.sum(X*X, axis=1, keepdims=True)
    D2 = cp.maximum(x2 + x2.T - 2.0*(X @ X.T), 0.0)
    cp.fill_diagonal(D2, cp.inf)
    # kth smallest along rows
    kth = cp.partition(D2, kth=k-1, axis=1)[:, k-1]
    return float(cp.sqrt(cp.median(kth)))

def _choose_sigmas(X, mode="auto_pair+mnn", knn_k=5,
                   mnn_multipliers=(0.5, 1.0, 2.0, 4.0),
                   pair_multipliers=(0.5, 1.0, 2.0)):
    sigs = []
    if "pair" in mode:
        md = _median_pairwise_dist(X)
        sigs += [md*m for m in pair_multipliers]
    if "mnn" in mode:
        mnn = _median_knn_dist(X, k=knn_k)
        sigs += [mnn*m for m in mnn_multipliers]
    sigs = sorted({float(s) for s in sigs if s > 0})
    if not sigs:  # fallback
        md = _median_pairwise_dist(X)
        sigs = [md]
    return cp.asarray(sigs, dtype=cp.float64)

def auto_multiscale_ksd(
    x, score_fn, *,
    sigmas="auto_pair+mnn",      # "auto_mnn", "auto_pair", "auto_pair+mnn", or array-like
    knn_k=5,
    mnn_multipliers=(0.5, 1.0, 2.0, 4.0),
    pair_multipliers=(0.5, 1.0, 2.0),
    max_pts=5000,
    agg="mean",                  # "mean" or "max"
):
    """
    Multiscale KSD with data-adaptive σ including median k-NN spacing.
    Uses your existing RBF-Stein formula and averages (or maxes) across σ's.
    """
    xp = cp
    X = xp.asarray(x, dtype=xp.float64)
    if X.shape[0] > max_pts:
        idx = xp.random.choice(X.shape[0], max_pts, replace=False)
        X = X[idx]
    N, D = X.shape

    s = score_fn(X)                             # (N,D)
    diff = X[:, None, :] - X[None, :, :]        # (N,N,D)
    r2 = xp.sum(diff*diff, axis=-1)             # (N,N)

    if isinstance(sigmas, str) and sigmas.startswith("auto"):
        sig = _choose_sigmas(
            X, mode=sigmas, knn_k=knn_k,
            mnn_multipliers=mnn_multipliers,
            pair_multipliers=pair_multipliers
        )
    else:
        sig = xp.asarray(sigmas, dtype=xp.float64).ravel()
    sig = sig[xp.isfinite(sig) & (sig > 0)]
    assert sig.size >= 1, "No valid σ found."

    # Precompute score inner products
    sdot = s @ s.T                                # (N,N)
    # r·s terms
    r_dot_sx = xp.einsum('ijd,id->ij', diff, s)   # (N,N)
    r_dot_sy = xp.einsum('ijd,jd->ij', diff, s)   # (N,N)

    vals = []
    for σ in sig:
        σ2 = σ*σ
        K = xp.exp(-r2 / (2.0*σ2))                # RBF
        term1 = sdot * K
        term2 = (r_dot_sx - r_dot_sy) * (K / σ2)
        term3 = (D/σ2 - r2/(σ2*σ2)) * K
        U = term1 + term2 + term3
        ksd2 = xp.sum(U) / (N*N)
        vals.append(ksd2)

    vals = xp.asarray(vals, dtype=xp.float64)
    out = (float(xp.mean(vals)) if agg == "mean" else float(xp.max(vals)))
    return out, cp.asnumpy(sig)  # return σ’s too for debugging


def compute_multiscale_ksd(x, score_fn, sigmas = (0.1, 0.2, 0.4, .8)): #sigmas=(0.33, 0.5, 1.0, 2.0)):
    xp = cp
    N, D = x.shape
    s = score_fn(x)
    diff = x[:,None,:] - x[None,:,:]
    r2   = xp.sum(diff**2, axis=-1)
    ksd2 = 0.0
    for σ in sigmas:
        K = xp.exp(-r2 / (2*σ*σ))
        sdot = s @ s.T
        r_dot_sx = xp.einsum('ijd,id->ij', diff, s)
        r_dot_sy = xp.einsum('ijd,jd->ij', diff, s)
        term1 = sdot * K
        term2 = (r_dot_sx - r_dot_sy) / (σ*σ) * K
        term3 = (D/(σ*σ) - r2/(σ**4)) * K
        U = term1 + term2 + term3
        ksd2 += xp.sum(U) / (N*N)
    return float(ksd2 / len(sigmas))


def xp_rmse(x,y):
  xp = cp
  diff = xp.square(x-y)
  return xp.sqrt(xp.mean(diff))



def blend_diagnostics(y, t, x_ref, s0_ref, *, xp=cp):
    # weights
    var = xp.maximum(1.0 - xp.exp(-2.0*t), 1e-12)
    inv_v = 1.0 / var
    et    = xp.exp(-t)
    diff  = y[:, None, :] - et * x_ref[None, :, :]
    logits = -0.5 * inv_v * xp.sum(diff*diff, axis=-1)
    m = xp.max(logits, axis=1, keepdims=True)
    w = xp.exp(logits - m); w /= xp.sum(w, axis=1, keepdims=True) + 1e-12

    # scores
    s_twd = -(inv_v) * (y - et * (w @ x_ref))
    s_kss = xp.exp(t) * (w @ s0_ref)

    # cosine
    num = xp.sum(s_twd * s_kss, axis=1)
    den = xp.linalg.norm(s_twd, axis=1) * xp.linalg.norm(s_kss, axis=1) + 1e-12
    cos = num / den

    # ESS and λ (using your current snis_blend)
    lam, det = snis_blend(y, x_ref, s0_ref, t, return_details=True)
    ess = det['ess']
    return dict(
        cos=cos, ess=ess, lam=lam, w=w,
        s_twd=s_twd, s_kss=s_kss
    )

def summarize_diag(diag, tag=""):
    to_np = lambda z: cp.asnumpy(z)
    cos = to_np(diag['cos']); ess = to_np(diag['ess']); lam = to_np(diag['lam'])
    print(f"[{tag}] cos  median/05%/95%: {np.median(cos):+.3f}  {np.percentile(cos,5):+.3f}  {np.percentile(cos,95):+.3f}")
    print(f"[{tag}] ESS  median/min/max: {np.median(ess):.1f}   {ess.min():.1f}  {ess.max():.1f}")
    print(f"[{tag}] lam  median/min/max: {np.median(lam):.3f}   {lam.min():.3f}  {lam.max():.3f}")



def blend_sampler(
    N_part, N_ref, time_pts, batch_size, sampler_func, score_func, mode,
    likelyhood_func=None, loglik_grad_fn=None, true_score_func=None,
    h_coeff=0.5, seed='rand', weight_mode='snis', w_correct_func=None,
    use_proxy_tweedie=False,          # <-- NEW
    proxy_mu_func=None,               # <-- NEW: callable x_ref -> mu_ref
):
    xp = cp
    rmse_list = []
    dim = int(sampler_func(5).shape[1])
    y = xp.random.randn(N_part, dim).astype(xp.float64)

    x0_ref = sampler_func(N_ref) if seed == 'rand' else sampler_func(N_ref, seed=seed)

    if w_correct_func is None:
        w_correct = 1.0
    else:
        w_correct = w_correct_func(x0_ref)

    s0_ref = score_func(x0_ref)

    # if requested, build proxy means once
    mu_ref = None
    if use_proxy_tweedie:
        if proxy_mu_func is None:
            raise ValueError("use_proxy_tweedie=True requires proxy_mu_func.")
        mu_ref, _ = proxy_mu_func(x0_ref) if isinstance(proxy_mu_func, type(lambda:0)) else proxy_mu_func(x0_ref)
        # allow proxy_mu_func to return (mu,var) or just mu
        if isinstance(mu_ref, tuple):
            mu_ref = mu_ref[0]

    for k_step in range(len(time_pts)-1):
        t_cur, t_prev = time_pts[k_step], time_pts[k_step+1]
        dt = t_cur - t_prev
        noise = xp.random.randn(N_part, dim).astype(xp.float64)

        if weight_mode == 'snis' or not true_score_func:
            S_cur = s_blend(y, t_cur, x0_ref, s0_ref,
                            loglik_fn=likelyhood_func, loglik_grad_fn=loglik_grad_fn,
                            w_correct=w_correct, twd_basis=mu_ref)
            y_hat = y + dt * (y + 2 * S_cur) + xp.sqrt(2.0 * dt) * noise
            S_proxy = s_blend(y_hat, t_prev, x0_ref, s0_ref,
                              loglik_fn=likelyhood_func, loglik_grad_fn=loglik_grad_fn,
                              w_correct=w_correct, twd_basis=mu_ref)
        elif weight_mode == 'oracle':
            S_cur = true_score_func(y, t_cur)
            y_hat = y + dt * (y + 2 * S_cur) + xp.sqrt(2.0 * dt) * noise
            S_proxy = true_score_func(y_hat, t_prev)

        if mode == 'ou_sde':
            y = y_hat
        elif mode == 'pf_ode':
            y = y + dt * (y + S_cur)
        elif mode == 'heun_pc':
            drift_avg = 0.5 * ((y + 2 * S_cur) + (y_hat + 2 * S_proxy))
            y = y + dt * drift_avg + xp.sqrt(2.0 * dt) * noise
        elif mode == 'heun_hop':
            A, B = xp.exp(dt), (xp.exp(2 * dt) - 1.0)
            C, D = xp.sqrt(B), B / A
            y = A * y + h_coeff * B * S_proxy + (1 - h_coeff) * D * S_cur + C * noise
        else:
            raise ValueError(f"Unknown mode {mode}")

    if true_score_func:
        S_true1 = true_score_func(y, t_cur)
        S_true2 = true_score_func(y_hat, t_prev)
        rmse_list.append(float(xp_rmse(S_cur,  S_true1)))
        rmse_list.append(float(xp_rmse(S_proxy, S_true2)))
        return y, rmse_list
    else:
        return y



# ─── 3. SAMPLER WITH K PRE-IMAGES FOR OU-HOP ──────────────────────────────────
def tweedie_sampler(
    N_part, N_ref, time_pts, batch_size,
    sampler_func, mode, likelyhood_func = None,
    true_score_func=None, h_coeff=0.5, seed='rand', w_correct_func = None):


    xp = cp
    rmse_list = []
    dim = int(sampler_func(5).shape[1]) # Get the dimension from means0
    y = xp.random.randn(N_part, dim).astype(xp.float64) # Initialize y with the correct dimension


    if seed == 'rand':
      x0_ref = sampler_func(N_ref)
    else:
      x0_ref = sampler_func(N_ref, seed = seed)

    if w_correct_func is None:
      w_correct = 1.0
    else:
      w_correct =w_correct_func(x0_ref)


    for k_step in range(len(time_pts)-1):
        t_cur, t_prev = time_pts[k_step], time_pts[k_step+1]

        dt = t_cur - t_prev
        noise = xp.random.randn(N_part,dim).astype(xp.float64)


        A = xp.exp(dt)
        B = (xp.exp(2 * dt) - 1.0)
        C = xp.sqrt((xp.exp(2.0 * dt) - 1.0))
        D = B/A
        E = xp.log(A)

        S_cur = tweedie_score_t(y, t_cur, x0_ref, loglik_fn = likelyhood_func, w_correct = w_correct)
        y_hat = y + dt * (y + 2 * S_cur) + xp.sqrt(2.0 * dt) * noise
        S_proxy = tweedie_score_t(y_hat, t_prev, x0_ref, loglik_fn = likelyhood_func, w_correct = w_correct)

        if mode == 'ou_sde':
            y = y_hat

        elif mode == 'pf_ode':
            y =  y + dt * (y + S_cur)

        elif mode == 'heun_pc':
            drift_avg = 0.5 * ((y + 2 * S_cur) + (y_hat + 2 * S_proxy))
            y = y + dt * drift_avg + xp.sqrt(2.0 * dt) * noise

        elif mode == 'heun_hop':
           A,B = xp.exp(dt),(xp.exp(2 * dt) - 1.0),
           C,D = xp.sqrt(B), B/A
           y = A * y + h_coeff * B * S_proxy + (1-h_coeff) * D * S_cur + C * noise
        else:
            raise ValueError(f"Unknown mode {mode}")

    if true_score_func:
            S_true1 = true_score_func(y, t_cur)   # params passed in closure
            S_true2 = true_score_func(y_hat, t_prev)

            S_1_hat = S_cur
            S_2_hat = S_proxy

            rmse_list.append(float(xp_rmse(S_1_hat, S_true1)))
            rmse_list.append(float(xp_rmse(S_2_hat, S_true2)))
            return y, rmse_list
    else:
      return y



# ─── 3. SAMPLER WITH K PRE-IMAGES FOR OU-HOP ──────────────────────────────────
def kss_sampler(
    N_part, N_ref, time_pts, batch_size,
    sampler_func, score_func, mode, likelyhood_func = None, loglik_grad_fn = None,
    true_score_func=None, h_coeff=0.5, seed='rand', w_correct_func = 1.0):

    xp = cp
    rmse_list = []
    dim = int(sampler_func(5).shape[1]) # Get the dimension from means0
    y = xp.random.randn(N_part, dim).astype(xp.float64) # Initialize y with the correct dimension


    if seed == 'rand':
      x0_ref = sampler_func(N_ref)
    else:
      x0_ref = sampler_func(N_ref, seed = seed)

    s0_ref =  score_func(x0_ref)

    if w_correct_func is None:
      w_correct = 1.0
    else:
      w_correct =w_correct_func(x0_ref)

    for k_step in range(len(time_pts)-1):
        t_cur, t_prev = time_pts[k_step], time_pts[k_step+1]

        dt = t_cur - t_prev
        noise = xp.random.randn(N_part,dim).astype(xp.float64)

        S_cur = kss_score_t(y, t_cur, x0_ref, s0_ref, loglik_fn = likelyhood_func,
                            loglik_grad_fn = loglik_grad_fn, w_correct = w_correct)
        y_hat = y + dt * (y + 2 * S_cur) + xp.sqrt(2.0 * dt) * noise
        S_proxy = kss_score_t(y_hat, t_prev, x0_ref, s0_ref, loglik_fn = likelyhood_func,
                              loglik_grad_fn = loglik_grad_fn, w_correct = w_correct)

        if mode == 'ou_sde':
            y = y_hat

        elif mode == 'pf_ode':
            y =  y + dt * (y + S_cur)

        elif mode == 'heun_pc':
            drift_avg = 0.5 * ((y + 2 * S_cur) + (y_hat + 2 * S_proxy))
            y = y + dt * drift_avg + xp.sqrt(2.0 * dt) * noise

        elif mode == 'heun_hop':
           A,B = xp.exp(dt),(xp.exp(2 * dt) - 1.0),
           C,D = xp.sqrt(B), B/A
           y = A * y + h_coeff * B * S_proxy + (1-h_coeff) * D * S_cur + C * noise
        else:
            raise ValueError(f"Unknown mode {mode}")

    if true_score_func:
            S_true1 = true_score_func(y, t_cur)   # params passed in closure
            S_true2 = true_score_func(y_hat, t_prev)

            S_1_hat = S_cur
            S_2_hat = S_proxy

            rmse_list.append(float(xp_rmse(S_1_hat, S_true1)))
            rmse_list.append(float(xp_rmse(S_2_hat, S_true2)))
            return y, rmse_list
    else:
      return y


def blackbox_sampler(
    N_part, time_pts, custom_score, sampler_func, *,
    mode="heun_hop", h_coeff=0.5, true_score_func=None,
    p_prune = 0, likelyhood_func = None, loglik_grad_fn = None):
    xp  = cp
    dim = int(sampler_func(5).shape[1])           # match model/data dim
    y   = xp.random.randn(N_part, dim).astype(xp.float64)

    rmse_list = []                                # collect RMSE vs oracle

    for k in range(len(time_pts)-1):
        t_cur, t_prev = time_pts[k], time_pts[k+1]
        dt    = t_cur - t_prev
        noise = xp.random.randn(N_part, dim).astype(xp.float64)

        S_cur  = custom_score(y, t_cur)           # [N_part, dim]
        y_hat  = y + dt*(y + 2*S_cur) + xp.sqrt(2.0*dt)*noise
        S_prev = custom_score(y_hat, t_prev)

        if mode == "heun_hop":
            A = xp.exp(dt); B = xp.exp(2*dt) - 1.0; C = xp.sqrt(B); D = B / A
            y = A*y + h_coeff*B*S_prev + (1-h_coeff)*D*S_cur + C*noise
        elif mode == "ou_sde":
            y = y_hat
        elif mode == "pf_ode":
            y = y + dt*(y + S_cur)
        elif mode == "heun_pc":
            drift_avg = 0.5*(y + 2*S_cur) + 0.5*(y_hat + 2*S_prev)
            y = y + dt*drift_avg + xp.sqrt(2.0 * dt) * noise
        else:
            raise ValueError(mode)

    if true_score_func is not None:
        S_cur_true = true_score_func(y, t_cur)
        S_prev_true = true_score_func(y_hat, t_prev)

        S_cur_safe, cur_idx_safe = prune_cp_arr(S_cur, p_percent = p_prune)
        S_prev_safe, prev_idx_safe = prune_cp_arr(S_prev, p_percent = p_prune)

        RMSE_cur = float(xp_rmse(S_cur_safe, S_cur_true[cur_idx_safe]))
        RMSE_prev = float(xp_rmse(S_prev_safe, S_prev_true[prev_idx_safe]))

        rmse_list.append(RMSE_cur)
        rmse_list.append(RMSE_prev)
        return y, rmse_list
    else:
      return y


# --------------------------------------------------------------------------
def _to_numpy(arr):
    """Return a NumPy array, copying from GPU if input is CuPy."""
    try:
        import cupy as cp
        if isinstance(arr, cp.ndarray):
            return arr.get()
    except ImportError:
        pass
    return np.asarray(arr)


def div_unique_pairs(n: int):
    pairs, cnt, out = list(combinations(range(1, n + 1), 2)), [0]*(n + 1), []
    while pairs:
        p = min(pairs, key=lambda t: (cnt[t[0]] + cnt[t[1]], t))
        pairs.remove(p); out.append(p)
        cnt[p[0]] += 1; cnt[p[1]] += 1
    return out


def get_row_norm(norm_ref, i,j, bins, norm_mode = "max"):
    H_true, xedges, yedges = np.histogram2d(
          norm_ref[:, i], norm_ref[:, j], bins=bins, density=False)
    density_true = H_true / norm_ref.shape[0]
    if norm_mode == "max":
        vmax = density_true.max()
    elif norm_mode == "median":
        vmax = 2.0 * np.median(density_true[density_true > 0])
    else:
        raise ValueError("norm_mode must be 'max' or 'median'.")
    norm = plt.Normalize(vmin=0.0, vmax=vmax)
    return norm



import numpy as np
import matplotlib.pyplot as plt

def plot_likelihood_contours(
    ax,
    likelihood_func,
    xlim, ylim,
    *,
    levels=6,
    color="white",
    alpha=0.9,
    linewidths=1.2,
    linestyles="solid",
    grid_res=200,
    is_log_density=False,
    normalize=True,
    transform=None,
    chunk_size=200_000,
    zorder=12,
    backend="auto",
    full_dim=None,
    ij=None,
    anchor=None,
    levels_mode="auto",
    percentile_range=(75, 99.5),
    coverage_clip=.4,
    min_level=None,
    mass_levels=None,
    # --- debug ---
    debug_save_path=None,
    debug_save_raw=False,
    verbose=False,
):
    xs = np.linspace(xlim[0], xlim[1], int(grid_res))
    ys = np.linspace(ylim[0], ylim[1], int(grid_res))
    X, Y = np.meshgrid(xs, ys, indexing="xy")
    P2 = np.stack([X.ravel(), Y.ravel()], axis=1)

    try:
        import cupy as cp
    except Exception:
        cp = None

    def is_cupy(a): return (cp is not None) and isinstance(a, cp.ndarray)
    def to_backend(a, use_cp):
        if use_cp:
            if cp is None: raise RuntimeError("backend='cupy' but CuPy unavailable.")
            return cp.asarray(a)
        return np.asarray(a)

    use_cp = (backend == "cupy") or (backend == "auto" and cp is not None)

    # ---- lift 2D -> d if requested ----
    if full_dim is not None and ij is not None:
        i, j = ij
        xp   = cp if use_cp else np
        P2xp = to_backend(P2, use_cp)
        if anchor is None:
            anchor_xp = xp.zeros(full_dim, dtype=P2xp.dtype)
        else:
            anchor_xp = xp.asarray(anchor, dtype=P2xp.dtype)
            if anchor_xp.shape != (full_dim,):
                raise ValueError(f"`anchor` must have shape ({full_dim},), got {anchor_xp.shape}")
        U = xp.broadcast_to(anchor_xp, (P2xp.shape[0], full_dim)).copy()
        U[:, i] = P2xp[:, 0]
        U[:, j] = P2xp[:, 1]
    else:
        U = to_backend(transform(P2) if transform is not None else P2, use_cp)

    # ---- evaluate likelihood ----
    try:
        vals = likelihood_func(U)
    except Exception:
        out = []
        N = U.shape[0]
        for k in range(0, N, chunk_size):
            out.append(likelihood_func(U[k:k+chunk_size]))
        vals = (cp.concatenate(out, 0) if use_cp else np.concatenate(out, 0))

    Z = cp.asnumpy(vals) if is_cupy(vals) else np.asarray(vals)
    Z = Z.reshape(Y.shape)

    # convert log-like -> like
    if is_log_density:
        Z = Z - np.nanmax(Z)
        Z = np.exp(Z)

    # optional normalization (does not affect mass-level ordering)
    if normalize:
        m = np.nanmax(Z)
        if np.isfinite(m) and m > 0:
            Z = Z / m

    zfin = Z[np.isfinite(Z)]
    if zfin.size == 0:
        if verbose: print("[contours] all values non-finite; skip")
        return None

    # === NEW: HDR mass-based levels ===
    if mass_levels is not None:
        masses = np.sort(np.clip(np.asarray(mass_levels, float), 0.0, 0.999999))
        # compute highest-density thresholds: find t s.t. ∫_{Z>=t} Z dx dy = m * ∫ Z dx dy
        dx = (xs[-1] - xs[0]) / (len(xs) - 1) if len(xs) > 1 else 1.0
        dy = (ys[-1] - ys[0]) / (len(ys) - 1) if len(ys) > 1 else 1.0
        area = dx * dy
        vals1d = Z.ravel()
        order = np.argsort(vals1d)[::-1]
        sorted_vals = vals1d[order]
        cums = np.cumsum(sorted_vals) * area
        total = cums[-1] if cums.size else 1.0
        lev = []
        actual = []
        for m in masses:
            k = np.searchsorted(cums, m * total, side="left")
            thr = sorted_vals[k] if k < sorted_vals.size else sorted_vals[-1]
            lev.append(thr)
            # report actual enclosed mass with this threshold
            mask = Z >= thr
            actual_mass = (Z[mask].sum() * area) / total
            actual.append(actual_mass)
        lev = np.array(sorted(lev))  # ascending for contour
        if verbose:
            print("[contours] HDR mass levels requested:", masses)
            print("[contours] actual enclosed masses:    ", np.round(actual, 4))
    else:
        # --- old value-based level selection (kept for compatibility) ---
        if isinstance(levels, int):
            zmin, zmax = float(np.min(zfin)), float(np.max(zfin))
            if levels_mode == "percentile":
                lo, hi = np.percentile(zfin, percentile_range)
            elif levels_mode == "linear":
                lo, hi = zmin, zmax
            else:
                p95 = np.percentile(zfin, 95.0)
                lo, hi = (np.percentile(zfin, percentile_range)
                          if p95 < 0.15 else (zmin, zmax))
            eps = 1e-12 * max(1.0, abs(hi))
            lev = np.linspace(lo + eps, hi - eps, max(1, levels))
        else:
            lev = np.asarray(levels, float)

        if min_level is not None:
            lev = lev[lev >= float(min_level)]

        if coverage_clip is not None and lev.size > 0:
            keep = []
            for lv in lev:
                cov = np.mean(Z >= lv)
                keep.append(cov <= coverage_clip)
            keep = np.array(keep, bool)
            if keep.any():
                lev = lev[keep]
            if lev.size == 0:
                lev = np.sort(np.unique(np.percentile(zfin, np.linspace(85, 99.5, max(3, levels if isinstance(levels, int) else 6)))))

    if lev.size == 0:
        if verbose: print("[contours] no levels; skip")
        return None

    cs = ax.contour(
        X, Y, Z, levels=lev,
        colors=color, linewidths=linewidths, linestyles=linestyles,
        alpha=alpha, zorder=zorder, antialiased=True,
    )

    if debug_save_path is not None:
        fig, axd = plt.subplots(1, 1, dpi=200)
        im = axd.imshow(Z, origin="lower",
                        extent=[xs[0], xs[-1], ys[0], ys[-1]],
                        aspect="equal", cmap="magma")
        fig.colorbar(im, ax=axd, fraction=0.046, pad=0.04).set_label("likelihood (scaled)" if normalize else "likelihood")
        axd.contour(X, Y, Z, levels=lev, colors="w", linewidths=1.2)
        axd.set_xlim(xlim); axd.set_ylim(ylim)
        axd.set_title("Likelihood contours (HDR mass levels)" if mass_levels is not None else "Likelihood contours")
        fig.savefig(debug_save_path, bbox_inches="tight"); plt.close(fig)
        if debug_save_raw:
            base = debug_save_path.rsplit(".", 1)[0]
            np.savez(base + ".npz", X=X, Y=Y, Z=Z, levels=np.asarray(lev),
                     xlim=np.asarray(xlim), ylim=np.asarray(ylim))

    return cs



import itertools
import numpy as np
try:
    import cupy as cp
except Exception:
    cp = None


def get_best_div_indexes(
    y_finals_dict: dict,
    compare_keys: dict,
    n_rows: int,
    div=sliced_wasserstein2,
    *,
    prior_samples=None,
    max_dim_pairs: int | None = None,
    eps: float = 1e-12,
) -> list[tuple[int, int]]:
    """
    Find the (i,j) dim pairs where the quality gap between two samplers
    is largest, measured by ratio r(i,j) = d1(i,j)/d2(i,j) with
      d1(i,j) = div( y[key_best][:, [i,j]], prior[:, [i,j]] )
      d2(i,j) = div( y[key_worst][:, [i,j]], prior[:, [i,j]] )

    Returns 1-based indices: [(i+1, j+1), ...] (length n_rows).

    Args:
      y_finals_dict: dict[str, array-like], each value is (N, D)
      compare_keys: {'best': key1, 'worst': key2}
      n_rows: number of pairs to return (lowest ratios first)
      div: callable(a, b) -> float, default sliced_wasserstein2
      prior_samples: array-like (N0, D), REQUIRED (used as reference)
      max_dim_pairs: optional cap on the number of pairs to scan
      eps: small positive to avoid divide-by-zero

    Raises:
      ValueError on missing keys, missing prior, or dim mismatches.
    """
    if prior_samples is None:
        raise ValueError("get_best_div_indexes: 'prior_samples' is required.")
    if 'best' not in compare_keys or 'worst' not in compare_keys:
        raise ValueError("compare_keys must be a dict with keys {'best','worst'}.")

    key1 = compare_keys['best']
    key2 = compare_keys['worst']
    if key1 not in y_finals_dict or key2 not in y_finals_dict:
        raise ValueError(f"compare_keys refer to '{key1}' and '{key2}', "
                         f"but available keys are {list(y_finals_dict.keys())}.")

    A = y_finals_dict[key1]
    B = y_finals_dict[key2]
    P = prior_samples

    # Convert to numpy for shape checks, we’ll move slices to CuPy for div()
    A_np = np.asarray(A)
    B_np = np.asarray(B)
    P_np = np.asarray(P)

    if A_np.ndim != 2 or B_np.ndim != 2 or P_np.ndim != 2:
        raise ValueError("All inputs must be 2D (N, D).")

    D = A_np.shape[1]
    if B_np.shape[1] != D or P_np.shape[1] != D:
        raise ValueError("Dimension mismatch across inputs for compare/div.")

    # Build list of unique unordered pairs (i<j)
    all_pairs = list(itertools.combinations(range(D), 2))
    if max_dim_pairs is not None:
        all_pairs = all_pairs[:max_dim_pairs]

    # Ensure CuPy is available for the provided div (your sliced_wasserstein2 uses cp)
    if cp is None:
        raise RuntimeError("CuPy not available; required for sliced_wasserstein2.")

    ratios: list[tuple[float, int, int]] = []

    # Pre-cast full arrays to CuPy once to avoid repeated host->device copies
    A_cp = cp.asarray(A_np)
    B_cp = cp.asarray(B_np)
    P_cp = cp.asarray(P_np)

    for (i, j) in all_pairs:
        # Slice the two coordinates; shape (N, 2) possibly with different N across A/B/P
        A_ij = A_cp[:, [i, j]]
        B_ij = B_cp[:, [i, j]]
        P_ij = P_cp[:, [i, j]]

        d1 = div(A_ij, P_ij)
        d2 = div(B_ij, P_ij)
        r = d1 / (d2 + eps)
        ratios.append((r, i, j))

    # Sort by ascending ratio; lowest means key1 is relatively closer to prior than key2
    ratios.sort(key=lambda t: t[0])

    # Return top n_rows as 1-based pairs
    top = ratios[:max(1, n_rows)]
    return [(i + 1, j + 1) for (_, i, j) in top]



from typing import Callable
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

try:
    import cupy as cp
except Exception:
    cp = None


from typing import Callable
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

try:
    import cupy as cp
except Exception:
    cp = None


def plot_pair_histograms(
        y_finals_dict: dict,
        prior_samples,
        post_samples,
        save_path: str,
        dim_pairs: list[tuple[int, int]] = None,
        *,
        bins: int = 100,
        norm_mode: str = "max",
        hist_norm: float = 1.25,
        nrows: int = 3,
        mode: str = 'first',
        display_mode: str = 'standard',
        pre_process_f=None,       # optional preprocessing callable
        colormap: str = 'inferno',
        post_ref = True,
        likelyhood_func = None,
        draw_countours = True,
        prior_scale = True,
        plot_prior = True,
        plot_post = True,
        post_hist_norm: float = 0.8,
        # ---- comparison / selection controls ----
        compare_keys: dict | None = None,
        div_fn: Callable = sliced_wasserstein2,
        # ---- NEW: per-panel normalization instead of shared reference ----
        self_normalize: bool = False,
    ):
    """
    Plot 2D histograms for selected dimension pairs across prior and samplers.

    If self_normalize=True, each panel computes its own normalization
    (via get_row_norm) using the panel's own data on (i,j). Otherwise,
    normalization is shared within each row and computed from either
    post_samples (when post_ref=True and provided) or prior_samples.
    """

    # ------------------------------------------------------------------ #
    # helpers
    # ------------------------------------------------------------------ #
    def _to_numpy_local(arr):
        try:
            import cupy as _cp
            if isinstance(arr, _cp.ndarray):
                return arr.get()
        except Exception:
            pass
        return np.asarray(arr)

    def _all_pairs(D: int):
        return [(i+1, j+1) for i in range(D) for j in range(i+1, D)]

    # ------------------------------------------------------------------ #
    # prep data & optional preprocessing
    # ------------------------------------------------------------------ #
    true_np = _to_numpy_local(prior_samples)

    if post_samples is not None and plot_post:
        post_dict = {'Posterior': _to_numpy_local(post_samples)}
        y_finals_dict = {**post_dict, **y_finals_dict}

    y_finals_np = {k: _to_numpy_local(v) for k, v in y_finals_dict.items()}

    # apply preprocessing if provided
    if pre_process_f is not None:
        true_np = _to_numpy_local(pre_process_f(true_np))
        y_finals_np = {k: _to_numpy_local(pre_process_f(v)) for k, v in y_finals_np.items()}

    # ------------------------------------------------------------------ #
    # choose coordinate pairs on the (possibly transformed) space
    # ------------------------------------------------------------------ #
    if dim_pairs is None:
        d = true_np.shape[1]
        if mode == 'first':
            pairs_all = _all_pairs(d)
            dim_pairs = pairs_all[: min(nrows, len(pairs_all))]

        elif mode == 'rand':
            pairs_all = _all_pairs(d)
            k = min(nrows, len(pairs_all))
            idx = np.random.choice(len(pairs_all), k, replace=False)
            dim_pairs = [pairs_all[i] for i in idx]

        elif mode == 'pca':
            pca = PCA(n_components=min(32, d))
            pca.fit(true_np)
            sorted_idx = np.argsort(-pca.explained_variance_)
            dim_pairs = [(sorted_idx[2*k] + 1, sorted_idx[2*k+1] + 1)
                        for k in range(min(nrows, d // 2))]

        elif mode == 'best_compare':
            if compare_keys is None:
                raise ValueError(
                    "mode='best_compare' requires compare_keys={'best': key1, 'worst': key2}."
                )
            # Use preprocessed arrays for consistency with plot
            dim_pairs = get_best_div_indexes(
                y_finals_np, compare_keys, n_rows=nrows, div=div_fn, prior_samples=true_np
            )
            if len(dim_pairs) == 0:
                raise ValueError("best_compare produced no dimension pairs.")
        else:
            raise ValueError(f"Unknown mode {mode}")

    # convert to 0-based indices
    pairs = [(i - 1, j - 1) for (i, j) in dim_pairs]
    max_dim = max(max(p) for p in pairs)
    if true_np.shape[1] <= max_dim:
        raise ValueError("Requested dimension exceeds data dimensionality.")
    for name, gen in y_finals_np.items():
        if gen.shape[1] <= max_dim:
            raise ValueError(f"{name!r} has too few dimensions for selected pairs.")

    # ------------------------------------------------------------------ #
    # figure grid
    # ------------------------------------------------------------------ #
    n_rows = len(pairs)
    if plot_prior:
        n_cols = 1 + len(y_finals_np)
    else:
        n_cols = len(y_finals_np)

    fig_w = 4 * n_cols
    fig_h = 4 * n_rows
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_w, fig_h), squeeze=False)

    # spacing
    if display_mode == 'min_label':
        plt.subplots_adjust(wspace=0.05, hspace=0.05)
    else:
        plt.subplots_adjust(wspace=0.25, hspace=0.25)

    # ------------------------------------------------------------------ #
    # normalization references for shared (non-self) mode
    # ------------------------------------------------------------------ #
    if post_samples is not None and post_ref:
        norm_ref = _to_numpy_local(post_samples)
        h_ref = post_hist_norm
    else:
        norm_ref = true_np
        h_ref = hist_norm

    # choose which data determines 2D bins/extent
    plot_refs = true_np if prior_scale else norm_ref

    # ------------------------------------------------------------------ #
    # per-row rendering
    # ------------------------------------------------------------------ #
    for row_idx, (i, j) in enumerate(pairs):

        # establish common edges for the row (kept same even in self_normalize)
        _, xedges, yedges = np.histogram2d(
            plot_refs[:, i], plot_refs[:, j], bins=bins, density=False
        )

        # Shared normalization (legacy behavior): one Normalize per row.
        # Self-normalize: computed per panel from that panel's data.
        if not self_normalize:
            shared_norm = get_row_norm(_to_numpy_local(norm_ref), i, j, bins, norm_mode)
        else:
            shared_norm = None  # unused

        # likelihood wrapper
        if likelyhood_func is not None and draw_countours:
            def like_wrapper(U_np):
                return cp.asnumpy(likelyhood_func(cp.asarray(U_np))) if cp is not None else None
        else:
            like_wrapper = None

        # ------------------------------------------------------------------ #
        # drawing helper (now supports per-panel norm)
        # ------------------------------------------------------------------ #
        def _draw(ax, data_np, n_samples, title, draw_title: bool,
                  h_norm: float,
                  likelihood_func=None,
                  plt_norm=None,
                  like_levels=5, like_color="#D3D3D3",
                  like_alpha=.92, like_linewidths=1.5, like_linestyles="solid",
                  like_grid_res=200, like_is_log_density=False,
                  like_normalize=True, like_transform=None,
                  like_backend="auto", like_full_dim=None,
                  mass_levels=None,  # e.g. [0.65, .75, 0.85, 0.95, .99]
                  percentiles=(75, 99.5),
                  like_ij=None, like_anchor=None, debug_save_path=None):

            # compute panel-specific normalization if requested
            if self_normalize:
                plt_norm = get_row_norm(data_np, i, j, bins, norm_mode)

            H, _, _ = np.histogram2d(
                data_np[:, i], data_np[:, j],
                bins=[xedges, yedges], density=False
            )
            density = (h_norm * H / max(1, n_samples)) ** 0.95

            ax.imshow(
                density.T,
                origin="lower",
                cmap=colormap,
                norm=plt_norm if plt_norm is not None else shared_norm,
                extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
                aspect="auto",
                interpolation="nearest",
            )

            if display_mode == 'min_label':
                ax.set_xticks([]); ax.set_yticks([])
            else:
                ax.set_xlabel(f"$d_{{{i+1}}}$")
                ax.set_ylabel(f"$d_{{{j+1}}}$")
            if draw_title:
                ax.set_title(title if isinstance(title, str) else "", fontsize=16)

            if likelihood_func is not None:
                xlim = ax.get_xlim()
                ylim = ax.get_ylim()
                plot_likelihood_contours(
                    ax, likelihood_func, xlim, ylim,
                    levels=like_levels, color=like_color,
                    alpha=like_alpha, linewidths=like_linewidths,
                    linestyles=like_linestyles,
                    grid_res=like_grid_res, is_log_density=like_is_log_density,
                    normalize=like_normalize, transform=like_transform,
                    backend=like_backend, full_dim=like_full_dim,
                    ij=like_ij, anchor=like_anchor,
                    percentile_range=percentiles,
                    mass_levels=mass_levels,
                    debug_save_path=debug_save_path
                )

        # ------------------------------------------------------------------ #
        # column 0: prior
        # ------------------------------------------------------------------ #
        if plot_prior:
            prior_panel_norm = None if self_normalize else get_row_norm(true_np, i, j, bins, norm_mode)
            _draw(
                axes[row_idx, 0],
                true_np, true_np.shape[0],
                True,  # title placeholder
                draw_title=(row_idx == 0),
                h_norm=hist_norm,
                likelihood_func=(
                    (lambda U_np: cp.asnumpy(likelyhood_func(cp.asarray(U_np))))
                    if (likelyhood_func is not None and draw_countours and cp is not None)
                    else None
                ),
                plt_norm=prior_panel_norm,
                like_full_dim=true_np.shape[1],
                like_is_log_density=True,
                like_ij=(i, j),
                like_anchor=None,
                debug_save_path=None
            )

        # ------------------------------------------------------------------ #
        # remaining columns: each sampler
        # ------------------------------------------------------------------ #
        for col_idx, (lab, y_gen) in enumerate(y_finals_np.items(),
                                               start=int(plot_prior)):
            gen_panel_norm = None  # computed in _draw if self_normalize else use shared
            _draw(
                axes[row_idx, col_idx],
                y_gen, y_gen.shape[0],
                lab,
                draw_title=(row_idx == 0),
                h_norm=h_ref,
                likelihood_func=None,
                plt_norm=gen_panel_norm
            )

        # compact y-label column (min_label mode)
        if display_mode == 'min_label':
            ax0 = axes[row_idx, 0]
            label = f"$d_{{{i+1}}}\\,v\\,d_{{{j+1}}}$"
            ax0.set_ylabel(label, rotation=90, labelpad=20,
                           va='center', ha='center', fontsize=14)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close(fig)




def lambda_two_step(N, T_end, T_target):
    # --- first step only ----------------
    delta = T_target ** (1.0 / N)
    dt0   = T_end * (1.0 - delta)

    B0    = math.exp(2*dt0) - 1.0
    D0    = B0 / math.exp(dt0)
    R0    = (math.exp(dt0) - 1.0 - dt0) / (dt0*math.exp(dt0)-math.exp(dt0)+1)
    lam1  = (R0 * D0) / (B0 + R0 * D0)

    # --- two-step cancellation ----------
    dt1   = dt0 * delta
    def R(dt): return (math.exp(dt)-1-dt)/(dt*math.exp(dt)-math.exp(dt)+1)
    def B(dt): return math.exp(2*dt)-1
    def D(dt): return B(dt)/math.exp(dt)
    lam2_num = R(dt0)*D(dt0)*dt0**2 + R(dt1)*D(dt1)*dt1**2
    lam2_den = B(dt0)*dt0**2       + B(dt1)*dt1**2
    lam2     = lam2_num / lam2_den

    return .5 * (lam1+lam2)



def power_grid(t0, tT, n, gamma=1.0):  # gamma<1 flattens big steps
    frac = (tT / t0)
    exps = (np.arange(n+1) / n) ** gamma
    return t0 * frac ** exps


# --- Get local Gaussian stats (mu_i, var_i+r) used by proxy Tweedie ----
def local_gaussian_stats(x_ref, params=None):
    """
    Returns (mu, var_r) from the same local Gaussian fit used in fit_kernel_score.
      mu   : (N,D) weighted local means
      var_r: (N,D) diagonal variances with ridge added
    """
    if params is None:
        params = {}

    X = cp.asarray(x_ref)
    N, D = X.shape
    dtype = params.get("dtype", cp.float32)
    X = X.astype(dtype, copy=False)

    # kNN with adaptive bandwidths (same as fit_kernel_score)
    N0, k0, alpha = 2000, max(4*D, 48), 0.4
    k_default = int(min(N-1, max(4*D, round(k0 * (X.shape[0]/N0) ** alpha))))
    k = int(params.get("k", k_default))
    k = max(1, min(k, N - 1))

    x2 = cp.sum(X * X, axis=1, keepdims=True)
    dist2 = cp.maximum(x2 + x2.T - 2.0 * (X @ X.T), 0.0)
    cp.fill_diagonal(dist2, cp.inf)
    idx = cp.argpartition(dist2, kth=k - 1, axis=1)[:, :k]
    d2_knn = cp.take_along_axis(dist2, idx, axis=1)
    h2 = cp.maximum(cp.max(d2_knn, axis=1), cp.finfo(dtype).tiny)

    w   = cp.exp(-d2_knn / (2.0 * h2[:, None]))
    Xnb = X[idx]  # (N,k,D)
    denom = cp.sum(w, axis=1, keepdims=True) + cp.finfo(dtype).eps
    mu   = cp.sum(w[:, :, None] * Xnb, axis=1) / denom  # (N,D)
    diff = Xnb - mu[:, None, :]
    var  = cp.sum(w[:, :, None] * diff * diff, axis=1) / denom  # (N,D)

    ridge_frac = float(params.get("ridge_frac", 1e-3))
    tau = ridge_frac * cp.mean(var, axis=1, keepdims=True)       # (N,1)
    var_r = var + tau
    return mu.astype(cp.float64), var_r.astype(cp.float64)



def fit_kernel_score(x_ref, params=None):
    """
    Local Gaussian (diagonal) kernel score fit, with optional GMM/KDE score recomputation.

    Inputs:
      x_ref   : (N, D) array-like (moved to GPU).
      params  : dict (optional)
        - k            : int, kNN size for local fit (default ~4*D, clipped to N-1)
        - ridge_frac   : float, ridge as fraction of per-point mean var (default: 1e-2)
        - dtype        : cp.float32 or cp.float64 (default: float32)
        - recompute    : bool, if True return GMM/KDE score at anchors (default: False)
        - gmm_batch    : int, component batch size for recompute (default: 1024)

    Returns:
      s_hat : (N, D) CuPy array.
              If recompute=False: local score  s_i = (mu_i - x_i) / (var_i + ridge).
              If recompute=True : GMM/KDE score s(x_i) = [∑_j N(x_i|mu_j,Σ_j) Σ_j^{-1}(mu_j-x_i)] / [∑_j N(x_i|mu_j,Σ_j)].
    """
    if params is None:
        params = {}

    X = cp.asarray(x_ref)
    N, D = X.shape
    dtype = params.get("dtype", cp.float32)
    X = X.astype(dtype, copy=False)

    # -------- Local fit (as before) -----------------------------------------
    N0, k0, alpha = 2000, max(4*D, 48), 0.4
    k_default = int(min(N-1, max(4*D, round(k0 * (X.shape[0]/N0) ** alpha))))
    k = int(params.get("k", k_default))
    k = max(1, min(k, N - 1))

    # Pairwise squared distances (self-excluded by inf on diag)
    x2 = cp.sum(X * X, axis=1, keepdims=True)
    dist2 = x2 + x2.T - 2.0 * (X @ X.T)
    dist2 = cp.maximum(dist2, 0)
    cp.fill_diagonal(dist2, cp.inf)

    # kNN
    idx = cp.argpartition(dist2, kth=k - 1, axis=1)[:, :k]
    d2_knn = cp.take_along_axis(dist2, idx, axis=1)

    # Adaptive bandwidth per point
    h2 = cp.maximum(cp.max(d2_knn, axis=1), cp.finfo(dtype).tiny)

    # RBF weights with per-point bandwidth
    w = cp.exp(-d2_knn / (2.0 * h2[:, None]))

    # Neighbor points
    X_nb = X[idx]                            # (N, k, D)

    # Weighted mean
    denom = cp.sum(w, axis=1, keepdims=True) + cp.finfo(dtype).eps
    mu = cp.sum(w[:, :, None] * X_nb, axis=1) / denom   # (N, D)

    # Weighted diagonal covariance
    diff = X_nb - mu[:, None, :]                        # (N, k, D)
    var = cp.sum(w[:, :, None] * diff * diff, axis=1) / denom  # (N, D)

    # Ridge
    ridge_frac = float(params.get("ridge_frac", 1e-3))
    tau = ridge_frac * cp.mean(var, axis=1, keepdims=True)      # (N, 1)
    var_r = var + tau                                           # (N, D)

    # Local score at anchors (diagonal precision)
    s_local = (mu - X) / var_r                                  # (N, D)

    # -------- Optional recomputation: KDE/GMM score at each x_i -------------
    if not params.get("recompute", False):
        return s_local

    # We treat {mu_j, var_r_j} as a uniform GMM; compute s(x_i) for all i.
    # log N(x | mu_j, diag(var_j)) = -0.5 * [sum_d log(2π var_jd) + sum_d (x_d - mu_jd)^2 / var_jd]
    # Use two passes with log-sum-exp for stability and batch over components j.

    B = int(params.get("gmm_batch", 1024))
    two_pi = dtype(2.0 * np.pi)

    # Precompute per-component constants: logdet terms
    # (We’ll slice them inside the loop)
    logdet = cp.sum(cp.log(two_pi * var_r), axis=1)   # (N,)

    # Pass 1: find per-i maxima of log weights over all j
    m = cp.full((N,), -cp.inf, dtype=dtype)
    for j0 in range(0, N, B):
        jb = slice(j0, min(j0 + B, N))
        mu_j = mu[jb]          # (B, D)
        var_j = var_r[jb]      # (B, D)
        logdet_j = logdet[jb]  # (B,)

        # (N, B, D)
        dX = X[:, None, :] - mu_j[None, :, :]
        quad = cp.sum(dX * dX / var_j[None, :, :], axis=2)      # (N, B)
        logw = -0.5 * (quad + logdet_j[None, :])                # (N, B)

        m = cp.maximum(m, cp.max(logw, axis=1))                 # (N,)

    # Pass 2: accumulate numerator and denominator in linear space
    den = cp.zeros((N,), dtype=dtype)
    num = cp.zeros((N, D), dtype=dtype)
    eps = cp.finfo(dtype).eps

    for j0 in range(0, N, B):
        jb = slice(j0, min(j0 + B, N))
        mu_j = mu[jb]             # (B, D)
        var_j = var_r[jb]         # (B, D)
        logdet_j = logdet[jb]     # (B,)

        dX = X[:, None, :] - mu_j[None, :, :]                    # (N, B, D)
        quad = cp.sum(dX * dX / var_j[None, :, :], axis=2)       # (N, B)
        logw = -0.5 * (quad + logdet_j[None, :])                 # (N, B)
        w_nb = cp.exp(logw - m[:, None])                         # (N, B)

        # Component scores at x_i: Σ_j^{-1}(μ_j - x_i) = -(x_i - μ_j)/var_j
        comp_score = -dX / var_j[None, :, :]                     # (N, B, D)

        # Weighted sums over components in this block
        num += cp.einsum('nb,nbd->nd', w_nb, comp_score)         # (N, D)
        den += cp.sum(w_nb, axis=1)                              # (N,)

    s_gmm = num / (den[:, None] + eps)                           # (N, D)

    return s_gmm


def KSE_diff(X_ref, target_score_func):
   proxy_score = fit_kernel_score(X_ref.get())
   real_score = target_score_func(X_ref)
   diff = cp.square(proxy_score - real_score)
   return cp.asnumpy(cp.sqrt(cp.mean(diff)).get())



def fit_kernel_score_SVD(x_ref, params=None):
    """
    Option A: Local low-rank-plus-diag precision (rank-r PCA/Woodbury).

    params:
      - k              : int, kNN size (default ~4*D, clipped to N-1)
      - rank           : int, PCA rank r (default: min(8, D))
      - dtype          : cp.float32 or cp.float64 (default: float32)
      - lam_clip_mult  : float, cap λ_i ≤ lam_clip_mult * τ (default: 1e3)
      - # --- new / clarified ridge controls ---
      - ridge_mode     : {"tail_mean","tail_trimmed","tail_median"} (default: "tail_mean")
      - tail_trim_q    : float in [0,0.5], trim frac for "tail_trimmed" (default: 0.1)
      - ridge_scale    : float, scales τ_i from tail (default: 1.0)
      - ridge_floor    : float, min τ_i (default: 1e-6)
      - # --- recompute controls ---
      - recompute      : bool, if True return KDE/GMM score using low-rank Σ_j (default: False)
      - gmm_batch      : int, component batch size for recompute (default: 128)
    """
    if params is None:
        params = {}

    # -------- defaults (add/ensure these lines) --------
    params.setdefault("recompute", False)
    params.setdefault("gmm_batch", 128)
    params.setdefault("ridge_mode", "tail_mean")
    params.setdefault("tail_trim_q", 0.1)
    params.setdefault("ridge_scale", 1.0)
    params.setdefault("ridge_floor", 1e-6)
    params.setdefault("lam_clip_mult", 1e3)
    params.setdefault("rank", min(8, x_ref.shape[1]))
    # ---------------------------------------------------

    X = cp.asarray(x_ref)
    N, D = X.shape
    dtype = params.get("dtype", cp.float32)
    X = X.astype(dtype, copy=False)

    # ---------------- kNN and weights (same as before) ----------------
    N0, k0, alpha = 2000, max(4*D, 48), 0.4
    k_default = int(min(N-1, max(4*D, round(k0 * (X.shape[0]/N0) ** alpha))))
    k = int(params.get("k", k_default))
    k = max(1, min(k, N - 1))

    # Pairwise squared distances (self-excluded)
    x2 = cp.sum(X * X, axis=1, keepdims=True)
    dist2 = x2 + x2.T - 2.0 * (X @ X.T)
    dist2 = cp.maximum(dist2, 0)
    cp.fill_diagonal(dist2, cp.inf)

    # kNN indices + distances
    idx = cp.argpartition(dist2, kth=k - 1, axis=1)[:, :k]
    d2_knn = cp.take_along_axis(dist2, idx, axis=1)

    # Adaptive bandwidth and RBF weights
    h2 = cp.maximum(cp.max(d2_knn, axis=1), cp.finfo(dtype).tiny)
    w = cp.exp(-d2_knn / (2.0 * h2[:, None]))                   # (N, k)
    X_nb = X[idx]                                                # (N, k, D)

    # Weighted mean μ_i
    denom = cp.sum(w, axis=1, keepdims=True) + cp.finfo(dtype).eps
    mu = cp.sum(w[:, :, None] * X_nb, axis=1) / denom           # (N, D)

    # Diagonal variances (only to set τ robustly)
    diff = X_nb - mu[:, None, :]                                 # (N, k, D)
    var = cp.sum(w[:, :, None] * diff * diff, axis=1) / denom    # (N, D)

    # Ridge floor τ_i (scalar per point)
    ridge_frac = float(params.get("ridge_frac", 1e-3))
    tau = ridge_frac * cp.mean(var, axis=1, keepdims=True)       # (N, 1)
    eps = cp.finfo(dtype).eps

    # ---------------- Local score at anchors via rank-r PCA ----------------
    r = int(params.get("rank", min(8, D)))
    lam_clip_mult = float(params.get("lam_clip_mult", 1e3))

    ridge_mode   = params.get("ridge_mode", "tail_mean")  # "tail_mean" | "tail_trimmed" | "tail_median"
    tail_trim_q  = float(params.get("tail_trim_q", 0.1))  # top fraction of tail to drop (0.0–0.5)
    ridge_scale  = float(params.get("ridge_scale", 1.0))  # multiply the tail estimate
    ridge_floor  = float(params.get("ridge_floor", 1e-6)) # absolute min tau
    eps = cp.finfo(dtype).eps

    s_local = cp.empty_like(X)
    def _estimate_tau_from_tail(S, r_eff):
        """Estimate tau_i from singular values S of R_i (so λ = S^2)."""
        if S.shape[0] <= r_eff:
            return ridge_floor
        lam_tail = (S[r_eff:] * S[r_eff:])  # λ_j for j>r
        if ridge_mode == "tail_trimmed":
            # drop the largest tail_trim_q fraction of the tail (robust to a few structured dirs)
            m = lam_tail.shape[0]
            if m > 2:
                kdrop = int(max(1, min(m-1, round(tail_trim_q * m))))
                lam_tail_sorted = cp.sort(lam_tail)
                lam_tail = lam_tail_sorted[:m - kdrop]
            tau_i = cp.mean(lam_tail)
        elif ridge_mode == "tail_median":
            tau_i = cp.median(lam_tail)
        else:  # "tail_mean"
            tau_i = cp.mean(lam_tail)
        tau_i = float(ridge_scale) * float(tau_i)
        tau_i = float(max(tau_i, ridge_floor))
        return tau_i

    def _apply_precision_single(v, R, r_eff, Vh, S, tau_i):
        """Compute (tau I + U Λ U^T)^(-1) v using top r_eff of Vh,S and scalar tau_i."""
        if r_eff == 0 or Vh is None or S is None:
            return v / (tau_i + eps)
        V_r = Vh[:r_eff, :].T
        lam = S[:r_eff] * S[:r_eff]
        lam = cp.minimum(lam, lam_clip_mult * tau_i)
        coef = V_r.T @ v
        scale = lam / (tau_i * (tau_i + lam) + eps)
        corr = V_r @ (scale * coef)
        return (v / (tau_i + eps)) - corr

    # -------- Local score with per-point tau_i from SVD tail --------
    taus = cp.empty((N,), dtype=dtype)   # keep if you want to inspect τ_i later
    for i in range(N):
        v_i = (mu[i] - X[i])                              # (D,)
        sw = cp.sqrt(w[i] / (denom[i, 0] + eps))          # (k,)
        R_i = sw[:, None] * (X_nb[i] - mu[i])             # (k, D)
        try:
            _, S, Vh = cp.linalg.svd(R_i, full_matrices=False)
            r_eff = int(min(r, Vh.shape[0]))
        except cp.linalg.LinAlgError:
            S, Vh, r_eff = None, None, 0

        # --- NEW: τ_i from the discarded spectrum ---
        tau_i = _estimate_tau_from_tail(S, r_eff) if S is not None else ridge_floor
        taus[i] = tau_i

        s_local[i] = _apply_precision_single(v_i, R_i, r_eff, Vh, S, tau_i)

    if not params.get("recompute", False):
        return s_local

    # ---------------- Recompute path: reuse per-component τ_j and (V, λ) --------------
    B = int(params.get("gmm_batch", 128))
    two_pi = dtype(2.0 * np.pi)
    m = cp.full((N,), -cp.inf, dtype=dtype)
    den = cp.zeros((N,), dtype=dtype)
    num = cp.zeros((N, D), dtype=dtype)

    for j0 in range(0, N, B):
        jb = slice(j0, min(j0 + B, N))
        Bcur = jb.stop - jb.start

        mu_j  = mu[jb]                      # (B, D)
        tau_j = taus[jb]                    # (B,)

        V_batch = cp.zeros((Bcur, D, r), dtype=dtype)
        lam_batch = cp.zeros((Bcur, r), dtype=dtype)
        r_effs = np.empty(Bcur, dtype=np.int32)
        logdet_const = cp.empty((Bcur,), dtype=dtype)

        for b in range(Bcur):
            j = j0 + b
            sw = cp.sqrt(w[j] / (denom[j, 0] + eps))
            R_j = sw[:, None] * (X_nb[j] - mu[j])
            try:
                _, S, Vh = cp.linalg.svd(R_j, full_matrices=False)
                r_eff = int(min(r, Vh.shape[0]))
            except cp.linalg.LinAlgError:
                S, Vh, r_eff = None, None, 0

            # IMPORTANT: recompute τ_j from this component’s tail as well
            tau_comp = _estimate_tau_from_tail(S, r_eff) if S is not None else float(tau_j[b])
            tau_j[b] = tau_comp

            r_effs[b] = r_eff
            if r_eff > 0:
                V_r = Vh[:r_eff, :].T
                lam = S[:r_eff] * S[:r_eff]
                lam = cp.minimum(lam, lam_clip_mult * tau_comp)
                V_batch[b, :, :r_eff] = V_r
                lam_batch[b, :r_eff] = lam
                logdet_const[b] = D * cp.log(two_pi) + (D - r_eff) * cp.log(tau_comp + eps) \
                                  + cp.sum(cp.log(tau_comp + lam + eps))
            else:
                logdet_const[b] = D * cp.log(two_pi) + D * cp.log(tau_comp + eps)

        dX = X[:, None, :] - mu_j[None, :, :]
        T = cp.einsum('nbd,bdr->nbr', dX, V_batch)
        scale = lam_batch / (tau_j[:, None] * (tau_j[:, None] + lam_batch) + eps)
        term = cp.einsum('nbr,bdr->nbd', T * scale[None, :, :], V_batch)
        u = dX / (tau_j[None, :, None] + eps) - term

        quad = cp.sum(dX * u, axis=2)
        logw = -0.5 * (quad + logdet_const[None, :])
        comp_score = -u

        new_m = cp.maximum(m, cp.max(logw, axis=1))
        scale_old = cp.exp(m - new_m)
        W = cp.exp(logw - new_m[:, None])

        num = num * scale_old[:, None] + cp.einsum('nb,nbd->nd', W, comp_score)
        den = den * scale_old + cp.sum(W, axis=1)
        m = new_m

    s_gmm = num / (den[:, None] + eps)
    return s_gmm


def run_comparison(
    samplers, steps_list,  true_sampler_func,
    prior_sampler_func, prior_score_func=None, prior_score_div_func = None,
    likelyhood_func = None, loglik_grad_fn = None,
    post_sampler_func = None, post_score_func = None, post_score_div_func = None,
    true_score_func=None,               # NEW: oracle score function (y, t) -> score
    true_init_score = None,
    track_score_rmse=False,             # NEW: whether to compute RMSE vs oracle
    N_ref=1000, N_part=2000, batch=1000,
    trials=20, nrows=3, trial_name='trial', div='M_KSD',
    T_end=1.5, T_target=1e-3, time_split='power', self_normalize_hists = False,
    plot_hists=True, csv_path=None, hist_mode='first', display_mode='standard',
    ref_seed='rand', plot_res=False, d_pairs=None, pre_process_f=None, plot_post = False,
    mean_bins = 80, best_bins = 60, p_prune = 0, plot_prior = True, hist_norm = 1.25,
    draw_countours = False, prior_scale = True, save_tag = '', r = 0, post_hist_norm = 1.0,
    w_correct_func = None, comparison_samples = None, comparison_label = None,
    rand_seed = False, ridge_frac = 3e-3, cov_rank = 3, compare_keys = None):


    if len(save_tag):
       print_tag = f'{save_tag}, '
    else:
       print_tag = ''


    results = {f'{label}': []  for (_, _, label, *_) in samplers}
    samples_dict =  {f'{label}': []  for (_, _, label, *_) in samplers}
    divs_dict = {f'{label}': []  for (_, _, label, *_) in samplers}


    #results = {f'{steps}_dict': {} for steps in steps_list}

    #best_samples_dict = {f'{label}': []  for (_, _, label, *_) in samplers



    if loglik_grad_fn is None:                 # Indication we arent doing any posterior
        post_sampler_func = true_sampler_func
        post_score_func = true_init_score
        likelyhood_func = None

    if true_init_score == None:
      div = 'W2'
      true_init_score = prior_score_func

    clip_plot = False
    floors = []

    if track_score_rmse and true_score_func is not None:
        score_rmse = {label: [] for  (_, _, label, *_) in samplers}


    for steps in steps_list:
        delta = T_target**(1/steps)

        if time_split == 'exp':
            times = T_end * (delta**np.arange(steps+1))


        elif time_split == 'power':
            gamma = min(1.0, 0.15*steps)   # adaptive γ
            times = power_grid(T_end, T_target, steps, gamma)

        else:
            times = np.linspace(T_end, T_target, steps+1)


        h_coeff = min(lambda_two_step(steps, T_end, T_target), .5)

        output_dict = {}

        for k , (sampler_tuple) in enumerate(samplers):

            if len(sampler_tuple) == 3:
                mode, score_mode, label = sampler_tuple
                custom_callable = None
            else:
                mode, score_mode, label, custom_callable = sampler_tuple

            per_trial_rmse = []
            if ('heun' in label.lower()) and steps > steps_list[-1]/2:
                #min_div =  results[f'{label}_min'][-1]
                avg_div = results[f'{label}'][-1]
                print_res = False
                clip_plot = True

            else:

                print_res = True
                divs = []
                t0 = time.time()

                for j in range(trials):
                    #test_samples = sampler_func(N_part)
                    if rand_seed:
                      ref_seed = j
                    #test_samples = OU_evolve_samples(post_sampler_func(N_part), T_target)
                    test_samples = post_sampler_func(N_part)


                    if score_mode == 'tweedie':
                        out = tweedie_sampler(N_part, N_ref, times, batch, prior_sampler_func, mode = mode, likelyhood_func = likelyhood_func,
                                              true_score_func=true_score_func, h_coeff = h_coeff,
                                              seed = ref_seed, w_correct_func = w_correct_func)

                    elif score_mode == 'kss':
                        out = kss_sampler(N_part, N_ref, times, batch, prior_sampler_func, prior_score_func, mode = mode,
                                          likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn, true_score_func=true_score_func,
                                          h_coeff = h_coeff, seed = ref_seed,  w_correct_func = w_correct_func)


                    elif score_mode == 'blend':
                        out = blend_sampler(N_part, N_ref, times, batch, prior_sampler_func, true_init_score, mode = mode,
                                            likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn, weight_mode='snis',
                                            seed = ref_seed, h_coeff = h_coeff, true_score_func=true_score_func, w_correct_func = w_correct_func)


                    elif score_mode == 'blend_proxy':
                        proxy_prior_score = lambda x: fit_kernel_score(x, params = {"ridge_frac": ridge_frac, "recompute": False} )
                        out = blend_sampler(N_part, N_ref, times, batch, prior_sampler_func, proxy_prior_score,  mode = mode,
                                            likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn, weight_mode='snis',
                                            seed = ref_seed, h_coeff = h_coeff, true_score_func=true_score_func, w_correct_func = w_correct_func)


                    elif score_mode == 'blend_proxy_SVD':
                        params = dict(rank=cov_rank,ridge_mode="tail_trimmed", tail_trim_q=0.1, ridge_scale=1.0,
                                      ridge_floor=1e-6, recompute = False, gmm_batch=128)
                        proxy_prior_score = lambda x: fit_kernel_score_SVD(x, params = params)
                        out = blend_sampler(N_part, N_ref, times, batch, prior_sampler_func, proxy_prior_score,  mode = mode,
                                            likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn, weight_mode='snis',
                                             seed = ref_seed, h_coeff = h_coeff, true_score_func=true_score_func, w_correct_func = w_correct_func)

                    elif score_mode == 'blend_proxy_SVD_recomp':
                        params = dict(rank=cov_rank,ridge_mode="tail_trimmed", tail_trim_q=0.1, ridge_scale=1.0,
                                      ridge_floor=1e-6, recompute = True, gmm_batch=128)
                        proxy_prior_score = lambda x: fit_kernel_score_SVD(x, params = params)
                        out = blend_sampler(N_part, N_ref, times, batch, prior_sampler_func, proxy_prior_score,  mode = mode,
                                            likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn, weight_mode='snis',
                                             seed = ref_seed, h_coeff = h_coeff, true_score_func=true_score_func, w_correct_func = w_correct_func)


                    elif score_mode == 'blend_proxy_recomp':
                        proxy_prior_score = lambda x: fit_kernel_score(x, params = {"ridge_frac": ridge_frac, "recompute": True} )
                        out = blend_sampler(N_part, N_ref, times, batch, prior_sampler_func, proxy_prior_score, mode = mode,
                                            likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn, weight_mode='snis',
                                            seed = ref_seed, h_coeff = h_coeff, true_score_func=true_score_func, w_correct_func = w_correct_func)


                    elif score_mode == 'custom':
                        out = blackbox_sampler( N_part, times, custom_callable, prior_sampler_func, mode = mode,
                                               likelyhood_func = likelyhood_func, loglik_grad_fn = loglik_grad_fn,
                                               h_coeff=h_coeff, true_score_func=true_score_func, p_prune = p_prune)

                    else:
                        raise ValueError(f"Unknown score mode {score_mode}")



                    if track_score_rmse and true_score_func:
                        output_samples, rmse_list = out
                        per_trial_rmse.append(np.mean(rmse_list))
                    else:
                        output_samples = out

                    purned_output_samples,_ = prune_cp_arr(output_samples, p_percent = p_prune)
                    #test_samples, _ = prune_cp_arr(test_samples, p_percent = p_prune)
                    if div == 'W2':
                        divs.append(sliced_wasserstein2(test_samples,  purned_output_samples, n_proj=2048, max_pts=5000))
                    elif div == 'M_KSD':
                      divs.append(compute_multiscale_ksd( purned_output_samples, post_score_func))
                     # divs.append(compute_multiscale_ksd(cp.concatenate((output_samples, test_samples), axis = 0), post_score_func))
                    elif div == 'M_MMD':
                        divs.append(M_MMD(test_samples,  purned_output_samples))
                    elif div == 'OT_MMD':
                        divs.append(OT_MMD(test_samples,  purned_output_samples))
                    elif div == 'KSE_diff':
                        divs.append(KSE_diff( purned_output_samples, post_score_func))
                    else:
                        raise ValueError(f"Unknown div {div}")


                    if label not in output_dict.keys():
                      output_dict[label] = output_samples

                    else:
                      output_dict[label] = np.concatenate((output_dict[label], output_samples))

                avg_div = np.mean(divs)
                divs_dict[label].append(avg_div)

                samples_dict[label].append(output_dict[label])


                if track_score_rmse and true_score_func:
                    avg_rmse = np.mean(per_trial_rmse)
                    score_rmse[label].append(avg_rmse)



            floor_samples = post_sampler_func(N_part) #OU_evolve_samples(post_sampler_func(N_part), T_target)
            #floor_samples, _ = prune_cp_arr(floor_samples, p_percent = p_prune)

            if div == 'W2':
                  floors.append(sliced_wasserstein2(floor_samples, test_samples))
                  if comparison_samples is not None:
                    compare_div = sliced_wasserstein2(comparison_samples,test_samples)
            elif div == 'M_KSD':
                  floors.append(compute_multiscale_ksd(floor_samples, post_score_func))
                  if comparison_samples is not None:
                    compare_div = compute_multiscale_ksd(comparison_samples, post_score_func)
            elif div == 'M_MMD':
                  floors.append(M_MMD(floor_samples, test_samples))
                  if comparison_samples is not None:
                    compare_div = M_MMD(comparison_samples,test_samples)
            elif div == 'OT_MMD':
                  floors.append(OT_MMD(floor_samples, test_samples))
                  if comparison_samples is not None:
                    compare_div = OT_MMD(comparison_samples,test_samples)
            elif div == 'KSE_diff':
                  floors.append(KSE_diff(floor_samples, post_score_func))
                  if comparison_samples is not None:
                    compare_div = KSE_diff(comparison_samples, post_score_func)
            else:
                  raise ValueError(f"Unknown div {div}")

            if comparison_label is not None:
              compare_str = f' {comparison_label}: {div} = {compare_div:.3e},'
              output_dict['comparison_label'] = comparison_samples
            else:
              compare_str = ''


            #results[f'{label}'].append(avg_div)

            if len(save_tag):
              print_tag = f'{save_tag}, '
            else:
              print_tag = ''

            mean_floor = np.mean(floors)
            divs_dict['floor'] = mean_floor

            if print_res:
              if track_score_rmse and true_score_func:
                print(f"{print_tag}{steps:2d} steps, {label}: {div} ={avg_div:.3e}, {div}_floor = {mean_floor:.3e}, score_RMSE={avg_rmse:.3e}, time={time.time()-t0:.1f}s")
                #print(f"{print_tag}{steps:2d} steps, {label}: {div} ={avg_div:.3e},{compare_str} {div}_floor = {np.mean(floors):.3e}, time={time.time()-t0:.1f}s")
                print(' ')
              else:
                print(f"{print_tag}{steps:2d} steps, {label}: {div} ={avg_div:.3e},{compare_str} {div}_floor = {np.mean(floors):.3e}, time={time.time()-t0:.1f}s")
                print(' ')


        #true_prior_samples = OU_evolve_samples(true_sampler_func(trials * N_part), T_target)
        true_prior_samples = true_sampler_func(trials * N_part)

        if plot_hists:
          if likelyhood_func is not None:
           #post_samples = OU_evolve_samples(post_sampler_func(trials * N_part, debug = True), T_target)
            post_samples = post_sampler_func(trials * N_part)
            #post_samples, _ = prune_cp_arr(post_samples, p_percent = p_prune)
          else:
            post_samples = None

          #true_prior_samples, _ = prune_cp_arr(true_prior_samples, p_percent = p_prune)
          plot_pair_histograms(output_dict, true_prior_samples, post_samples, save_path=f"{save_tag}_{steps}_steps_mean_hist.png",
                               bins = mean_bins, nrows = nrows,  mode = hist_mode, display_mode = display_mode, hist_norm = hist_norm,
                               dim_pairs = d_pairs, pre_process_f = pre_process_f, likelyhood_func = likelyhood_func,
                               draw_countours = draw_countours, prior_scale = prior_scale, compare_keys = compare_keys,
                               plot_prior = plot_prior, post_hist_norm = post_hist_norm, plot_post = plot_post, self_normalize = self_normalize_hists)

        results[f'{steps}_dict'] = (output_dict, divs_dict)

    if plot_res:
      plt.figure(figsize=(6,4))
      colors = ['blue', 'red', 'green', 'black',
                'orange', 'purple','pink', 'yellow', 'grey']

      floor = np.mean(floors)
      print(f"Floor={floor:.4e}")


      max_avg = -np.inf
      for i, (_, _, sampler, *_) in enumerate(samplers[:len(colors)]):
        if ('heun' in sampler.lower()):
            NFEs = 2 * np.asarray(steps_list)
        else:
            NFEs =  np.asarray(steps_list)

        plt.plot(NFEs , np.log(results[f'{sampler}']), marker='o',
                label = f'{sampler}', color = colors[i])

        if np.max(np.log(results[f'{sampler}'])) >max_avg:
            max_avg = np.max(np.log(results[f'{sampler}']))


      plt.plot(NFEs, np.log(np.asarray([floor for NFE in NFEs])),
               label = 'floor', color = 'black', linestyle = 'dashed')


      plt.xlabel("NFE", fontsize=11)


      if clip_plot:
        plt.xlim(left = 2 *  steps_list[0], right = steps_list[-1])

      if div == 'W2':
        plt.ylabel(f" Mean Log Wasserstein", fontsize=13)
      elif div == 'M_KSD':
        plt.ylabel(f" Mean Log M-KSD", fontsize=13)

      plt.title(f"Sampler Comparison", fontsize=13)
      plt.legend(loc='upper right')
      plt.grid(ls='--', alpha=0.4)
      #plt.ylim(bottom = floor - 1,  top = min(max_avg + 2,5))
      plt.tight_layout()
      plt.savefig(f'{trial_name}.png')
      plt.show()

    samples_dict['True prior'] = true_prior_samples
    try:
      samples_dict['True posterior'] = OU_evolve_samples(post_sampler_func(trials * N_part), T_target)
    except Exception:
      print('There was an issue big dawg')


    results['samples_dict'] = samples_dict
    results['divs_dict'] = divs_dict
    if track_score_rmse and true_score_func:
        results['score_rmse'] = score_rmse
    return results


# ─── 4. MAIN SWEEP ─────────────────────────────────────────────────────────────

def main1(N_REF = 500, plot_hists = False, do_inv = False,
          div = 'W2',save_tag = '', ridge_frac = 3e-3, trials = 5):
    ################################################################################################################################################################################
    SEED  = 2
    NUM_C, K_DIM, M_DIM, = 6000, 12, 3
    VARIANT, STD, SCALE, norm_size = "helix", .12, 3.0, 1.0
    T_target, T_end = 5e-4, 1.5
    EMBED_MODE = 'sine_wiggle'
    #N_REF = 500
    ################################################################################################################################################################################

    # 1) Build your pathological GMM and grab its params for the oracle
    params, sampler_func, score_func, density_func, score_div_func = get_gmm_funcs(num_c = NUM_C, variant = VARIANT, embedding_mode=EMBED_MODE,
                                                     overall_scale = SCALE, comp_std=STD, k_dim =K_DIM, m_dim = M_DIM, normalize = True)

    proxy_score_func = fit_kernel_score


    test_sample = sampler_func(1)
    test_density = density_func(test_sample)

    #params, sampler_func, score_func = make_pathological_gmm(
        #C=500, d=39, sep=2, cov_scale=0.01, seed=0)
    means0, covars0, w0 = params   # these are NumPy arrays



    # **Convert to Cupy once** so all downstream calls get Cupy arrays
    means0_cp = cp.asarray(means0, dtype=cp.float64)
    covars0_cp  = cp.asarray(covars0,  dtype=cp.float64)
    w0_cp  = cp.asarray(w0,dtype=cp.float64)


    ################################################################################################################################################################################
    if do_inv:
      D = means0.shape[1]
      rank_k = 2          # e.g., project down to 3 observed dims
      obs_sigma = 1/10

      #y_obs = sampler_func(1, seed=SEED).reshape(-1)
      A, y_obs_k, likelyhood_func, log_likelyhood_func, loglik_grad_fn = make_rank_k_likelihood(
        D=D, rank_k=rank_k, obs_sigma=obs_sigma,
        sampler_func=sampler_func, seed=5)

      def log_density_func(x_ref):
        return cp.log(density_func(x_ref))  # unchanged

      Rk = (obs_sigma**2) * cp.eye(rank_k, dtype=cp.float64)

      w_post, means_post, covars_post = gmm_posterior_params(y_obs_k, w0, means0, covars0, A=A, b=None, R=Rk)

      post_params = (means_post, covars_post, w_post)
      _, post_sampler_func, post_score_func, post_density_func, post_score_div_func = get_gmm_funcs(num_c = NUM_C, variant = 'custom',
                                                                               preset_params = post_params,  normalize = False)

      means_post_cp = cp.asarray(means_post, dtype=cp.float64)
      covars_post_cp  = cp.asarray(covars_post,  dtype=cp.float64)
      w_post_cp     = cp.asarray(w_post,dtype=cp.float64)

    else:
      post_sampler_func = None
      post_score_func = score_func
      post_score_div_func = score_div_func
      likelyhood_func = None
      loglik_grad_fn = None
      means_post_cp = means0_cp
      covars_post_cp  = covars0_cp
      w_post_cp     = w0_cp

    ################################################################################################################################################################################


    # 2) Wrap the true‐score calculator so it matches run_comparison’s API:
    #    true_score_func(y_batch, t_cur) -> [B, d] cupy array
    def true_score_func(y_batch, t_cur):
        # here batch_size is only used for micro‐batching inside calculate_true_score_at_t,
        # but since y_batch is already a batch we just pass len(y_batch)
        return calculate_true_score_at_t(
            y_batch,
            t_cur,
            means_post_cp, covars_post_cp, w_post_cp,
            batch_size=y_batch.shape[0])


    # 3) Define your samplers
    samplers = [
        ('heun_pc', 'blend',        'Blend score( True ) '),
        #('heun_pc', 'blend_proxy',  'Blend score( Proxy)' ),
        #('heun_pc', 'blend_proxy_SVD',  'Blend score(Proxy SVD)' ),
        #('heun_pc', 'blend_proxy_SVD',  'Blend score(Proxy SVD recomp)' ),
        #('heun_pc', 'tweedie',      'Tweedie score'),
    ]
    # 4) Call run_comparison with the new arguments:


    results = run_comparison(
        samplers              = samplers,
        steps_list            = [8],
        true_sampler_func     = sampler_func,
        prior_sampler_func    = sampler_func,
        prior_score_func      = score_func,
        prior_score_div_func  = score_div_func,
        likelyhood_func       = likelyhood_func,
        loglik_grad_fn        = loglik_grad_fn,
        post_sampler_func     = post_sampler_func,
        post_score_func       = post_score_func,
        post_score_div_func   = post_score_div_func,
        true_init_score       = score_func,
        ref_seed              = 0,
        trials                = trials,
        N_part                = 5000,
        nrows                 = 3,
        time_split            = 'power',
        trial_name            = 'test_with_score_rmse',
        div                   = div,
        plot_hists            = plot_hists,
        hist_mode             = 'pca',
        display_mode          =  'min_label',
        d_pairs               = None,
        plot_res              = False,
        N_ref = N_REF,
        T_end= T_end,
        T_target=T_target,
        true_score_func       = true_score_func,
        track_score_rmse      = True,
        mean_bins = 120,
        prior_scale =   negate(do_inv),
        plot_prior =  negate(do_inv),
        plot_post = True,
        draw_countours = False,
        hist_norm= 1.5,
        post_hist_norm = 1.0,
        save_tag = save_tag,
        ridge_frac = ridge_frac,
        rand_seed= False
        )

    divs_dict = results['divs_dict']
    samples_dict = results['samples_dict']

    return results

    # 5) results will now include keys like
    #    'HHop, Tweedie score_score_rmse' and 'HHop, Blend score_score_rmse'
    #print(results)


# --- sampler roster -----------------------------------------------------

def negate(arg):
  if arg:
    return False
  else:
    return True

if __name__ == "__main__":
  pass

  main1(500, plot_hists = True, div = 'W2',
        ridge_frac = 1e-5, trials = 1, do_inv = False)


 8 steps, Blend score( True ) : W2 =9.068e-02, W2_floor = 2.175e-02, score_RMSE=5.252e+01, time=7.3s
 


In [ ]:
# ==========================================
# Tweedie-only Neural Distillation (OU score) + PROBE
#  - log- or uniform-t sampling
#  - cosine LR, EMA, AMP, grad clipping
#  - training target from Tweedie (NP) via streamed refs
#  - eval: RMSE(net_ema vs TRUE) across γ-time grid
#  - NEW: probe_with_run_comparison() during training (Heun-PC path)
# ==========================================

import math, os, time
import numpy as np
import cupy as cp  # <--- NEW
import torch
import torch.nn as nn
import torch.optim as optim

# -------------------- Config --------------------
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
USE_AMP  = (DEVICE == "cuda")

########################################################################################
SEED  = 0
NUM_C, K_DIM = 6000, 32
VARIANT, STD, SCALE = "crossing_torus", .02, 3.0
EMBED_MODE = 'sine_wiggle'
T_target, T_end = 5e-4, 1.5
N_Train = 500000
N_REF = 3000
HIDDEN = 512
STEPS = 90000
########################################################################################

BATCH = 512
LR_INIT, LR_MIN = 5e-3, 1e-5
T_MIN, T_MAX = T_target, T_end
PRINT_EVERY = 1000
EMA_DECAY = 0.999
REF_CHUNK  = BATCH
MAX_REF_CALL = 50_000
REFRESH_DATA = False

# PROBE controls (matched to your snippet)
PROBE_STEPS      = 8
PROBE_TRIALS     = 1
RUN_PROBE_EVERY  = PRINT_EVERY   # call probe on the same cadence as prints
N_TEST = 5000

# time sampling mode for training
T_MODE = 'uniform'
SAVE_PATH = "tweedie_distill_ema.pt"


torch.manual_seed(SEED); np.random.seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True


# ===== Gamma-prior eval helpers =====
GAMMA_PRIOR   = 1
EVAL_N_T      = 24     # number of midpoint times for training-time eval
EVAL_PER_T    = 2048   # eval samples per time
T_END, T_TGT  = T_MAX, T_MIN

@torch.no_grad()
def make_gamma_times(n_t: int = EVAL_N_T, gamma: float = GAMMA_PRIOR,
                     T_end: float = T_END, T_tgt: float = T_TGT, device: str = DEVICE):
    u = (torch.arange(n_t, device=device, dtype=torch.float32) + 0.5) / n_t  # midpoints
    return T_end * ((T_tgt / T_end) ** (u ** (1.0 / gamma)))                # [n_t], decreasing

def rmse_t(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.sqrt(torch.mean((a - b) ** 2))

# ----------------- Basic GMM helpers -----------------
@torch.no_grad()
def torch_sample_gmm(n: int, means: torch.Tensor, stds: torch.Tensor, weights: torch.Tensor) -> torch.Tensor:
    K, D = means.shape
    idx = torch.multinomial(weights, num_samples=n, replacement=True)
    noise = torch.randn(n, D, device=means.device) * stds[idx].unsqueeze(1)
    return means[idx] + noise

@torch.no_grad()
def ou_params_at_t(means0: torch.Tensor, stds0: torch.Tensor, t: torch.Tensor):
    et   = torch.exp(-t)                 # [B,1]
    et2  = torch.exp(-2.0 * t)
    means_t = et.view(-1,1,1) * means0.unsqueeze(0)       # [B,K,D]
    var_t   = stds0.pow(2).unsqueeze(0) * et2 + (1.0 - et2)
    stds_t  = torch.sqrt(var_t + 1e-12)
    return means_t, stds_t

@torch.no_grad()
def true_score_xt_torch(x: torch.Tensor, t: torch.Tensor, means0: torch.Tensor, stds0: torch.Tensor, weights0: torch.Tensor) -> torch.Tensor:
    B, D = x.shape
    K, _ = means0.shape
    means_t, stds_t = ou_params_at_t(means0, stds0, t)
    inv_vars = 1.0 / (stds_t ** 2)
    log_w    = torch.log(weights0 + 1e-40).view(1, K).expand(B, K)
    norm_const = -0.5 * D * (math.log(2*math.pi) + torch.log(stds_t**2))
    diff = means_t - x.unsqueeze(1)                               # [B,K,D]
    d2   = (diff**2).sum(-1)
    logp = log_w + norm_const - 0.5 * d2 * inv_vars
    w    = torch.softmax(logp, dim=1)
    return (w.unsqueeze(-1) * diff * inv_vars.unsqueeze(-1)).sum(1)

@torch.no_grad()
def sample_evolved_gmm(n: int, t_scalar: float,
                       means0: torch.Tensor, stds0: torch.Tensor, w0: torch.Tensor) -> torch.Tensor:
    t  = torch.tensor(t_scalar, device=means0.device, dtype=means0.dtype)
    et = torch.exp(-t); et2 = torch.exp(-2.0*t)
    means_t = means0 * et
    var_t   = stds0**2 * et2 + (1.0 - et2)
    stds_t  = torch.sqrt(var_t.clamp_min(1e-12))
    return torch_sample_gmm(n, means_t, stds_t, w0)

# ----------------- Tweedie (NP) target -----------------
@torch.no_grad()
def tweedie_score_np(y: torch.Tensor, t: torch.Tensor, x_ref: torch.Tensor,
                     ref_chunk: int = 4096, max_ref: int | None = 10_000) -> torch.Tensor:
    """
    Streaming Tweedie target s_twd(y,t) using references x_ref:
      s_twd = - (y - E[e^{-t} X_0 | y]) / Var(y|x0) ,
      with OU variance Var = 1 - e^{-2t}.
    Returns [B,D].
    """
    B, D = y.shape
    N = x_ref.shape[0]
    if (max_ref is not None) and (N > max_ref):
        idx = torch.randperm(N, device=x_ref.device)[:max_ref]
        x_ref = x_ref[idx]; N = x_ref.shape[0]

    et   = torch.exp(-t)                     # [B,1]
    et2  = torch.exp(-2.0 * t)
    var  = (1.0 - et2).clamp_min(1e-12)      # [B,1]
    invv = 1.0 / var

    y2     = (y*y).sum(-1)                   # [B]
    m      = torch.full((B,), -float("inf"), device=y.device, dtype=y.dtype)
    s      = torch.zeros(B, device=y.device, dtype=y.dtype)
    num_x  = torch.zeros(B, D, device=y.device, dtype=y.dtype)

    for start in range(0, N, ref_chunk):
        end = min(start + ref_chunk, N)
        xr  = x_ref[start:end].to(y.device, non_blocking=True).contiguous()   # [Nc,D]
        xr2   = (xr*xr).sum(-1)                                # [Nc]
        cross = y @ xr.T                                       # [B,Nc]
        d2    = y2[:,None] + (et*et) * xr2[None,:] - 2.0 * et * cross
        L     = -0.5 * invv * d2                               # [B,Nc]

        m_new = torch.maximum(m, L.max(dim=1).values)          # [B]
        scale = torch.exp(m - m_new)                           # [B]
        wtil  = torch.exp(L - m_new[:,None])                   # [B,Nc]

        s      = s * scale + wtil.sum(1)
        num_x  = num_x  * scale[:,None] + torch.einsum('bn,nd->bd', wtil, xr)
        m      = m_new

    x_bar  = num_x  / (s[:,None] + 1e-30)                     # E[X0|y] proxy (scaled later)
    mu_bar = et * x_bar                                       # E[e^{-t} X0 | y]
    return -invv * (y - mu_bar)

# ----------------- Net ----------------------
class ScoreNet(nn.Module):
    def __init__(self, dim: int, hidden: int = 512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim + 1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden),   nn.SiLU(),
            nn.Linear(hidden, dim),
        )
    def forward(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        return self.net(torch.cat([x, torch.log(t)], dim=1))  # predicts sigma * s(y,t)

def update_ema(target: nn.Module, src: nn.Module, decay: float):
    with torch.no_grad():
        for p_t, p_s in zip(target.parameters(), src.parameters()):
            p_t.data.mul_((decay)).add_(p_s.data, alpha=(1.0 - decay))

# ----------------- Build targets (GMM params only) -----------------
# Expectation: get_gmm_funcs is available in your codebase
params, sampler_func, score_func = get_gmm_funcs(
    NUM_C, k_dim=K_DIM, variant=VARIANT, comp_std=STD, overall_scale=SCALE,
    seed=0, embedding_mode=EMBED_MODE
)[:3]
means0, stds0, w0 = params

# convert to torch tensors directly
means0 = torch.as_tensor(means0, device=DEVICE, dtype=torch.float32).contiguous()
stds0  = torch.as_tensor(stds0,  device=DEVICE, dtype=torch.float32).contiguous()
w0     = torch.as_tensor(w0,     device=DEVICE, dtype=torch.float32).contiguous()
w0    /= w0.sum()

# Fixed reference set for Tweedie targets (CPU pinned for streaming)
with torch.no_grad():
    x_ref = torch_sample_gmm(N_REF, means0, stds0, w0).detach().cpu().float().pin_memory()

D = means0.shape[1]
net_twd     = ScoreNet(D, hidden=HIDDEN).to(DEVICE)
net_twd_ema = ScoreNet(D, hidden=HIDDEN).to(DEVICE)
net_twd_ema.load_state_dict(net_twd.state_dict())

# optional compile
try:
    if DEVICE == "cuda":
        net_twd     = torch.compile(net_twd)
        net_twd_ema = torch.compile(net_twd_ema)
except Exception:
    pass

opt_twd   = optim.Adam(net_twd.parameters(), lr=LR_INIT)
sched_twd = torch.optim.lr_scheduler.CosineAnnealingLR(opt_twd, T_max=STEPS, eta_min=LR_MIN)
scaler_twd= torch.amp.GradScaler('cuda', enabled=USE_AMP)
mse = nn.MSELoss()

def update_ema_all():
    update_ema(net_twd_ema, net_twd, EMA_DECAY)

# ----------------- Fixed training dataset (CPU pinned, deterministic) -----------------
TRAIN_SEED  = int(globals().get("TRAIN_SEED", globals().get("SEED", 0)))
_gen_cpu    = torch.Generator(device='cpu').manual_seed(TRAIN_SEED)

def _sample_gmm_cpu_with_gen(n, means_cpu, stds_cpu, w_cpu, gen):
    K, D = means_cpu.shape
    idx   = torch.multinomial(w_cpu, num_samples=n, replacement=True, generator=gen)
    noise = torch.randn(n, D, generator=gen, dtype=means_cpu.dtype)
    return means_cpu[idx] + noise * stds_cpu[idx].unsqueeze(1)

with torch.no_grad():
    _means_cpu = means0.detach().cpu().float()
    _stds_cpu  = stds0.detach().cpu().float()
    _w_cpu     = (w0 / w0.sum()).detach().cpu().float()

    x0_train_cpu  = _sample_gmm_cpu_with_gen(N_Train, _means_cpu, _stds_cpu, _w_cpu, _gen_cpu).pin_memory()
    u_train_cpu   = torch.rand(N_Train, 1, generator=_gen_cpu, dtype=torch.float32).pin_memory()
    eps_train_cpu = torch.randn(N_Train, _means_cpu.shape[1], generator=_gen_cpu, dtype=torch.float32).pin_memory()

_train_ptr  = 0
_train_perm = torch.randperm(N_Train, generator=_gen_cpu)

def _next_batch_indices(B):
    global _train_ptr, _train_perm
    if _train_ptr + B <= N_Train:
        idx = _train_perm[_train_ptr:_train_ptr+B]; _train_ptr += B; return idx
    tail = _train_perm[_train_ptr:]
    _train_perm = torch.randperm(N_Train, generator=_gen_cpu)  # <--- FIXED the tiny typo from your paste
    _train_ptr  = 0
    need = B - tail.numel()
    head = _train_perm[_train_ptr:_train_ptr+need]; _train_ptr += need
    return torch.cat([tail, head], dim=0)

# ----------------- Batch maker (Tweedie NP target) -----------------
@torch.no_grad()
def make_batch(B, *, ref_chunk=None, max_ref=None, t_mode=T_MODE):
    if ref_chunk is None:
        ref_chunk = min(4096, int(globals().get("N_REF", 4096)))
    if max_ref is None:
        max_ref = int(globals().get("N_REF", 10_000))

    if globals().get("REFRESH_DATA", False):
        gen = torch.Generator(device='cpu').manual_seed(time.time_ns() % (2**63))
        x0  = _sample_gmm_cpu_with_gen(B, _means_cpu, _stds_cpu, _w_cpu, gen).to(DEVICE, non_blocking=True)
        u   = torch.rand(B, 1, generator=gen, dtype=torch.float32).to(DEVICE, non_blocking=True)
        eps = torch.randn(B, _means_cpu.shape[1], generator=gen, dtype=torch.float32).to(DEVICE, non_blocking=True)
    else:
        idx = _next_batch_indices(B)
        x0  = x0_train_cpu[idx].to(DEVICE, non_blocking=True)
        u   = u_train_cpu[idx].to(DEVICE, non_blocking=True)
        eps = eps_train_cpu[idx].to(DEVICE, non_blocking=True)

    if t_mode == 'log':
        logTmin = math.log(T_MIN); logTmax = math.log(T_MAX)
        t = torch.exp(torch.tensor(logTmin, device=DEVICE) + u * (logTmax - logTmin))
    elif t_mode == 'uniform':
        t = torch.tensor(T_MIN, device=DEVICE) + u * (T_MAX - T_MIN)
    else:
        raise ValueError(f"Unknown t_mode: {t_mode}")

    et    = torch.exp(-t)
    sigma = torch.sqrt((1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12))
    x_t   = et * x0 + sigma * eps

    # streamed Tweedie NP target
    x_ref_src = globals().get("x_ref_cpu", globals().get("x_ref", None))
    if x_ref_src is None:
        raise RuntimeError("x_ref (or x_ref_cpu) must be defined.")
    if isinstance(x_ref_src, torch.Tensor) and x_ref_src.is_cuda:
        # prefer CPU pinned for streaming
        x_ref_src = x_ref_src.detach().cpu().pin_memory()
    s_twd_np = tweedie_score_np(x_t, t, x_ref_src, ref_chunk=ref_chunk, max_ref=max_ref)

    # true score for eval only
    s_true = true_score_xt_torch(x_t, t, means0, stds0, w0)

    return x_t, t, sigma, s_twd_np, s_true

# ----------------- Eval (γ-grid RMSE, EMA) -----------------
@torch.no_grad()
def eval_gamma_grid_rmse(n_t: int = EVAL_N_T, per_t: int = EVAL_PER_T):
    t_grid = make_gamma_times(n_t=n_t)
    ssum = 0.0
    for t in t_grid.tolist():
        x_eval = sample_evolved_gmm(per_t, t, means0, stds0, w0)     # [B,D]
        tcol   = torch.full((per_t,1), t, device=DEVICE, dtype=x_eval.dtype)
        sigma  = torch.sqrt((1.0 - torch.exp(-2.0*tcol)).clamp_min(1e-12))
        s_true = true_score_xt_torch(x_eval, tcol, means0, stds0, w0)
        pred   = net_twd_ema(x_eval, tcol) / sigma
        ssum  += rmse_t(pred, s_true).item()
    return {"TWDNN_TRUE": ssum / len(t_grid)}

# ===================== NEW: PROBE INTEGRATION =====================

# 1) build CuPy copies of params for the run_comparison utilities
means0_cp, stds0_cp, w0_cp = map(cp.asarray, (means0.detach().cpu().numpy(),
                                              stds0.detach().cpu().numpy(),
                                              (w0 / w0.sum()).detach().cpu().numpy()))

# 2) true score in CuPy space, using your existing helper from the run_comparison stack
def true_score_func_cp(y_cp, t_scalar: float):
    # relies on your project-defined function
    return calculate_true_score_at_t(
        y_cp, t_scalar, means0_cp, stds0_cp, w0_cp, batch_size=4096
    )

# 3) wrap EMA torch model → CuPy score(y,t)
@torch.no_grad()
def ema_score_xt_torch(x_t: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    if t.dim() == 1: t = t[:, None]
    sigma_t  = torch.sqrt((1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12))
    q_scaled = net_twd_ema(x_t, t)                      # predicts σ_t * s
    s = q_scaled / sigma_t
    return torch.nan_to_num(s, nan=0.0, posinf=0.0, neginf=0.0)

def make_custom_score_from_torch(score_xt_torch, device):
    def score_cp(y_cp, t_scalar):
        y_np = cp.asnumpy(y_cp)
        x_t  = torch.as_tensor(y_np, device=device, dtype=torch.float32)
        t    = torch.full((x_t.shape[0], 1), float(t_scalar), device=device, dtype=x_t.dtype)
        s    = score_xt_torch(x_t, t).detach().cpu().numpy().astype(np.float64)
        return cp.asarray(s)
    return score_cp

# 4) EXACT blackbox sampler (Heun-PC path included)
def blackbox_sampler(
    N_part, time_pts, custom_score, sampler_func, *,
    mode="heun_pc", h_coeff=0.5, true_score_func=None,
    p_prune=0, likelyhood_func=None, loglik_grad_fn=None
):
    xp  = cp
    dim = int(sampler_func(5).shape[1])           # match model/data dim
    y   = xp.random.randn(N_part, dim).astype(xp.float64)

    rmse_list = []                                # collect RMSE vs oracle

    for k in range(len(time_pts)-1):
        t_cur, t_prev = time_pts[k], time_pts[k+1]
        dt    = t_cur - t_prev
        noise = xp.random.randn(N_part, dim).astype(xp.float64)

        S_cur  = custom_score(y, t_cur)           # [N_part, dim]
        y_hat  = y + dt*(y + 2*S_cur) + xp.sqrt(2.0*dt)*noise
        S_prev = custom_score(y_hat, t_prev)

        if mode == "heun_pc":
            # predictor-corrector Heun step for the OU-like forward ODE/SDE drift
            drift_avg = 0.5*(y + 2*S_cur) + 0.5*(y_hat + 2*S_prev)
            y = y + dt*drift_avg + xp.sqrt(2.0 * dt) * noise
        elif mode == "ou_sde":
            y = y_hat
        elif mode == "pf_ode":
            y = y + dt*(y + S_cur)
        else:
            raise ValueError(mode)

    if true_score_func is not None:
        S_cur_true  = true_score_func(y, t_cur)
        S_prev_true = true_score_func(y_hat, t_prev)

        # These helpers are part of your run_comparison utilities
        S_cur_safe,  cur_idx_safe  = prune_cp_arr(S_cur,  p_percent=p_prune)
        S_prev_safe, prev_idx_safe = prune_cp_arr(S_prev, p_percent=p_prune)

        RMSE_cur  = float(xp_rmse(S_cur_safe,  S_cur_true[cur_idx_safe]))
        RMSE_prev = float(xp_rmse(S_prev_safe, S_prev_true[prev_idx_safe]))

        rmse_list.append(RMSE_cur)
        rmse_list.append(RMSE_prev)
        return y, rmse_list
    else:
        return y

# 5) bind EMA score to CuPy callback
_CUSTOM_SCORE = make_custom_score_from_torch(ema_score_xt_torch, DEVICE)

# 6) the probe wrapper (uses your run_comparison)
def probe_with_run_comparison(epoch_or_step: int, nsteps: int = 8, trials: int = 3,
                              p_prune: int = 50, N_ref = 2000, N_part = 2000):
    if 'run_comparison' not in globals():
        raise RuntimeError("run_comparison is not defined in this environment.")
    # ensure blackbox path override is used by run_comparison internals
    run_comparison.__globals__['blackbox_sampler'] = blackbox_sampler

    samplers = [
        ('heun_pc', 'custom', 'NN-EMA (DSM/Tweedie)', _CUSTOM_SCORE),
    ]
    _ = run_comparison(
        samplers           = samplers,
        steps_list         = [int(nsteps)],   # use requested nsteps
        true_sampler_func  = sampler_func,
        prior_sampler_func = sampler_func,
        prior_score_func   = score_func,
        true_score_func    = true_score_func_cp,
        true_init_score    = score_func,
        track_score_rmse   = True,
        N_ref              = N_ref,
        N_part             = N_part,
        trials             = int(trials),
        nrows              = 3,
        mean_bins          = 120,
        time_split         = 'power',
        T_end              = T_end,
        T_target           = T_target,
        div                = 'M_KSD',
        plot_hists         = True,
        hist_mode          = 'pca',
        display_mode       = 'min_label',
        trial_name         = f'TWDNN_hist_compare_e{epoch_or_step}',
        p_prune            = p_prune,
        ref_seed           = 1,
        plot_res           = False,
        hist_norm          = 2.0,
        plot_prior         = True,
        save_tag           = f"run_data/TWDNN_epoch_{epoch_or_step}",
    )

# =================== END PROBE INTEGRATION ===================

# ----------------- Training -------------------------
SUBSTEPS = int(globals().get("SUBSTEPS", 1))
REF_CHUNK_TRAIN = min(4096, int(globals().get("N_REF", 4096)))
MAX_REF_TRAIN   = int(globals().get("N_REF", 10_000))

for step in range(1, STEPS+1):
    for _ in range(SUBSTEPS):
        x_t, t, sigma_t, s_twd_np, s_true = make_batch(
            BATCH, ref_chunk=REF_CHUNK_TRAIN, max_ref=MAX_REF_TRAIN, t_mode=T_MODE
        )
        tgt_twd = sigma_t * s_twd_np  # train on sigma * score

        opt_twd.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=USE_AMP):
            pred_twd_scaled = net_twd(x_t, t)
            loss_twd = mse(pred_twd_scaled, tgt_twd)
        scaler_twd.scale(loss_twd).backward()
        scaler_twd.unscale_(opt_twd)
        torch.nn.utils.clip_grad_norm_(net_twd.parameters(), 1.0)
        scaler_twd.step(opt_twd); scaler_twd.update(); sched_twd.step()

        update_ema_all()

    if step % PRINT_EVERY == 0:
        metrics = eval_gamma_grid_rmse(n_t=EVAL_N_T, per_t=EVAL_PER_T)
        lr_now = sched_twd.get_last_lr()[0]
        print(f"[{step:5d}/{STEPS}] lr:{lr_now:.2e} | loss(DSM/Tweedie):{loss_twd.item():.3e}")
        print(f"  TRUE(γ): TNN:{metrics['TWDNN_TRUE']:.3e}")
        print()

    # ---- PROBE on cadence ----
    if (RUN_PROBE_EVERY > 0) and (step % RUN_PROBE_EVERY == 0):
        try:
            # --- end-to-end probe through EXACT blackbox path (Heun-PC) ---
            probe_with_run_comparison(step, nsteps=PROBE_STEPS, trials=PROBE_TRIALS)
        except Exception as e:
            print(f"[WARN] run_comparison probe failed: {e}\n")

# === Save the EMA net =========
def unwrap_module(m):
    if hasattr(m, "module"):    m = m.module
    if hasattr(m, "_orig_mod"): m = m._orig_mod
    return m

torch.save(
    {
        "net_twd": unwrap_module(net_twd_ema).state_dict(),
        "dim":     D,
        "hidden":  HIDDEN,
    },
    SAVE_PATH,
)
print(f"[INFO] EMA network written to {SAVE_PATH}")

# Optional final quick eval batch
do_final_eval = False
if do_final_eval:
    with torch.no_grad():
        x_t, t, sigma_t, s_twd_np, s_true = make_batch(N_TEST)
        pred_twd = net_twd_ema(x_t, t) / sigma_t
        print("\n========== FINAL (EMA) RMSE ==========")
        print(f"TWDNN_to_true: {rmse_t(pred_twd, s_true).item():.4e}")


/usr/local/lib/python3.12/dist-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


[ 1000/90000] lr:5.00e-03 | loss(DSM/Tweedie):1.372e-02
  TRUE(γ): TNN:5.939e+00

run_data/TWDNN_epoch_1000,  8 steps, NN-EMA (DSM/Tweedie): M_KSD =2.089e+04, M_KSD_floor = 4.007e+01, score_RMSE=1.836e+02, time=17.4s
 
[WARN] run_comparison probe failed: [Errno 2] No such file or directory: 'run_data/TWDNN_epoch_1000_8_steps_mean_hist.png'

[ 2000/90000] lr:4.99e-03 | loss(DSM/Tweedie):1.688e-02
  TRUE(γ): TNN:4.567e+00

run_data/TWDNN_epoch_2000,  8 steps, NN-EMA (DSM/Tweedie): M_KSD =1.261e+04, M_KSD_floor = 4.047e+01, score_RMSE=9.980e+01, time=4.6s
 
[WARN] run_comparison probe failed: [Errno 2] No such file or directory: 'run_data/TWDNN_epoch_2000_8_steps_mean_hist.png'

[ 3000/90000] lr:4.99e-03 | loss(DSM/Tweedie):1.573e-02
  TRUE(γ): TNN:3.815e+00

run_data/TWDNN_epoch_3000,  8 steps, NN-EMA (DSM/Tweedie): M_KSD =1.129e+04, M_KSD_floor = 4.054e+01, score_RMSE=8.379e+01, time=4.6s
 
[WARN] run_comparison probe failed: [Errno 2] No such file or directory: 'run_data/TWDNN_epoch_30

/tmp/ipython-input-3103856076.py:2880: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_w, fig_h), squeeze=False)


[WARN] run_comparison probe failed: [Errno 2] No such file or directory: 'run_data/TWDNN_epoch_21000_8_steps_mean_hist.png'

[22000/90000] lr:4.30e-03 | loss(DSM/Tweedie):7.804e-03
  TRUE(γ): TNN:2.618e+00

run_data/TWDNN_epoch_22000,  8 steps, NN-EMA (DSM/Tweedie): M_KSD =2.323e+03, M_KSD_floor = 3.993e+01, score_RMSE=4.168e+01, time=4.5s
 
[WARN] run_comparison probe failed: [Errno 2] No such file or directory: 'run_data/TWDNN_epoch_22000_8_steps_mean_hist.png'

[23000/90000] lr:4.24e-03 | loss(DSM/Tweedie):6.550e-03
  TRUE(γ): TNN:2.595e+00

run_data/TWDNN_epoch_23000,  8 steps, NN-EMA (DSM/Tweedie): M_KSD =2.728e+03, M_KSD_floor = 4.006e+01, score_RMSE=4.076e+01, time=4.5s
 
[WARN] run_comparison probe failed: [Errno 2] No such file or directory: 'run_data/TWDNN_epoch_23000_8_steps_mean_hist.png'

[24000/90000] lr:4.17e-03 | loss(DSM/Tweedie):1.075e-02
  TRUE(γ): TNN:2.582e+00

run_data/TWDNN_epoch_24000,  8 steps, NN-EMA (DSM/Tweedie): M_KSD =2.388e+03, M_KSD_floor = 4.080e+01, sc

In [ ]:
# ===========================
# Critic + Gate only (GPU/fast, stabilized)
#  - log-uniform t, σ_t-scaled critic target
#  - cosine LR, EMA, AMP, grad clipping
#  - γ-grid eval (RMSE to true), λ agreement
#  - NP targets only for metrics; no NN KSS/TWD heads
#  - *** s0 is now a learned proxy via local Gaussian kernel (PyTorch) ***

import math, os, time
import numpy as nptr
import torch
import torch.nn as nn
import torch.optim as optim

# -----------trrrreedee--------- Config & safe defaults --------------------
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
USE_AMP  = (DEVICE == "cuda")

# NEW: toggle to choose KSS/CSEM teacher: proxy s0 vs TRUE s0
#SAVE_PATH = "learned_score_critic_gate_ema_recomp.pt"  # <-- renamed
SAVE_PATHS =  ["learned_score_critic_gate_recomp_ema.pt"]
#["critic_gate_ema.pt", "learned_score_critic_gate_ema.pt", "learned_score_critic_gate_recomp_ema.pt"]
for SAVE_PATH in SAVE_PATHS:
    if SAVE_PATH == "critic_gate_ema.pt":
      USE_PROXY_SCORES = False
    else:
      USE_PROXY_SCORES = True

    if 'recomp' in SAVE_PATH:
      PROXY_RECOMPUTE = True
    else:
      PROXY_RECOMPUTE = False

    #USE_PROXY_SCORES = bool(globals().get("USE_PROXY_SCORES", True))
    print(f"[INFO] Using {'PROXY' if USE_PROXY_SCORES else 'TRUE'} initial scores for KSS/CSEM teacher.")

    #SAVE_PATH = "learned_score_critic_gate_ema.pt"  # <-- renamed
    #SAVE_PATH = "critic_gate_ema.pt"  # <-- renamed

    # Fallbacks if not defined upstream
    SEED        = globals().get("SEED", 0)
    NUM_C       = globals().get("NUM_C", 2000)
    K_DIM       = globals().get("K_DIM", 12)
    VARIANT     = globals().get("VARIANT", "helix")
    STD         = globals().get("STD", 0.12)
    SCALE       = globals().get("SCALE", 3.0)
    T_target    = globals().get("T_target", 5e-4)
    T_end       = globals().get("T_end", 1.5)
    N_Train     = globals().get("N_Train", globals().get("N_TRAIN", 500000))
    N_REF       = globals().get("N_REF", 5000)
    HIDDEN      = globals().get("HIDDEN", 512)
    HIDDEM_LAM  = globals().get("HIDDEM_LAM", 128)
    EVAL_N_T    = globals().get("EVAL_N_T", 24)
    EVAL_PER_T  = globals().get("EVAL_PER_T", 2048)
    GAMMA_PRIOR = globals().get("GAMMA_PRIOR", 1)
    EMBED_MODE = globals().get("EMBED_MODE", 'sine_wiggle')

    # ---- NEW: proxy-score controls ----
    PROXY_N           = int(globals().get("N_train", 500_000))
    PROXY_K_MIX       = 400 #int(globals().get("PROXY_K_MIX", 100))   # how many anchor comps to mix per query
    PROXY_COMP_K      = globals().get("PROXY_COMP_K", None)


    PROXY_RIDGE_FRAC        = 1e-6
    PROXY_ALPHA             = float(globals().get("PROXY_ALPHA", 0.4))
    PROXY_REF_CHUNK         = int(globals().get("PROXY_REF_CHUNK", 8192))
    PRECOMPUTE_TRAIN_SCORES = bool(globals().get("PRECOMPUTE_TRAIN_SCORES", True))
    PRECOMP_BATCH           = int(globals().get("PRECOMP_BATCH", 8192))
    SHOW_LAMBDA_STATS       = bool(globals().get("SHOW_LAMBDA_STATS", True))


    # ---- NEW: toggle & limits for KDE/GMM recomputation mode ----
    PROXY_GMM_BATCH                 = int(globals().get("PROXY_GMM_BATCH", 1024))
    PROXY_RECOMPUTE_MAX_ANCHORS     = 30000 #int(globals().get("N_train", 500_000))  # guardrail

    # ---------------- Base training knobs ----------------
    # (keep your existing knob defaults)
    STEPS = 29500 #int(globals().get("STEPS", 17000))
    do_final_eval = bool(globals().get("do_final_eval", False))

    ############## critic gate specific ###################
    CRIT_STEPS_PER_LAM = 3 #int(globals().get("CRIT_STEPS_PER_LAM", 4))
    WARMUP_LAM_STEPS   = 500
    LAM_EPS            = float(globals().get("LAM_EPS", 1e-4))
    LR_LAM_SCALE       = .66
    WD_LAM             = float(globals().get("WD_LAM", 1e-4))
    STEPS += WARMUP_LAM_STEPS
    #######################################################


    # ===== Helper: γ-prior time grid =====
    @torch.no_grad()
    def make_gamma_times(n_t: int = EVAL_N_T, gamma: float = GAMMA_PRIOR,
                        T_end: float = T_MAX, T_tgt: float = T_MIN, device: str = DEVICE):
        u = (torch.arange(n_t, device=device, dtype=torch.float32) + 0.5) / n_t
        return T_end * ((T_tgt / T_end) ** (u ** (1.0 / gamma)))

    def rmse_t(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        return torch.sqrt(torch.mean((a - b) ** 2))

    # ===== GMM helpers (you already have get_gmm_funcs) =====
    @torch.no_grad()
    def torch_sample_gmm(n: int, means: torch.Tensor, stds: torch.Tensor, weights: torch.Tensor) -> torch.Tensor:
        K, D = means.shape
        idx = torch.multinomial(weights, num_samples=n, replacement=True)
        noise = torch.randn(n, D, device=means.device) * stds[idx].unsqueeze(1)
        return means[idx] + noise

    @torch.no_grad()
    def torch_score_gmm_batch(x: torch.Tensor, means: torch.Tensor, stds: torch.Tensor, weights: torch.Tensor) -> torch.Tensor:
        """(Kept for 'true score' checks; no longer used as the teacher s0.)"""
        B, D = x.shape
        K = means.shape[0]
        inv_vars = 1.0 / (stds ** 2)
        log_w    = torch.log(weights + 1e-40)
        norm_const = -0.5 * D * (math.log(2 * math.pi) + torch.log(stds**2))
        diff = means.unsqueeze(0) - x.unsqueeze(1)                      # [B,K,D]
        d2   = (diff ** 2).sum(dim=-1)                                  # [B,K]
        logp = log_w + norm_const - 0.5 * d2 * inv_vars.unsqueeze(0)    # [B,K]
        w = torch.softmax(logp, dim=1)
        return (w.unsqueeze(-1) * diff * inv_vars.view(1, K, 1)).sum(dim=1)

    @torch.no_grad()
    def ou_params_at_t(means0: torch.Tensor, stds0: torch.Tensor, t: torch.Tensor):
        et   = torch.exp(-t)
        et2  = torch.exp(-2.0 * t)
        means_t = et.view(-1,1,1) * means0.unsqueeze(0)       # [B,K,D]
        var_t   = stds0.pow(2).unsqueeze(0) * et2 + (1.0 - et2)
        stds_t  = torch.sqrt(var_t + 1e-12)
        return means_t, stds_t

    @torch.no_grad()
    def true_score_xt_torch(x: torch.Tensor, t: torch.Tensor, means0: torch.Tensor, stds0: torch.Tensor, weights0: torch.Tensor) -> torch.Tensor:
        B, D = x.shape
        K, _ = means0.shape
        means_t, stds_t = ou_params_at_t(means0, stds0, t)
        inv_vars = 1.0 / (stds_t ** 2)
        log_w    = torch.log(weights0 + 1e-40).view(1, K).expand(B, K)
        norm_const = -0.5 * D * (math.log(2*math.pi) + torch.log(stds_t**2))
        diff = means_t - x.unsqueeze(1)                               # [B,K,D]
        d2   = (diff**2).sum(-1)
        logp = log_w + norm_const - 0.5 * d2 * inv_vars
        w    = torch.softmax(logp, dim=1)
        return (w.unsqueeze(-1) * diff * inv_vars.unsqueeze(-1)).sum(1)

    # ======= NEW: Learned proxy for initial score s0 (PyTorch) =======

    @torch.no_grad()
    def _adaptive_k_default(n_ref: int, D: int, alpha: float = PROXY_ALPHA) -> int:
        N0, k0 = 2000, max(4*D, 48)
        k_default = int(min(n_ref-1, max(1, round(k0 * (n_ref / N0) ** alpha))))
        return k_default

    @torch.no_grad()
    def _knn_topk_across_chunks(Q: torch.Tensor,
                                R: torch.Tensor,
                                k: int,
                                ref_chunk: int = PROXY_REF_CHUNK):
        """
        Maintain top-k smallest squared distances from each query in Q to rows in R
        without materializing the full distance matrix. Returns (best_d2, best_idx).
        """
        device = Q.device
        Nq, D = Q.shape
        Nr = R.shape[0]

        # Precompute squared norms
        q2 = (Q*Q).sum(dim=1, keepdim=True)           # [Nq,1]
        r2_all = (R*R).sum(dim=1)                     # [Nr]

        best_d2  = torch.full((Nq, k), float('inf'), device=device, dtype=Q.dtype)
        best_idx = torch.full((Nq, k), -1, device=device, dtype=torch.long)

        # Process reference in chunks
        arng = torch.arange(Nq, device=device)
        for start in range(0, Nr, ref_chunk):
            end = min(start + ref_chunk, Nr)
            Rc = R[start:end]                         # [Br,D]
            r2 = r2_all[start:end]                    # [Br]
            # d2 = ||q||^2 + ||r||^2 - 2 q r^T
            d2 = q2 + r2.view(1,-1) - 2.0 * (Q @ Rc.T)   # [Nq,Br]
            d2.clamp_(min=0)

            # Merge with running top-k
            d2_cat   = torch.cat([best_d2, d2], dim=1)   # [Nq, k+Br]
            idx_new  = start + torch.arange(Rc.shape[0], device=device)
            idx_cat  = torch.cat([best_idx, idx_new.view(1,-1).expand(Nq,-1)], dim=1)
            # pick smallest k
            d2_k, ord_k = torch.topk(d2_cat, k=k, dim=1, largest=False, sorted=False)
            idx_k = torch.gather(idx_cat, dim=1, index=ord_k)
            best_d2, best_idx = d2_k, idx_k

        return best_d2, best_idx



    @torch.no_grad()
    def kernel_proxy_score_torch(x_query: torch.Tensor,
                                x_anchor: torch.Tensor,
                                *,
                                k: int | None = None,
                                ridge_frac: float = PROXY_RIDGE_FRAC,
                                ref_chunk: int = PROXY_REF_CHUNK,
                                recompute: bool = False,
                                gmm_batch: int = PROXY_GMM_BATCH,
                                recompute_max_anchors: int = PROXY_RECOMPUTE_MAX_ANCHORS,
                                k_mix: int | None = None,  #New
                                comp_k: int | None = None,  #New
                                return_params: bool = False
                                ) -> torch.Tensor | tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        assert x_query.dim() == 2 and x_anchor.dim() == 2, "Inputs must be [N, D]"
        device = x_query.device
        dtype  = x_query.dtype
        Nq, D  = x_query.shape
        Nr, Da = x_anchor.shape
        assert Da == D, f"Dim mismatch: x_anchor has D={Da}, x_query has D={D}"

        if k is None:
            k = _adaptive_k_default(Nr, D)


        # defaults for recompute mode
        if k_mix is None:
            k_mix = min(k, Nr)          # how many comps we mix per query
        if comp_k is None:
            comp_k = min(k, Nr-1)       # how many neighbors per anchor for μ,var estimation
        comp_k = max(1, int(comp_k))
        k_mix  = max(1, int(k_mix))


        # ---------- Fast path: standard local estimator ----------
        if not recompute:
            # kNN from query -> anchors
            d2_knn, idx = _knn_topk_across_chunks(x_query, x_anchor, k=k, ref_chunk=ref_chunk)  # [Nq,k], [Nq,k]
            # adaptive bandwidth from max d2 among the k-NN (per query)
            tiny = torch.finfo(dtype).tiny
            h2   = torch.maximum(d2_knn.max(dim=1).values, torch.tensor(tiny, device=device, dtype=dtype))  # [Nq]
            w    = torch.exp(-d2_knn / (2.0 * h2[:, None]))                                                 # [Nq,k]
            denom= w.sum(dim=1, keepdim=True).clamp_min(torch.finfo(dtype).eps)                             # [Nq,1]
            X_nb = x_anchor[idx]                                                                            # [Nq,k,D]

            mu   = (w.unsqueeze(-1) * X_nb).sum(dim=1) / denom                                              # [Nq,D]
            diff = X_nb - mu.unsqueeze(1)                                                                    # [Nq,k,D]
            var  = (w.unsqueeze(-1) * (diff * diff)).sum(dim=1) / denom                                      # [Nq,D]

            # ridge proportional to mean variance per point
            tau   = ridge_frac * var.mean(dim=1, keepdim=True)                                              # [Nq,1]
            var_r = (var + tau).clamp_min(torch.finfo(dtype).eps)                                           # [Nq,D]

            score = (mu - x_query) / var_r                                                                   # [Nq,D]
            return (score, mu, var_r) if return_params else score

                    # ---------- Recompute path: gated mixture over compact anchor subset ----------
        if return_params:
            raise NotImplementedError("return_params=True is only supported for recompute=False (fast path).")

        # A) Gate: per-query k_mix nearest anchors (from FULL pool)
        d2_qk, idx_qk = _knn_topk_across_chunks(x_query, x_anchor, k=k_mix, ref_chunk=ref_chunk)  # [Nq,k_mix]

        # Build compact unique set by global frequency
        flat_idx = idx_qk.reshape(-1)
        counts   = torch.bincount(flat_idx, minlength=Nr)
        keep_n   = min(recompute_max_anchors, int((counts > 0).sum().item()))
        top_idx  = torch.topk(counts, k=keep_n, largest=True).indices
        uniq_idx = top_idx[counts[top_idx] > 0]      # [Nu]
        Nu       = uniq_idx.numel()

        # Map global anchor id -> position in compact set
        pos_map = torch.full((Nr,), -1, device=x_query.device, dtype=torch.long)
        pos_map[uniq_idx] = torch.arange(Nu, device=x_query.device)

        # Per-query validity after capping
        pos         = pos_map[idx_qk]                 # [Nq,k_mix] in [-1,Nu-1]
        has_valid   = pos.ge(0).any(dim=1)            # [Nq]
        no_valid    = ~has_valid                      # [Nq]

        # If some queries lost all anchors, compute their scores with the fast (non-recompute) path
        out = torch.empty(Nq, D, device=device, dtype=dtype)

        # B) Learn per-anchor diagonal params ONLY for kept anchors, using neighbors in FULL pool
        if Nu > 0:
            Q_comp = x_anchor[uniq_idx]  # [Nu,D]
            d2_all, idx_all = _knn_topk_across_chunks(Q_comp, x_anchor, k=min(comp_k+1, Nr), ref_chunk=ref_chunk)

            # remove self (if present); else drop farthest to keep size=comp_k
            ar        = uniq_idx.view(-1,1).expand(-1, idx_all.shape[1])
            is_self   = (idx_all == ar)
            has_self  = is_self.any(dim=1)

            keep_mask = torch.ones_like(idx_all, dtype=torch.bool)
            if has_self.any():
                self_pos = torch.argmax(is_self.int(), dim=1)
                keep_mask[torch.arange(Nu, device=device), self_pos] = False
            need_drop = ~has_self
            if need_drop.any():
                far_pos = torch.argmax(d2_all, dim=1)
                keep_mask[torch.arange(Nu, device=device)[need_drop], far_pos[need_drop]] = False

            idx_comp = idx_all[keep_mask].view(Nu, -1)  # [Nu,comp_k]
            d2_comp  = d2_all[keep_mask].view(Nu, -1)   # [Nu,comp_k]

            tiny   = torch.finfo(dtype).tiny
            eps    = torch.finfo(dtype).eps
            h2_c   = torch.maximum(d2_comp.max(dim=1).values, torch.tensor(tiny, device=device, dtype=dtype))
            w_c    = torch.exp(-d2_comp / (2.0 * h2_c[:, None]))                           # [Nu,comp_k]
            denomc = w_c.sum(dim=1, keepdim=True).clamp_min(eps)
            Xnb_c  = x_anchor[idx_comp]                                                    # [Nu,comp_k,D]

            mu_c   = (w_c.unsqueeze(-1) * Xnb_c).sum(dim=1) / denomc                       # [Nu,D]
            diffc  = Xnb_c - mu_c.unsqueeze(1)                                             # [Nu,comp_k,D]
            var_c  = (w_c.unsqueeze(-1) * (diffc*diffc)).sum(dim=1) / denomc               # [Nu,D]
            tau_c  = ridge_frac * var_c.mean(dim=1, keepdim=True)                          # [Nu,1]
            varr_c = (var_c + tau_c).clamp_min(eps)                                        # [Nu,D]

            two_pi   = torch.tensor(2.0 * math.pi, device=device, dtype=dtype)
            logdet_c = torch.sum(torch.log(two_pi * varr_c), dim=1)                        # [Nu]

            # C) Evaluate mixture for queries that HAVE at least one valid anchor
            if has_valid.any():
                sel      = has_valid.nonzero(as_tuple=True)[0]
                pos_sel  = pos[sel].clamp_min(0)             # [Nv,k_mix]
                mask_sel = pos[sel].ge(0)                    # [Nv,k_mix]
                x_sel    = x_query[sel]                      # [Nv,D]

                mu_sel   = mu_c[pos_sel]                     # [Nv,k_mix,D]
                var_sel  = varr_c[pos_sel]                   # [Nv,k_mix,D]
                logd_sel = logdet_c[pos_sel]                 # [Nv,k_mix]

                dX    = x_sel[:, None, :] - mu_sel
                quad  = torch.sum(dX * (dX / var_sel), dim=2)           # [Nv,k_mix]
                logw  = -0.5 * (quad + logd_sel)
                logw  = logw.masked_fill(~mask_sel, -float("inf"))

                m     = torch.max(logw, dim=1, keepdim=True).values     # [Nv,1], finite because at least one valid
                W     = torch.exp(logw - m)                              # [Nv,k_mix]
                compS = -(dX / var_sel)                                  # [Nv,k_mix,D]

                num   = torch.einsum("nk,nkd->nd", W, compS)             # [Nv,D]
                den   = W.sum(dim=1).clamp_min(eps)                      # [Nv]
                out[sel] = num / den[:, None]

        # D) Fast-path fallback for queries with no valid anchors
        if no_valid.any():
            sel_nv = no_valid.nonzero(as_tuple=True)[0]
            out[sel_nv] = kernel_proxy_score_torch(
                x_query[sel_nv], x_anchor,
                k=k, ridge_frac=ridge_frac, ref_chunk=ref_chunk,
                recompute=False,  # <— fast local estimator
                gmm_batch=gmm_batch
            )

        # Final numerical hygiene
        return torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)

        '''
        # ---------- Recompute path: gated mixture over compact anchor subset ----------
        if return_params:
            raise NotImplementedError("return_params=True is only supported for recompute=False (fast path).")

        # A) Gate: for each query, take its k_mix nearest anchors from the FULL pool
        d2_qk, idx_qk = _knn_topk_across_chunks(x_query, x_anchor, k=k_mix, ref_chunk=ref_chunk)  # [Nq,k_mix]

        # Make a compact unique set of anchors actually needed by this batch
        flat_idx = idx_qk.reshape(-1)
        # Count frequency to keep the most-used anchors if we exceed the cap
        counts = torch.bincount(flat_idx, minlength=Nr)
        # Keep at most recompute_max_anchors anchors; prefer most frequent
        keep_n = min(recompute_max_anchors, (counts > 0).sum().item())
        top_idx = torch.topk(counts, k=keep_n, largest=True).indices
        uniq_idx = top_idx[counts[top_idx] > 0]     # drop zeros if any
        Nu = uniq_idx.numel()
        # Map global anchor id -> position in compact set
        pos_map = torch.full((Nr,), -1, device=x_query.device, dtype=torch.long)
        pos_map[uniq_idx] = torch.arange(Nu, device=x_query.device)

        # B) Learn per-anchor diagonal components for ONLY the kept anchors,
        #    BUT using neighbors drawn from the FULL pool (global params).
        #    Q = x_anchor[uniq_idx], R = x_anchor
        Q_comp = x_anchor[uniq_idx]                       # [Nu,D]
        # comp_k+1 to allow removing self if it appears
        d2_all, idx_all = _knn_topk_across_chunks(Q_comp, x_anchor, k=min(comp_k+1, Nr), ref_chunk=ref_chunk)  # [Nu,comp_k+1]

        # Remove self if present; if not present, drop farthest to keep size = comp_k
        ar = uniq_idx.view(-1,1).expand(-1, idx_all.shape[1])        # [Nu,comp_k+1] (global ids)
        is_self = (idx_all == ar)
        has_self = is_self.any(dim=1)

        keep_mask = torch.ones_like(idx_all, dtype=torch.bool)
        if has_self.any():
            self_pos = torch.argmax(is_self.int(), dim=1)
            keep_mask[torch.arange(Nu, device=x_query.device), self_pos] = False
        need_drop_farthest = ~has_self
        if need_drop_farthest.any():
            far_pos = torch.argmax(d2_all, dim=1)
            keep_mask[torch.arange(Nu, device=x_query.device)[need_drop_farthest], far_pos[need_drop_farthest]] = False

        idx_comp = idx_all[keep_mask].view(Nu, -1)  # [Nu, comp_k]
        d2_comp  = d2_all[keep_mask].view(Nu, -1)   # [Nu, comp_k]

        # Local RBF weights with bandwidth h^2 = max d2 among neighbors (per anchor)
        tiny = torch.finfo(dtype).tiny
        h2_c = torch.maximum(d2_comp.max(dim=1).values, torch.tensor(tiny, device=device, dtype=dtype))  # [Nu]
        w_c  = torch.exp(-d2_comp / (2.0 * h2_c[:, None]))                                               # [Nu,comp_k]
        denom_c = w_c.sum(dim=1, keepdim=True).clamp_min(torch.finfo(dtype).eps)
        X_nb_c = x_anchor[idx_comp]                                                                       # [Nu,comp_k,D]

        mu_c  = (w_c.unsqueeze(-1) * X_nb_c).sum(dim=1) / denom_c                                        # [Nu,D]
        diffc = X_nb_c - mu_c.unsqueeze(1)                                                                # [Nu,comp_k,D]
        var_c = (w_c.unsqueeze(-1) * (diffc * diffc)).sum(dim=1) / denom_c                                # [Nu,D]
        tau_c = ridge_frac * var_c.mean(dim=1, keepdim=True)                                              # [Nu,1]
        varr_c = (var_c + tau_c).clamp_min(torch.finfo(dtype).eps)                                        # [Nu,D]

        # Precompute log det for diagonal Gaussians
        two_pi = torch.tensor(2.0 * math.pi, device=device, dtype=dtype)
        logdet_c = torch.sum(torch.log(two_pi * varr_c), dim=1)                                           # [Nu]

        # C) Evaluate mixture only over the gated anchors per query
        # Remap per-query indices to compact positions and mask dropped anchors
        pos = pos_map[idx_qk]                         # [Nq,k_mix] in [-1,Nu-1]
        mask_valid = pos.ge(0)
        pos_clamped = pos.clamp_min(0)

        mu_sel   = mu_c[pos_clamped]                  # [Nq,k_mix,D]
        var_sel  = varr_c[pos_clamped]                # [Nq,k_mix,D]
        logd_sel = logdet_c[pos_clamped]              # [Nq,k_mix]

        dX   = x_query[:, None, :] - mu_sel
        quad = torch.sum(dX * (dX / var_sel), dim=2)  # [Nq,k_mix]
        logw = -0.5 * (quad + logd_sel)
        logw.masked_fill_(~mask_valid, -float("inf"))

        m = torch.max(logw, dim=1, keepdim=True).values
        W = torch.exp(logw - m)
        comp_score = -(dX / var_sel)                  # [Nq,k_mix,D]
        num = torch.einsum("nk,nkd->nd", W, comp_score)
        den = W.sum(dim=1).clamp_min(torch.finfo(dtype).eps)
        return num / den[:, None]
        '''

    @torch.no_grad()
    def precompute_proxy_scores(dataset_cpu: torch.Tensor,
                                anchor_pool: torch.Tensor,
                                *,
                                batch: int = PRECOMP_BATCH,
                                k: int | None = None,
                                ridge_frac: float = PROXY_RIDGE_FRAC,
                                ref_chunk: int = PROXY_REF_CHUNK,
                                recompute: bool = PROXY_RECOMPUTE,
                                gmm_batch: int = PROXY_GMM_BATCH,
                                recompute_max_anchors: int = PROXY_RECOMPUTE_MAX_ANCHORS) -> torch.Tensor:
        N, D = dataset_cpu.shape
        out = torch.empty(N, D, dtype=torch.float32)
        for start in range(0, N, batch):
            end = min(start + batch, N)
            q = dataset_cpu[start:end].to(DEVICE, non_blocking=True)
            s = kernel_proxy_score_torch(
                q, anchor_pool, k=k, ridge_frac=ridge_frac, ref_chunk=ref_chunk,
                recompute=recompute, gmm_batch=gmm_batch, recompute_max_anchors=recompute_max_anchors,
                k_mix=PROXY_K_MIX, comp_k=PROXY_COMP_K)   #NEW
            # Repair any non-finite rows by falling back to the fast estimator
            bad = ~torch.isfinite(s).all(dim=1)
            if bad.any():
                s[bad] = kernel_proxy_score_torch(
                    q[bad], anchor_pool, k=k, ridge_frac=ridge_frac, ref_chunk=ref_chunk,
                    recompute=False, gmm_batch=gmm_batch)
            out[start:end] = s.detach().to('cpu')


        return out


    # ----------------- Nets ----------------------
    class LambdaNet(nn.Module):
        """Small MLP → scalar λ∈(0,1)."""
        def __init__(self, dim: int, hidden: int = 128):
            super().__init__()
            self.f = nn.Sequential(
                nn.Linear(dim + 1, hidden), nn.SiLU(),
                nn.Linear(hidden, hidden),  nn.SiLU(),
                nn.Linear(hidden, 1),
                nn.Sigmoid()
            )
        def forward(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
            return self.f(torch.cat([x, torch.log(t)], dim=1)).squeeze(-1)

    class CriticNet(nn.Module):
        """Outputs σ_t-scaled score estimate (we unscale at eval)."""
        def __init__(self, dim: int, hidden: int = 512):
            super().__init__()
            self.f = nn.Sequential(
                nn.Linear(dim + 1, hidden), nn.SiLU(),
                nn.Linear(hidden, hidden),  nn.SiLU(),
                nn.Linear(hidden, dim),
            )
        def forward(self, y: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
            return self.f(torch.cat([y, torch.log(t)], dim=1))

    def update_ema(target: nn.Module, src: nn.Module, decay: float):
        with torch.no_grad():
            for p_t, p_s in zip(target.parameters(), src.parameters()):
                p_t.data.mul_(decay).add_(p_s.data, alpha=(1.0 - decay))

    # ----------------- Build targets -----------------
    params, sampler_func, score_func = get_gmm_funcs(NUM_C, k_dim=K_DIM, variant=VARIANT, comp_std=STD,
                                                    overall_scale=SCALE, seed=0, embedding_mode = EMBED_MODE)[:3]

    means0_cp, stds0_cp, w0_cp = map(cp.asarray, params)
    means0_np, stds0_np, w0_np = params
    means0 = torch.as_tensor(means0_np, device=DEVICE).float().contiguous()
    stds0  = torch.as_tensor(stds0_np,  device=DEVICE).float().contiguous()
    w0     = torch.as_tensor(w0_np,     device=DEVICE).float().contiguous(); w0 /= w0.sum()


    def true_score_func_cp(y_cp, t_scalar):
        return calculate_true_score_at_t(
            y_cp, t_scalar, means0_cp, stds0_cp, w0_cp, batch_size=4096
        )


    # --- Build a large anchor pool (prior samples) for proxy s0 ---
    with torch.no_grad():
        proxy_pool = torch_sample_gmm(PROXY_N, means0, stds0, w0)  # [PROXY_N, D]

    # --- Reference anchors used in KSS/TWD teachers (always needed) ---
    with torch.no_grad():
        x_ref  = torch_sample_gmm(N_REF, means0, stds0, w0)  # [N,D]

    # --- Build s0_ref depending on USE_PROXY_SCORES ---
    if USE_PROXY_SCORES:
        # Build a large anchor pool (prior samples) for proxy s0
        with torch.no_grad():
            proxy_pool = torch_sample_gmm(PROXY_N, means0, stds0, w0)                              # [PROXY_N, D]
            s0_ref = kernel_proxy_score_torch(x_ref, proxy_pool,
                                              recompute_max_anchors=PROXY_RECOMPUTE_MAX_ANCHORS,   # NEW
                                              k_mix=PROXY_K_MIX, comp_k=PROXY_COMP_K,)             # PROXY teacher on refs
    else:
        # TRUE initial scores (no proxy pool or KDE/GMM work)
        with torch.no_grad():
            s0_ref = torch_score_gmm_batch(x_ref, means0, stds0, w0)

    D = means0.shape[1]

    # ------- Gate (λ-net) -------
    lam_net     = LambdaNet(D, hidden=HIDDEM_LAM).to(DEVICE)
    lam_net_ema = LambdaNet(D, hidden=HIDDEM_LAM).to(DEVICE)
    lam_net_ema.load_state_dict(lam_net.state_dict())
    print(f"[INFO] Gate init: random (D={D}).")

    # ------- Critic -------
    critic      = CriticNet(D, hidden=HIDDEN).to(DEVICE)
    critic_ema  = CriticNet(D, hidden=HIDDEN).to(DEVICE)
    critic_ema.load_state_dict(critic.state_dict())

    # Unscaled EMA score from the critic.
    # During training the critic target was:
    #   q_scaled = sigma_t * ((1-λ) * a + λ * b)  ≈  sigma_t * s_theta(x_t, t)
    # so at inference we simply undo the scale:
    #   s_theta(x_t, t) ≈ q_scaled / sigma_t


    # optional compile
    try:
        if DEVICE == "cuda":
            lam_net     = torch.compile(lam_net)
            critic      = torch.compile(critic)
            lam_net_ema = torch.compile(lam_net_ema)
            critic_ema  = torch.compile(critic_ema)
    except Exception:
        pass

    opt_lam    = optim.Adam(lam_net.parameters(), lr=LR_INIT*LR_LAM_SCALE, weight_decay=WD_LAM)
    opt_crit   = optim.Adam(critic.parameters(),  lr=LR_INIT)
    sched_lam  = torch.optim.lr_scheduler.CosineAnnealingLR(opt_lam,  T_max=STEPS, eta_min=LR_MIN*LR_LAM_SCALE)
    sched_crit = torch.optim.lr_scheduler.CosineAnnealingLR(opt_crit, T_max=STEPS, eta_min=LR_MIN)
    scaler_crit = torch.amp.GradScaler('cuda', enabled=USE_AMP)
    scaler_lam  = torch.amp.GradScaler('cuda', enabled=USE_AMP)
    mse        = nn.MSELoss()

    def update_all_ema():
        update_ema(lam_net_ema, lam_net, EMA_DECAY)
        update_ema(critic_ema, critic, EMA_DECAY)

    # ----------------- Fixed training dataset (CPU pinned, deterministic) -----------------
    TRAIN_SEED  = int(globals().get("TRAIN_SEED", globals().get("SEED", 0)))
    _gen_cpu    = torch.Generator(device='cpu').manual_seed(TRAIN_SEED)

    def _sample_gmm_cpu_with_gen(n, means_cpu, stds_cpu, w_cpu, gen):
        K, D = means_cpu.shape
        idx   = torch.multinomial(w_cpu, num_samples=n, replacement=True, generator=gen)
        noise = torch.randn(n, D, generator=gen, dtype=means_cpu.dtype)
        return means_cpu[idx] + noise * stds_cpu[idx].unsqueeze(1)

    with torch.no_grad():
        _means_cpu = means0.detach().cpu().float()
        _stds_cpu  = stds0.detach().cpu().float()
        _w_cpu     = (w0 / w0.sum()).detach().cpu().float()

        x0_train_cpu  = _sample_gmm_cpu_with_gen(N_Train, _means_cpu, _stds_cpu, _w_cpu, _gen_cpu).pin_memory()
        u_train_cpu   = torch.rand(N_Train, 1, generator=_gen_cpu, dtype=torch.float32).pin_memory()
        eps_train_cpu = torch.randn(N_Train, _means_cpu.shape[1], generator=_gen_cpu, dtype=torch.float32).pin_memory()

    # --- Optional precompute of s0 for training pool (only for PROXY mode) ---
    S0_TRAIN_CPU = None
    if USE_PROXY_SCORES and PRECOMPUTE_TRAIN_SCORES:
        print(f"[INFO] Precomputing learned s0 proxies for training pool of size {N_Train}..."
              + (" (recompute/KDE mode)" if PROXY_RECOMPUTE else ""))
        S0_TRAIN_CPU = precompute_proxy_scores(
            x0_train_cpu, proxy_pool, batch=PRECOMP_BATCH
        ).pin_memory()
        print("[INFO] Done precomputing s0 proxies.")

    _train_ptr  = 0
    _train_perm = torch.randperm(N_Train, generator=_gen_cpu)

    def _next_batch_indices(B):
        global _train_ptr, _train_perm
        if _train_ptr + B <= N_Train:
            idx = _train_perm[_train_ptr:_train_ptr+B]
            _train_ptr += B
            return idx
        tail = _train_perm[_train_ptr:]
        _train_perm = torch.randperm(N_Train, generator=_gen_cpu)
        _train_ptr  = 0
        need = B - tail.numel()
        head = _train_perm[_train_ptr:_train_ptr+need]
        _train_ptr += need
        return torch.cat([tail, head], dim=0)

    # ----------------- NP teachers using proxy s0_ref -----------------
    @torch.no_grad()
    def kss_score_torch(y: torch.Tensor, t: torch.Tensor, x_ref: torch.Tensor, s0_ref: torch.Tensor) -> torch.Tensor:
        B, D = y.shape
        N    = x_ref.shape[0]
        et   = torch.exp(-t); et2 = torch.exp(-2.0 * t)
        var  = (1.0 - et2).clamp_min(1e-12); invv = 1.0 / var
        mu   = et.view(B,1,1) * x_ref.view(1,N,D)
        diff = y.view(B,1,D) - mu
        d2   = (diff**2).sum(-1)
        logw = -0.5 * invv.view(B,1) * d2
        w    = torch.softmax(logw, dim=1)
        return torch.exp(t).view(B,1) * (w @ s0_ref)

    @torch.no_grad()
    def tweedie_score_torch(y: torch.Tensor, t: torch.Tensor, x_ref: torch.Tensor) -> torch.Tensor:
        B, D = y.shape
        N    = x_ref.shape[0]
        et   = torch.exp(-t); et2 = torch.exp(-2.0 * t)
        var  = (1.0 - et2).clamp_min(1e-12); invv = 1.0 / var
        mu   = et.view(B,1,1) * x_ref.view(1,N,D)
        diff = y.view(B,1,D) - mu
        d2   = (diff**2).sum(-1)
        logw = -0.5 * invv.view(B,1) * d2
        w    = torch.softmax(logw, dim=1)
        mu_bar = (w.unsqueeze(-1) * mu).sum(dim=1)
        return -invv.view(B,1) * (y - mu_bar)

    @torch.no_grad()
    def snis_lambda_torch(
        y, t, x_ref, s0_ref,
        s_kss=None, s_twd=None, eps: float = 1e-12,
        max_ref: int | None = 10_000, chunk: int = 4096
    ):
        """Streaming SNIS λ*: (Var[â]-Cov[â,b̂])/(Var[â]+Var[b̂]-2Cov[â,b̂])."""
        B, D = y.shape
        N = x_ref.shape[0]
        if (max_ref is not None) and (N > max_ref):
            idx = torch.randperm(N, device=x_ref.device)[:max_ref]
            x_ref  = x_ref[idx]
            s0_ref = s0_ref[idx]
            N = x_ref.shape[0]

        et   = torch.exp(-t)                     # [B,1]
        et2  = torch.exp(-2.0 * t)
        var  = (1.0 - et2).clamp_min(1e-12)      # [B,1]
        invv = 1.0 / var

        # Pass 1
        m = torch.full((B,), -float("inf"), device=y.device, dtype=y.dtype)
        s = torch.zeros(B, device=y.device, dtype=y.dtype)
        acc_a = torch.zeros(B, D, device=y.device, dtype=y.dtype)
        acc_b = torch.zeros(B, D, device=y.device, dtype=y.dtype)

        for start in range(0, N, chunk):
            end = min(start + chunk, N)
            xr = x_ref[start:end]
            s0 = s0_ref[start:end]
            mu   = et.view(B,1,1) * xr.view(1,-1,D)
            diff = y.view(B,1,D) - mu
            d2   = (diff**2).sum(-1)
            L    = -0.5 * invv.view(B,1) * d2
            m_new = torch.maximum(m, L.max(dim=1).values)
            scale = torch.exp(m - m_new)
            wtil  = torch.exp(L - m_new[:,None])

            a_i   = torch.exp(t).view(B,1,1) * s0.view(1,-1,D)
            b_i   = -(invv.view(B,1,1)) * (y.view(B,1,D) - mu)
            s     = s * scale + wtil.sum(1)
            acc_a = acc_a * scale[:,None] + torch.einsum('bn,bnd->bd', wtil, a_i)
            acc_b = acc_b * scale[:,None] + torch.einsum('bn,bnd->bd', wtil, b_i)
            m = m_new

        mu_a = acc_a / s[:,None]
        mu_b = acc_b / s[:,None]

        # Pass 2
        S0    = torch.zeros(B, device=y.device, dtype=y.dtype)
        Vknum = torch.zeros(B, device=y.device, dtype=y.dtype)
        Vtnum = torch.zeros(B, device=y.device, dtype=y.dtype)
        Cnum  = torch.zeros(B, device=y.device, dtype=y.dtype)
        logZ  = torch.log(s) + m

        for start in range(0, N, chunk):
            end = min(start + chunk, N)
            xr = x_ref[start:end]
            s0 = s0_ref[start:end]
            mu   = et.view(B,1,1) * xr.view(1,-1,D)
            diff = y.view(B,1,D) - mu
            d2   = (diff**2).sum(-1)
            L    = -0.5 * invv.view(B,1) * d2
            w    = torch.exp(L - logZ[:,None])
            w2   = w * w
            S0  += w2.sum(1)

            a_i  = torch.exp(t).view(B,1,1) * s0.view(1,-1,D)
            b_i  = -(invv.view(B,1,1)) * (y.view(B,1,D) - mu)
            ac   = a_i - mu_a[:,None,:]
            bc   = b_i - mu_b[:,None,:]

            Vknum += (w2[:,:,None] * (ac*ac)).sum((1,2))
            Vtnum += (w2[:,:,None] * (bc*bc)).sum((1,2))
            Cnum  += (w2[:,:,None] * (ac*bc)).sum((1,2))

        den = (1.0 - S0).clamp_min(1e-10)
        Vk  = Vknum / den
        Vt  = Vtnum / den
        C   =  Cnum / den
        lam = ((Vk - C) / (Vk + Vt - 2.0*C + eps)).clamp(0.0, 1.0)
        return lam

    # ----------------- Batch maker (log-uniform t) -----------------
    _train_indices_last = None  # for exposing the last indices to caller
    @torch.no_grad()
    def make_batch(B, *, ref_chunk=None, max_ref=None, t_mode='log', return_idx=True):
        global _train_indices_last
        if ref_chunk is None:
            ref_chunk = min(4096, int(globals().get("N_REF", 4096)))
        if max_ref is None:
            max_ref = int(globals().get("N_REF", 10_000))

        # fixed pool slicing
        idx = _next_batch_indices(B)
        _train_indices_last = idx  # store
        x0  = x0_train_cpu[idx].to(DEVICE, non_blocking=True)
        u   = u_train_cpu[idx].to(DEVICE, non_blocking=True)
        eps = eps_train_cpu[idx].to(DEVICE, non_blocking=True)

        if t_mode == 'log':
            logTmin = math.log(T_MIN); logTmax = math.log(T_MAX)
            t = torch.exp(torch.tensor(logTmin, device=DEVICE) + u * (logTmax - logTmin))
        elif t_mode == 'uniform':
            t = torch.tensor(T_MIN, device=DEVICE) + u * (T_MAX - T_MIN)
        else:
            raise ValueError(f"Unknown t_mode: {t_mode}")

        et    = torch.exp(-t)
        sigma = torch.sqrt((1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12))
        x_t   = et * x0 + sigma * eps

        # streamed NP teachers (for metrics and SNIS λ)
        x_ref_src, s0_ref_src = x_ref, s0_ref
        N_total = x_ref_src.shape[0]
        if (max_ref is not None) and (N_total > max_ref):
            sel = torch.randperm(N_total, generator=_gen_cpu, device='cpu')[:max_ref]
            x_ref_src  = x_ref_src[sel]
            s0_ref_src = s0_ref_src[sel]
            N_total    = max_ref

        Bsz, D = x_t.shape
        y      = x_t
        invv   = 1.0 / (1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12)
        y2     = (y*y).sum(-1)

        m      = torch.full((Bsz,), -float("inf"), device=DEVICE, dtype=y.dtype)
        s      = torch.zeros(Bsz, device=DEVICE, dtype=y.dtype)
        num_x  = torch.zeros(Bsz, D, device=DEVICE, dtype=y.dtype)
        num_s0 = torch.zeros(Bsz, D, device=DEVICE, dtype=y.dtype)

        for start in range(0, N_total, ref_chunk):
            end = min(start + ref_chunk, N_total)
            xr  = x_ref_src[start:end].to(DEVICE, non_blocking=True).contiguous()
            s0  = s0_ref_src[start:end].to(DEVICE, non_blocking=True).contiguous()

            xr2   = (xr*xr).sum(-1)
            cross = y @ xr.T
            d2    = y2[:,None] + (et*et) * xr2[None,:] - 2.0 * et * cross
            L     = -0.5 * invv * d2

            m_new = torch.maximum(m, L.max(dim=1).values)
            scale = torch.exp(m - m_new)
            wtil  = torch.exp(L - m_new[:,None])

            s      = s * scale + wtil.sum(1)
            num_x  = num_x  * scale[:,None] + torch.einsum('bn,nd->bd', wtil, xr)
            num_s0 = num_s0 * scale[:,None] + torch.einsum('bn,nd->bd', wtil, s0)
            m      = m_new

        x_bar  = num_x  / (s[:,None] + 1e-30)
        s0_bar = num_s0 / (s[:,None] + 1e-30)

        s_kss  = torch.exp(t) * s0_bar
        s_twd  = -invv * (y - torch.exp(-t) * x_bar)
        s_true = true_score_xt_torch(x_t, t, means0, stds0, w0)

        if return_idx:
            return x_t, t, sigma, s_kss, s_twd, s_true, x0, idx
        else:
            return x_t, t, sigma, s_kss, s_twd, s_true, x0

    @torch.no_grad()
    def sample_evolved_gmm(n: int, t_scalar: float,
                          means0: torch.Tensor, stds0: torch.Tensor, w0: torch.Tensor) -> torch.Tensor:
        # Draw x_t directly from the OU-evolved mixture (means_t, stds_t).
        t  = torch.tensor(t_scalar, device=means0.device, dtype=means0.dtype)
        et = torch.exp(-t); et2 = torch.exp(-2.0 * t)
        means_t = means0 * et                               # [K,D]
        var_t   = stds0**2 * et2 + (1.0 - et2)             # [K,D]
        stds_t  = torch.sqrt(var_t.clamp_min(1e-12))       # [K,D]
        return torch_sample_gmm(n, means_t, stds_t, w0)

    # ----------------- Eval (γ-grid) -----------------
    @torch.no_grad()
    def eval_gamma_grid_rmse(n_t: int = EVAL_N_T, per_t: int = EVAL_PER_T):
        t_grid = make_gamma_times(n_t=n_t)
        sums = {k: 0.0 for k in [
            "KSSNP_TRUE","TWDNP_TRUE","BLNDNP_TRUE_NP","BLNDNP_TRUE_NN",
            "LAM_ABS_DIFF","LAM_PEARSON","Critic_RMSE"
        ]}
        cnt_total = 0
        sum_np = 0.0; sum_nn = 0.0
        sum_np2 = 0.0; sum_nn2 = 0.0; sum_prod = 0.0

        for t in t_grid.tolist():
            x_eval = sample_evolved_gmm(per_t, t, means0, stds0, w0)
            tcol   = torch.full((per_t,1), t, device=DEVICE, dtype=x_eval.dtype)
            sigma  = torch.sqrt((1.0 - torch.exp(-2.0*tcol)).clamp_min(1e-12))

            s_true = true_score_xt_torch(x_eval, tcol, means0, stds0, w0)
            s_kss  = kss_score_torch(x_eval, tcol, x_ref, s0_ref)       # uses proxy s0_ref
            s_twd  = tweedie_score_torch(x_eval, tcol, x_ref)

            pred_crit = critic_ema(x_eval, tcol) / sigma

            lam_np = snis_lambda_torch(x_eval, tcol, x_ref, s0_ref, s_kss, s_twd).view(-1,1)
            lam_nn = lam_net_ema(x_eval, tcol).view(-1,1)

            s_bl_np_np = (1 - lam_np)*s_kss + lam_np*s_twd
            s_bl_np_nn = (1 - lam_nn)*s_kss + lam_nn*s_twd

            sums["KSSNP_TRUE"]     += rmse_t(s_kss,   s_true).item()
            sums["TWDNP_TRUE"]     += rmse_t(s_twd,   s_true).item()
            sums["BLNDNP_TRUE_NP"] += rmse_t(s_bl_np_np, s_true).item()
            sums["BLNDNP_TRUE_NN"] += rmse_t(s_bl_np_nn, s_true).item()
            sums["Critic_RMSE"]    += rmse_t(pred_crit, s_true).item()

            d = (lam_nn - lam_np).abs().mean().item()
            sums["LAM_ABS_DIFF"]   += d

            ln = lam_nn.view(-1); lp = lam_np.view(-1)
            cnt_total += ln.numel()
            sum_nn  += ln.sum().item(); sum_np  += lp.sum().item()
            sum_nn2 += (ln*ln).sum().item(); sum_np2 += (lp*lp).sum().item()
            sum_prod+= (ln*lp).sum().item()

        for k in sums:
            if k != "LAM_PEARSON":
                sums[k] /= len(t_grid)

        mean_nn = sum_nn / cnt_total; mean_np = sum_np / cnt_total
        var_nn  = max(0.0, sum_nn2 / cnt_total - mean_nn**2)
        var_np  = max(0.0, sum_np2 / cnt_total - mean_np**2)
        cov     = (sum_prod / cnt_total) - mean_nn*mean_np
        den = max(1e-12, math.sqrt(max(var_nn, 0.0) * max(var_np, 0.0)))
        sums["LAM_PEARSON"] = (cov / den) if den > 0 else 0.0
        return sums


    # ===== add once, above your training loop =====
    import numpy as np, cupy as cp, torch
    @torch.no_grad()
    def ema_score_xt_torch(x_t: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        if t.dim() == 1: t = t[:, None]
        sigma_t  = torch.sqrt((1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12))
        q_scaled = critic_ema(x_t, t)
        s = q_scaled / sigma_t
        return torch.nan_to_num(s, nan=0.0, posinf=0.0, neginf=0.0)

    def make_custom_score_from_torch(score_xt_torch, device):
        def score_cp(y_cp, t_scalar):
            y_np = cp.asnumpy(y_cp)
            x_t  = torch.as_tensor(y_np, device=device, dtype=torch.float32)
            t    = torch.full((x_t.shape[0], 1), float(t_scalar), device=device, dtype=x_t.dtype)
            s    = score_xt_torch(x_t, t).detach().cpu().numpy().astype(np.float64)
            return cp.asarray(s)
        return score_cp

    # --- your EXACT blackbox sampler ---
    def blackbox_sampler(
        N_part, time_pts, custom_score, sampler_func, *,
        mode="heun_hop", h_coeff=0.5, true_score_func=None,
        p_prune = 0, likelyhood_func = None, loglik_grad_fn = None):
        xp  = cp
        dim = int(sampler_func(5).shape[1])           # match model/data dim
        y   = xp.random.randn(N_part, dim).astype(xp.float64)

        rmse_list = []                                # collect RMSE vs oracle

        for k in range(len(time_pts)-1):
            t_cur, t_prev = time_pts[k], time_pts[k+1]
            dt    = t_cur - t_prev
            noise = xp.random.randn(N_part, dim).astype(xp.float64)

            S_cur  = custom_score(y, t_cur)           # [N_part, dim]
            y_hat  = y + dt*(y + 2*S_cur) + xp.sqrt(2.0*dt)*noise
            S_prev = custom_score(y_hat, t_prev)

            if mode == "heun_hop":
                A = xp.exp(dt); B = xp.exp(2*dt) - 1.0; C = xp.sqrt(B); D = B / A
                y = A*y + h_coeff*B*S_prev + (1-h_coeff)*D*S_cur + C*noise
            elif mode == "ou_sde":
                y = y_hat
            elif mode == "pf_ode":
                y = y + dt*(y + S_cur)
            elif mode == "heun_pc":
                drift_avg = 0.5*(y + 2*S_cur) + 0.5*(y_hat + 2*S_prev)
                y = y + dt*drift_avg + xp.sqrt(2.0 * dt) * noise
            else:
                raise ValueError(mode)

        if true_score_func is not None:
            S_cur_true  = true_score_func(y, t_cur)
            S_prev_true = true_score_func(y_hat, t_prev)

            S_cur_safe,  cur_idx_safe  = prune_cp_arr(S_cur,  p_percent = p_prune)
            S_prev_safe, prev_idx_safe = prune_cp_arr(S_prev, p_percent = p_prune)

            RMSE_cur  = float(xp_rmse(S_cur_safe,  S_cur_true[cur_idx_safe]))
            RMSE_prev = float(xp_rmse(S_prev_safe, S_prev_true[prev_idx_safe]))

            rmse_list.append(RMSE_cur)
            rmse_list.append(RMSE_prev)
            return y, rmse_list
        else:
            return y

    # build once and reuse
    _CUSTOM_SCORE = make_custom_score_from_torch(ema_score_xt_torch, DEVICE)

    def probe_with_run_comparison(epoch_or_step: int, N_ref = 2000, N_part = 2000,
                                  nsteps: int = 8 , trials = 3, p_prune = 50, plot_hists = False):
        # force run_comparison to use THIS blackbox sampler
        run_comparison.__globals__['blackbox_sampler'] = blackbox_sampler

        samplers = [
            ('heun_pc', 'custom', 'NN-Critic-EMA', _CUSTOM_SCORE),
        ]
        _ = run_comparison(
            samplers           = samplers,
            steps_list         = [16],
            true_sampler_func  = sampler_func,
            prior_sampler_func = sampler_func,
            prior_score_func   = score_func,
            true_score_func    = true_score_func_cp,
            true_init_score    = score_func,
            track_score_rmse   = True,
            N_ref              = N_ref,
            N_part             = N_part,
            trials             = trials,
            nrows              = 3,
            mean_bins          = 80,
            time_split         = 'power',
            T_end              = T_end,
            T_target           = T_target,
            div                = 'M_KSD',
            plot_hists         = plot_hists,
            hist_mode          = 'pca',
            display_mode       = 'min_label',
            trial_name         = f'NN_hist_compare_e{epoch_or_step}',
            p_prune            = p_prune,
            ref_seed           = 1,
            plot_res           = False,
            hist_norm          = 2.0,
            plot_prior         = True,
            save_tag           = f"run_data/BLEND_epoch_{epoch_or_step}",
        )

    # ----------------- Training -------------------------
    SUBSTEPS = int(globals().get("SUBSTEPS", 1))
    REF_CHUNK_TRAIN = min(4096, int(globals().get("N_REF", 4096)))
    MAX_REF_TRAIN   = int(globals().get("N_REF", 10_000))

    def _s0_teacher_for_batch(x0_batch: torch.Tensor, idx_in_train: torch.Tensor | None):
        """
        Returns s0(x0_batch) from either the PROXY estimator (if enabled) or the TRUE teacher.
        Skips all proxy work when USE_PROXY_SCORES=False.
        """
        if USE_PROXY_SCORES:
            # Use precomputed or on-the-fly proxy scores
            if (S0_TRAIN_CPU is not None) and (idx_in_train is not None):
                return S0_TRAIN_CPU[idx_in_train].to(DEVICE, non_blocking=True)
            return kernel_proxy_score_torch(
                x0_batch, proxy_pool,
                recompute=PROXY_RECOMPUTE, gmm_batch=PROXY_GMM_BATCH,
                ref_chunk=PROXY_REF_CHUNK, ridge_frac=PROXY_RIDGE_FRAC,
                recompute_max_anchors=PROXY_RECOMPUTE_MAX_ANCHORS,
                k_mix=PROXY_K_MIX, comp_k=PROXY_COMP_K,
            )
        else:
            # TRUE t=0 GMM score (fast, no proxy deps)
            return torch_score_gmm_batch(x0_batch, means0, stds0, w0)


    def _s0_proxy_for_batch(x0_batch: torch.Tensor, idx_in_train: torch.Tensor | None):
        if (S0_TRAIN_CPU is not None) and (idx_in_train is not None):
            return S0_TRAIN_CPU[idx_in_train].to(DEVICE, non_blocking=True)
        return kernel_proxy_score_torch(
            x0_batch, proxy_pool,
            recompute=PROXY_RECOMPUTE, gmm_batch=PROXY_GMM_BATCH,
            ref_chunk=PROXY_REF_CHUNK, ridge_frac=PROXY_RIDGE_FRAC,
            recompute_max_anchors=PROXY_RECOMPUTE_MAX_ANCHORS,
            k_mix=PROXY_K_MIX, comp_k=PROXY_COMP_K,
        )

    for step in range(1, STEPS+1):
        for _ in range(SUBSTEPS):
            for _ in range(CRIT_STEPS_PER_LAM):
                  x_t, t, sigma_t, s_kss_np, s_twd_np, s_true, x0, idx_train = make_batch(
                      BATCH, ref_chunk=REF_CHUNK_TRAIN, max_ref=MAX_REF_TRAIN, t_mode='log', return_idx=True
                  )
                  #######
                  with torch.no_grad():
                      et   = torch.exp(-t)
                      invv = 1.0 / (1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12)


                      s0_base  = _s0_teacher_for_batch(x0, idx_train)
                      z_base_twd = x0

                      a = torch.exp(t) * s0_base
                      b = -invv * (x_t - et * z_base_twd)

                      lam_frozen = lam_net_ema(x_t, t).unsqueeze(-1).clamp(LAM_EPS, 1.0 - LAM_EPS)
                      z_scaled   = sigma_t * ((1 - lam_frozen) * a + lam_frozen * b)

                  opt_crit.zero_grad(set_to_none=True)
                  with torch.amp.autocast('cuda', enabled=USE_AMP):
                      q_scaled = critic(x_t, t)
                      loss_crit = mse(q_scaled, z_scaled)

                  scaler_crit.scale(loss_crit).backward()
                  scaler_crit.unscale_(opt_crit)
                  torch.nn.utils.clip_grad_norm_(critic.parameters(), 1.0)
                  scaler_crit.step(opt_crit)
                  scaler_crit.update()
                  sched_crit.step()
                  update_ema(critic_ema, critic, EMA_DECAY)

        # ---------------- Gate: 1 step against frozen critic from EMA ---------------
        x_t, t, sigma_t, s_kss_np, s_twd_np, s_true, x0, idx_train = make_batch(
            BATCH, ref_chunk=REF_CHUNK_TRAIN, max_ref=MAX_REF_TRAIN, t_mode='log', return_idx=True
        )
        #####
        with torch.no_grad():
            et   = torch.exp(-t)
            invv = 1.0 / (1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12)

            s0_base  = _s0_teacher_for_batch(x0, idx_train)
            z_base_twd = x0

            a = torch.exp(t) * s0_base
            b = -invv * (x_t - et * z_base_twd)
            q_scaled_frozen = critic_ema(x_t, t)

        opt_lam.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=USE_AMP):
            lam_pred = lam_net(x_t, t).unsqueeze(-1).clamp(LAM_EPS, 1.0 - LAM_EPS)
            z_scaled = sigma_t * ((1 - lam_pred) * a + lam_pred * b)
            loss_lam = mse(z_scaled, q_scaled_frozen)

        if step > WARMUP_LAM_STEPS:
            scaler_lam.scale(loss_lam).backward()
            scaler_lam.unscale_(opt_lam)
            torch.nn.utils.clip_grad_norm_(lam_net.parameters(), 1.0)
            scaler_lam.step(opt_lam)
            scaler_lam.update()
            sched_lam.step()
            update_ema(lam_net_ema, lam_net, EMA_DECAY)
        else:
            sched_lam.step()

    # ---------------- Logging every PRINT_EVERY steps ----------------
        if step % PRINT_EVERY == 0:
            metrics = eval_gamma_grid_rmse(n_t=EVAL_N_T, per_t=EVAL_PER_T)
            lr_c = sched_crit.get_last_lr()[0]; lr_l = sched_lam.get_last_lr()[0]
            lam_mean = lam_net_ema(x_t, t).mean().item()
            lam_std  = lam_net_ema(x_t, t).std(unbiased=False).item()
            print(f"[{step:5d}/{STEPS}] lr(crit):{lr_c:.2e} lr(λ):{lr_l:.2e} | "
                  f"loss(crit):{loss_crit.item():.3e}"
                  + (f" loss(λ):{loss_lam.item():.3e}" if step>WARMUP_LAM_STEPS else " (λ warmup)"))
            print(f"  λ(EMA) mean={lam_mean:.3f} std={lam_std:.3f}")
            print("  TRUE(γ): "
                  f"Critic RMSE:{metrics['Critic_RMSE']:.3e}  "
                  f"KNP:{metrics['KSSNP_TRUE']:.3e}  "
                  f"TNP:{metrics['TWDNP_TRUE']:.3e}  "
                  f"BNP[SNIS-λ]:{metrics['BLNDNP_TRUE_NP']:.3e}  "
                  f"BNP[NN-λ]: {metrics['BLNDNP_TRUE_NN']:.3e}  |  ")
            if SHOW_LAMBDA_STATS:
                print(f"  λ agreement (net vs SNIS): mean|Δ|={metrics['LAM_ABS_DIFF']:.3e}  "
                      f"pearson={metrics['LAM_PEARSON']:.3f}\n")

            # --- end-to-end probe through EXACT blackbox path (Heun-PC, 10 steps) ---
            try:
                probe_with_run_comparison(step, nsteps=8, trials = 1)
            except Exception as e:
                print(f"[WARN] run_comparison probe failed: {e}\n")

    # === Save EMA versions ===
    def unwrap_module(m):
        if hasattr(m, "module"):   m = m.module
        if hasattr(m, "_orig_mod"): m = m._orig_mod
        return m

    torch.save(
        {
            "lam_net": unwrap_module(lam_net_ema).state_dict(),
            "critic":  unwrap_module(critic_ema).state_dict(),
            "dim":     D,
            "hidden":  HIDDEN,
            "proxy_meta": {
                "USE_PROXY_SCORES": USE_PROXY_SCORES,                    # <--- NEW
                "PROXY_N": PROXY_N,
                "PROXY_RIDGE_FRAC": PROXY_RIDGE_FRAC,
                "PROXY_ALPHA": PROXY_ALPHA,
                "PROXY_REF_CHUNK": PROXY_REF_CHUNK,
                "PRECOMPUTE_TRAIN_SCORES": PRECOMPUTE_TRAIN_SCORES,
                "PROXY_RECOMPUTE": PROXY_RECOMPUTE,
                "PROXY_GMM_BATCH": PROXY_GMM_BATCH,
                "PROXY_RECOMPUTE_MAX_ANCHORS": PROXY_RECOMPUTE_MAX_ANCHORS,
            }
        },
        SAVE_PATH,
    )
    print(f"[INFO] EMA networks written to {SAVE_PATH}")

In [ ]:
# ===========================
# Critic + Gate only (GPU/fast, stabilized)
#  - log-uniform t, σ_t-scaled critic target
#  - cosine LR, EMA, AMP, grad clipping
#  - γ-grid eval (RMSE to true), λ agreement
#  - NP targets only for metrics; no NN KSS/TWD heads
#  - *** s0 is now a learned proxy via local Gaussian kernel (PyTorch) ***
# ===========================

import torch, cupy as cp, math, gc


# -----------trrrreedee--------- Config & safe defaults --------------------
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"
USE_AMP  = (DEVICE == "cuda")
DTORCH  = torch.float32

# -------------------- VRAM hygiene --------------------
def clear_gpu_mem():
    try:
        import cupy as cp
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()
    except Exception:
        pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        for dev in range(torch.cuda.device_count()):
            torch.cuda.set_device(dev)
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()
clear_gpu_mem()

import math, os, time
import numpy as np         # <- use np everywhere
import cupy as cp          # <- needed before build targets
import torch
import torch.nn as nn
import torch.optim as optim

SAVE_PATH = "learned_lr_score_critic_gate_ema.pt"  # <-- renamed

# Fallbacks if not defined upstream
SEED        = globals().get("SEED", 0)
NUM_C       = globals().get("NUM_C", 2000)
K_DIM       = globals().get("K_DIM", 12)
VARIANT     = globals().get("VARIANT", "helix")
STD         = globals().get("STD", 0.12)
SCALE       = globals().get("SCALE", 3.0)
T_target    = globals().get("T_target", 5e-4)
T_end       = globals().get("T_end", 1.5)
N_Train     = globals().get("N_Train", globals().get("N_TRAIN", 500000))
N_REF       = globals().get("N_REF", 5000)
HIDDEN      = globals().get("HIDDEN", 512)
HIDDEM_LAM  = globals().get("HIDDEM_LAM", 128)
EVAL_N_T    = globals().get("EVAL_N_T", 24)
EVAL_PER_T  = globals().get("EVAL_PER_T", 2048)
GAMMA_PRIOR = globals().get("GAMMA_PRIOR", 1)
EMBED_MODE = globals().get("EMBED_MODE", 'sine_wiggle')

# ---- NEW: proxy-score controls ----
PROXY_N                 = int(globals().get("N_Train", 500_000))
PROXY_RIDGE_FRAC        = 1e-5 #4e-3 #float(globals().get("PROXY_RIDGE_FRAC", 1e-2))
PROXY_RIDGE_MODE       = globals().get("PROXY_RIDGE_MODE", "tail_mean")  # or tail_mean/median
PROXY_ALPHA             = float(globals().get("PROXY_ALPHA", 0.4))
PROXY_REF_CHUNK = 2048 #int(globals().get("PROXY_REF_CHUNK", 2048))
PRECOMPUTE_TRAIN_SCORES = True #bool(globals().get("PRECOMPUTE_TRAIN_SCORES", True))
PRECOMP_BATCH = 1012 #int(globals().get("PRECOMP_BATCH", 2048))

# ---- New cache/scaling knobs ----
PROXY_RANK           = 3 #int(globals().get("PROXY_RANK", 0))
PROXY_V_DTYPE        = torch.float16  # settable to bfloat16 on A100/H100
PROXY_TAU_MODE       = globals().get("PROXY_TAU_MODE", "tail_mean")  # or tail_mean/median
PROXY_TAU_TRIM_Q     = float(globals().get("PROXY_TAU_TRIM_Q", 0.1))
PROXY_TAU_SCALE      = float(globals().get("PROXY_TAU_SCALE", 1.0))
PROXY_TAU_FLOOR      = float(globals().get("PROXY_TAU_FLOOR", 1e-6))
PROXY_LAM_CLIP_MULT  = float(globals().get("PROXY_LAM_CLIP_MULT", 1e3))

# ---- LR+diag-tail + k-mix recompute toggles ----
PROXY_DIAG_TAIL   = bool(globals().get("PROXY_DIAG_TAIL", True))   # use diagonal tail τ⃗ (LR+D)
PROXY_K_MIX   = 20 #int(globals().get("PROXY_K_MIX", 64))              # mix over 64 KNN anchors / query
PROXY_COMP_K      = globals().get("PROXY_COMP_K", None)
PROXY_RECOMPUTE_MAX_ANCHORS = int(globals().get("PROXY_RECOMPUTE_MAX_ANCHORS", 10_000))

USE_LOCAL_LRSVD_FOR_S0_BATCH = False
# ---- NEW: toggle & limits for KDE/GMM recomputation mode ----
PROXY_RECOMPUTE                 = True #bool(globals().get("PROXY_RECOMPUTE", True))
PROXY_GMM_BATCH                 = int(globals().get("PROXY_GMM_BATCH", 1024))


if DEVICE == "cuda":
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

# ---------------- Base training knobs ----------------
# (keep your existing knob defaults)
STEPS = 30000 #int(globals().get("STEPS", 17000))

############## critic gate specific ###################
CRIT_STEPS_PER_LAM = 3 #int(globals().get("CRIT_STEPS_PER_LAM", 4))
WARMUP_LAM_STEPS   = 500
LAM_EPS            = float(globals().get("LAM_EPS", 1e-4))
LR_LAM_SCALE       = 0.666
WD_LAM             = float(globals().get("WD_LAM", 1e-4))
STEPS += WARMUP_LAM_STEPS
#######################################################


BATCH   = int(globals().get("BATCH", 512))
LR_INIT, LR_MIN = 5e-3, 1e-5
T_MIN, T_MAX = T_target, T_end
PRINT_EVERY = 1000 #int(globals().get("PRINT_EVERY", 200))
EMA_DECAY = float(globals().get("EMA_DECAY", 0.999))
REF_CHUNK  = BATCH
MAX_REF_CALL = 50_000
REFRESH_DATA15 = bool(globals().get("REFRESH_DATA", False))


torch.manual_seed(SEED); np.random.seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True

# Human-readable labels
NP_LAMBDA_LABEL = "NPλ(SNIS)"
NN_LAMBDA_LABEL = "NNλ(net)"
SHOW_LAMBDA_STATS = True

# ===== Helper: γ-prior time grid =====
@torch.no_grad()
def make_gamma_times(n_t: int = EVAL_N_T, gamma: float = GAMMA_PRIOR,
                     T_end: float = T_MAX, T_tgt: float = T_MIN, device: str = DEVICE):
    u = (torch.arange(n_t, device=device, dtype=torch.float32) + 0.5) / n_t
    return T_end * ((T_tgt / T_end) ** (u ** (1.0 / gamma)))

def rmse_t(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.sqrt(torch.mean((a - b) ** 2))

# ===== GMM helpers (you already have get_gmm_funcs) =====
@torch.no_grad()
def torch_sample_gmm(n: int, means: torch.Tensor, stds: torch.Tensor, weights: torch.Tensor) -> torch.Tensor:
    K, D = means.shape
    idx = torch.multinomial(weights, num_samples=n, replacement=True)
    noise = torch.randn(n, D, device=means.device) * stds[idx].unsqueeze(1)
    return means[idx] + noise

@torch.no_grad()
def torch_score_gmm_batch(x: torch.Tensor, means: torch.Tensor, stds: torch.Tensor, weights: torch.Tensor) -> torch.Tensor:
    """(Kept for 'true score' checks; no longer used as the teacher s0.)"""
    B, D = x.shape
    K = means.shape[0]
    inv_vars = 1.0 / (stds ** 2)
    log_w    = torch.log(weights + 1e-40)
    norm_const = -0.5 * D * (math.log(2 * math.pi) + torch.log(stds**2))
    diff = means.unsqueeze(0) - x.unsqueeze(1)                      # [B,K,D]
    d2   = (diff ** 2).sum(dim=-1)                                  # [B,K]
    logp = log_w + norm_const - 0.5 * d2 * inv_vars.unsqueeze(0)    # [B,K]
    w = torch.softmax(logp, dim=1)
    return (w.unsqueeze(-1) * diff * inv_vars.view(1, K, 1)).sum(dim=1)

@torch.no_grad()
def ou_params_at_t(means0: torch.Tensor, stds0: torch.Tensor, t: torch.Tensor):
    et   = torch.exp(-t)
    et2  = torch.exp(-2.0 * t)
    means_t = et.view(-1,1,1) * means0.unsqueeze(0)       # [B,K,D]
    var_t   = stds0.pow(2).unsqueeze(0) * et2 + (1.0 - et2)
    stds_t  = torch.sqrt(var_t + 1e-12)
    return means_t, stds_t

@torch.no_grad()
def true_score_xt_torch(x: torch.Tensor, t: torch.Tensor, means0: torch.Tensor, stds0: torch.Tensor, weights0: torch.Tensor) -> torch.Tensor:
    B, D = x.shape
    K, _ = means0.shape
    means_t, stds_t = ou_params_at_t(means0, stds0, t)
    inv_vars = 1.0 / (stds_t ** 2)
    log_w    = torch.log(weights0 + 1e-40).view(1, K).expand(B, K)
    norm_const = -0.5 * D * (math.log(2*math.pi) + torch.log(stds_t**2))
    diff = means_t - x.unsqueeze(1)                               # [B,K,D]
    d2   = (diff**2).sum(-1)
    logp = log_w + norm_const - 0.5 * d2 * inv_vars
    w    = torch.softmax(logp, dim=1)
    return (w.unsqueeze(-1) * diff * inv_vars.unsqueeze(-1)).sum(1)

# ======= NEW: Learned proxy for initial score s0 (PyTorch) =======

@torch.no_grad()
def _adaptive_k_default(n_ref: int, D: int, alpha: float = PROXY_ALPHA) -> int:
    N0, k0 = 2000, max(4*D, 48)
    k_default = int(min(n_ref-1, max(1, round(k0 * (n_ref / N0) ** alpha))))
    return k_default


@torch.no_grad()
def _knn_topk_across_chunks(Q: torch.Tensor,
                            R: torch.Tensor,
                            k: int,
                            ref_chunk: int = PROXY_REF_CHUNK):
    """
    Maintain top-k smallest squared distances from each query in Q to rows in R
    without materializing the full distance matrix. Returns (best_d2, best_idx).
    """
    device = Q.device
    Nq, D = Q.shape
    Nr = R.shape[0]

    # Precompute squared norms
    q2 = (Q*Q).sum(dim=1, keepdim=True)           # [Nq,1]
    r2_all = (R*R).sum(dim=1)                     # [Nr]

    best_d2  = torch.full((Nq, k), float('inf'), device=device, dtype=Q.dtype)
    best_idx = torch.full((Nq, k), -1, device=device, dtype=torch.long)

    # Process reference in chunks
    arng = torch.arange(Nq, device=device)
    for start in range(0, Nr, ref_chunk):
        end = min(start + ref_chunk, Nr)
        Rc = R[start:end]                         # [Br,D]
        r2 = r2_all[start:end]                    # [Br]
        # d2 = ||q||^2 + ||r||^2 - 2 q r^T
        d2 = q2 + r2.view(1,-1) - 2.0 * (Q @ Rc.T)   # [Nq,Br]
        d2.clamp_(min=0)

        # Merge with running top-k
        d2_cat   = torch.cat([best_d2, d2], dim=1)   # [Nq, k+Br]
        idx_new  = start + torch.arange(Rc.shape[0], device=device)
        idx_cat  = torch.cat([best_idx, idx_new.view(1,-1).expand(Nq,-1)], dim=1)
        # pick smallest k
        d2_k, ord_k = torch.topk(d2_cat, k=k, dim=1, largest=False, sorted=False)
        idx_k = torch.gather(idx_cat, dim=1, index=ord_k)
        best_d2, best_idx = d2_k, idx_k

    return best_d2, best_idx


@torch.no_grad()
def kernel_proxy_score_torch(x_query: torch.Tensor,
                             x_anchor: torch.Tensor,
                             *,
                             k: int | None = None,
                             ridge_frac: float = PROXY_RIDGE_FRAC,
                             ref_chunk: int = PROXY_REF_CHUNK,
                             recompute: bool = False,
                             gmm_batch: int = PROXY_GMM_BATCH,
                             recompute_max_anchors: int = PROXY_RECOMPUTE_MAX_ANCHORS,
                             return_params: bool = False
                             ) -> torch.Tensor | tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    assert x_query.dim() == 2 and x_anchor.dim() == 2, "Inputs must be [N, D]"
    device = x_query.device
    dtype  = x_query.dtype
    Nq, D  = x_query.shape
    Nr, Da = x_anchor.shape
    assert Da == D, f"Dim mismatch: x_anchor has D={Da}, x_query has D={D}"

    if k is None:
        k = _adaptive_k_default(Nr, D)

    # ---------- Fast path: standard local estimator ----------
    if not recompute:
        # kNN from query -> anchors
        d2_knn, idx = _knn_topk_across_chunks(x_query, x_anchor, k=k, ref_chunk=ref_chunk)  # [Nq,k], [Nq,k]
        # adaptive bandwidth from max d2 among the k-NN (per query)
        tiny = torch.finfo(dtype).tiny
        h2   = torch.maximum(d2_knn.max(dim=1).values, torch.tensor(tiny, device=device, dtype=dtype))  # [Nq]
        w    = torch.exp(-d2_knn / (2.0 * h2[:, None]))                                                 # [Nq,k]
        denom= w.sum(dim=1, keepdim=True).clamp_min(torch.finfo(dtype).eps)                             # [Nq,1]
        X_nb = x_anchor[idx]                                                                            # [Nq,k,D]

        mu   = (w.unsqueeze(-1) * X_nb).sum(dim=1) / denom                                              # [Nq,D]
        diff = X_nb - mu.unsqueeze(1)                                                                    # [Nq,k,D]
        var  = (w.unsqueeze(-1) * (diff * diff)).sum(dim=1) / denom                                      # [Nq,D]

        # ridge proportional to mean variance per point
        tau   = ridge_frac * var.mean(dim=1, keepdim=True)                                              # [Nq,1]
        var_r = (var + tau).clamp_min(torch.finfo(dtype).eps)                                           # [Nq,D]

        score = (mu - x_query) / var_r                                                                   # [Nq,D]
        return (score, mu, var_r) if return_params else score

    # ---------- Recompute path: build per-anchor components, then KDE/GMM score ----------
    if return_params:
        raise NotImplementedError("return_params=True is only supported for recompute=False (fast path).")

    # Guardrail for very large anchor pools
    if Nr > recompute_max_anchors:
        print(f"[WARN] recompute=True with Nr={Nr} > recompute_max_anchors={recompute_max_anchors}. "
              f"Falling back to fast local estimator.")
        return kernel_proxy_score_torch(
            x_query, x_anchor, k=k, ridge_frac=ridge_frac, ref_chunk=ref_chunk,
            recompute=False, gmm_batch=gmm_batch, recompute_max_anchors=recompute_max_anchors,
            return_params=False
        )

    # Step 1: per-anchor local fits (exclude self)
    k_self = min(k + 1, max(2, Nr))  # ensure at least 2 neighbors in the provisional set
    d2_all, idx_all = _knn_topk_across_chunks(x_anchor, x_anchor, k=k_self, ref_chunk=ref_chunk)  # [Nr,k_self]

    # Remove self from neighbor lists (if present), otherwise drop farthest
    ar = torch.arange(Nr, device=device).view(-1, 1).expand(-1, k_self)  # [Nr,k_self]
    is_self = (idx_all == ar)
    has_self = is_self.any(dim=1)

    keep_mask = torch.ones_like(idx_all, dtype=torch.bool)
    if has_self.any():
        self_pos = torch.argmax(is_self.int(), dim=1)  # position of 'self' per row
        keep_mask[torch.arange(Nr, device=device), self_pos] = False

    need_drop_farthest = ~has_self
    if need_drop_farthest.any():
        far_pos = torch.argmax(d2_all, dim=1)
        keep_mask[torch.arange(Nr, device=device)[need_drop_farthest], far_pos[need_drop_farthest]] = False

    idx_comp = idx_all[keep_mask].view(Nr, k_self - 1)   # [Nr, k]
    d2_comp  = d2_all[keep_mask].view(Nr, k_self - 1)    # [Nr, k]

    # Adaptive bandwidth per anchor
    tiny = torch.finfo(dtype).tiny
    h2_c = torch.maximum(d2_comp.max(dim=1).values, torch.tensor(tiny, device=device, dtype=dtype))  # [Nr]
    w_c  = torch.exp(-d2_comp / (2.0 * h2_c[:, None]))                                              # [Nr,k]
    denom_c = w_c.sum(dim=1, keepdim=True).clamp_min(torch.finfo(dtype).eps)                        # [Nr,1]
    X_nb_c = x_anchor[idx_comp]                                                                     # [Nr,k,D]

    # Component parameters (μ_j, diag Σ_j) with ridge
    mu_c  = (w_c.unsqueeze(-1) * X_nb_c).sum(dim=1) / denom_c                                       # [Nr,D]
    diffc = X_nb_c - mu_c.unsqueeze(1)                                                              # [Nr,k,D]
    var_c = (w_c.unsqueeze(-1) * (diffc * diffc)).sum(dim=1) / denom_c                              # [Nr,D]
    tau_c  = ridge_frac * var_c.mean(dim=1, keepdim=True)                                           # [Nr,1]
    varr_c = (var_c + tau_c).clamp_min(torch.finfo(dtype).eps)                                      # [Nr,D]

    # Step 2: evaluate mixture score at queries via two-pass log-sum-exp
    two_pi = torch.tensor(2.0 * math.pi, device=device, dtype=dtype)
    logdet_c = torch.sum(torch.log(two_pi * varr_c), dim=1)                                         # [Nr]

    # Pass 1: get per-query max log weight
    m = torch.full((Nq,), -float("inf"), device=device, dtype=dtype)
    for j0 in range(0, Nr, gmm_batch):
        jb = slice(j0, min(j0 + gmm_batch, Nr))
        mu_j  = mu_c[jb]          # [B,D]
        var_j = varr_c[jb]        # [B,D]
        logd  = logdet_c[jb]      # [B]
        dX = x_query[:, None, :] - mu_j[None, :, :]                                               # [Nq,B,D]
        quad = torch.sum(dX * dX / var_j[None, :, :], dim=2)                                      # [Nq,B]
        logw = -0.5 * (quad + logd[None, :])                                                      # [Nq,B]
        m = torch.maximum(m, torch.max(logw, dim=1).values)

    # Pass 2: accumulate numerator and denominator in linear space
    den = torch.zeros(Nq, device=device, dtype=dtype)
    num = torch.zeros(Nq, D, device=device, dtype=dtype)
    eps = torch.finfo(dtype).eps
    for j0 in range(0, Nr, gmm_batch):
        jb = slice(j0, min(j0 + gmm_batch, Nr))
        mu_j  = mu_c[jb]          # [B,D]
        var_j = varr_c[jb]        # [B,D]
        logd  = logdet_c[jb]      # [B]
        dX = x_query[:, None, :] - mu_j[None, :, :]                                               # [Nq,B,D]
        quad = torch.sum(dX * dX / var_j[None, :, :], dim=2)                                      # [Nq,B]
        logw = -0.5 * (quad + logd[None, :])                                                      # [Nq,B]
        w_nb = torch.exp(logw - m[:, None])                                                       # [Nq,B]
        comp_score = -dX / var_j[None, :, :]                                                      # [Nq,B,D]
        num += torch.einsum('nb,nbd->nd', w_nb, comp_score)
        den += torch.sum(w_nb, dim=1)

    return num / (den[:, None] + eps)


@torch.no_grad()
def kernel_proxy_score_local_lrsvd(
    x_query: torch.Tensor,
    x_anchor: torch.Tensor,
    *,
    k: int | None = None,
    rank: int | None = None,
    ref_chunk: int = 2048,
    # ridge / tail controls
    ridge_mode: str = "tail_mean",
    tail_trim_q: float = 0.1,
    ridge_scale: float = 1.0,
    ridge_floor: float = 1e-6,
    lam_clip_mult: float = 1e3,
    # k / bandwidth defaults
    alpha: float = 0.4,
    n0_for_k: int = 2000,
    use_cpu_svd: bool = True,
    enforce_dtype: torch.dtype | None = None,
    # NEW:
    diag_tail: bool = True,           # LR + diagonal tail (LR+D)
    recompute: bool = False,          # enable k-mix recompute
    k_mix: int | None = None,         # components per query in recompute (defaults to k)
    comp_k: int | None = None,        # neighbors per anchor when building components (defaults to k)
    ridge_frac: float = PROXY_RIDGE_FRAC
) -> torch.Tensor:
    """
    Local LR-SVD proxy with diagonal tail option (diag_tail=True), and an optional
    cheap k-mix recompute step (recompute=True) that mixes only over the query's
    k_mix nearest anchors.  All heavy work is batched and stays on GPU.
    """
    assert x_query.dim() == 2 and x_anchor.dim() == 2
    if enforce_dtype is not None:
        x_query  = x_query.to(enforce_dtype)
        x_anchor = x_anchor.to(enforce_dtype)
    device = x_query.device
    dtype  = x_query.dtype
    eps    = torch.finfo(dtype).eps
    tiny   = torch.finfo(dtype).tiny

    Nq, D = x_query.shape
    Na, _ = x_anchor.shape
    if rank is None:
        rank = min(8, D)

    # --- default k (as in your Script A) ---
    if k is None:
        k0 = max(4 * D, 48)
        k_default = int(min(Na - 1, max(4 * D, round(k0 * ((Na / float(n0_for_k)) ** alpha)))))
        k = max(1, k_default)
    else:
        k = int(max(1, min(k, Na - 1)))

    if k_mix is None:
        k_mix = k
    if comp_k is None:
        comp_k = k

    # --- helper: KNN (in chunks) ---
    def _knn(Q, R, k_val):
        return _knn_topk_across_chunks(Q, R, k=k_val, ref_chunk=ref_chunk)

    # ======================================================================
    # (A) Fast path (no recompute): local LR+(diag tail) precision with Woodbury
    # ======================================================================
    if not recompute:
        d2_knn, idx = _knn(x_query, x_anchor, k)                      # [Nq,k], [Nq,k]
        h2  = torch.maximum(d2_knn.max(dim=1).values, torch.tensor(tiny, device=device, dtype=dtype))
        w   = torch.exp(-d2_knn / (2.0 * h2[:, None]))                # [Nq,k]
        denom = w.sum(dim=1, keepdim=True).clamp_min(eps)
        X_nb  = x_anchor[idx]                                         # [Nq,k,D]
        mu    = (w[..., None] * X_nb).sum(dim=1) / denom              # [Nq,D]
        diff  = X_nb - mu[:, None, :]                                  # [Nq,k,D]

        out = torch.empty_like(x_query)

        # τ̂ from SVD tail; LR+D optional
        def _tau_scalar_from_tail(S, r_eff):
            m_total = S.shape[0]
            if m_total <= r_eff: return float(ridge_floor)
            lam_tail = S[r_eff:]**2
            if ridge_mode == "tail_trimmed":
                m = lam_tail.numel()
                if m > 2:
                    kdrop = int(max(1, min(m-1, round(tail_trim_q * m))))
                    lam_tail = lam_tail.sort().values[:m-kdrop]
            elif ridge_mode == "tail_median":
                return max(ridge_floor, ridge_scale * float(lam_tail.median().item()))
            return max(ridge_floor, ridge_scale * float(lam_tail.mean().item()))

        for i in range(Nq):
            wi = (w[i] / denom[i, 0]).to(dtype)                       # [k]
            R  = (wi.sqrt().unsqueeze(1) * diff[i]).contiguous()      # [k,D]

            R_svd = R.cpu() if use_cpu_svd and R.is_cuda else R
            try:
                U_, S, Vh = torch.linalg.svd(R_svd, full_matrices=False)
                if use_cpu_svd and R.is_cuda:
                    S  = S.to(device=device, dtype=dtype)
                    Vh = Vh.to(device=device, dtype=dtype)
                r_eff = int(min(rank, Vh.shape[0]))
            except RuntimeError:
                S, Vh, r_eff = None, None, 0

            if (S is None) or (r_eff == 0):
                # pure diagonal fallback
                # diagonal variance estimate:
                var = (wi[:, None] * (diff[i]**2)).sum(0)             # [D]
                tau_s = ridge_frac * var.mean()
                var_r = (var + tau_s).clamp_min(eps)
                out[i] = (mu[i] - x_query[i]) / var_r
                continue

            V_r = Vh[:r_eff, :].T                                     # [D,r_eff]
            lam = (S[:r_eff]**2)
            # --- LR+D tail ---
            if diag_tail:
                # per-dim tail energy: τ_d ≈ sum_{j>r} λ_j (u_j ⊙ u_j)
                if Vh.shape[0] > r_eff:
                    V_tail = Vh[r_eff:, :].T                          # [D, D-r_eff]
                    lam_tail = (S[r_eff:]**2)                         # [D-r_eff]
                    if ridge_mode == "tail_trimmed":
                        m = lam_tail.numel()
                        if m > 2:
                            kdrop = int(max(1, min(m-1, round(tail_trim_q * m))))
                            lam_tail = lam_tail.sort().values[:m-kdrop]
                            V_tail   = V_tail[:, :lam_tail.shape[0]]
                    tau_vec = ridge_scale * torch.maximum(
                        torch.full((D,), ridge_floor, device=device, dtype=dtype),
                        (V_tail*V_tail @ lam_tail)
                    )                                                  # [D]
                else:
                    tau_vec = torch.full((D,), ridge_floor, device=device, dtype=dtype)

                lam = torch.minimum(lam, lam_clip_mult * tau_vec.max())
                invD = 1.0 / (tau_vec + eps)                           # [D]
                dX   = (x_query[i] - mu[i])                            # [D]
                invDv = invD * dX
                if r_eff > 0:
                    A = torch.diag(1.0 / (lam + eps)) + V_r.T @ (invD[:, None] * V_r)
                    z = torch.linalg.solve(A, V_r.T @ invDv)
                    u = invDv - (invD[:, None] * V_r) @ z              # Σ^{-1}(x-μ)
                else:
                    u = invDv
                out[i] = -u
            else:
                # scalar tau tail (old path)
                tau_s = _tau_scalar_from_tail(S, r_eff)
                lam   = torch.minimum(lam, lam_clip_mult * tau_s)
                dX    = (x_query[i] - mu[i]).unsqueeze(0)              # [1,D]
                T     = dX @ V_r                                       # [1,r]
                scale = lam / (tau_s * (tau_s + lam) + eps)            # [r]
                term  = (T * scale) @ V_r.T
                u     = dX / (tau_s + eps) - term
                out[i]= (-u).squeeze(0)

        return out

    # ======================================================================
    # (B) k-mix recompute: mix only over each query's k_mix nearest anchors
    #      (components are diagonal locals for cheapness)
    # ======================================================================
    # Step B.1: find per-query KNN anchors
    d2_qk, idx_qk = _knn(x_query, x_anchor, k_mix)                     # [Nq,k_mix], [Nq,k_mix]
    uniq_idx = torch.unique(idx_qk.reshape(-1))
    Nu = uniq_idx.numel()

    # Step B.2: build local diagonal components (μ_j, diag Σ_j) for the subset
    #           using each anchor's own comp_k nearest neighbors (exclude self)
    d2_all, idx_all = _knn(x_anchor[uniq_idx], x_anchor, comp_k + 1)   # [Nu,comp_k+1]
    # mask out self and re-select top comp_k
    self_mask = idx_all.eq(uniq_idx.view(-1,1))
    d2_all = d2_all.masked_fill(self_mask, float('inf'))
    d2_comp, ord2 = torch.topk(d2_all, k=comp_k, dim=1, largest=False)
    idx_comp = torch.gather(idx_all, 1, ord2)                          # [Nu,comp_k]

    tiny_t = torch.tensor(tiny, device=device, dtype=dtype)
    h2_c   = torch.maximum(d2_comp.max(dim=1).values, tiny_t)          # [Nu]
    w_c    = torch.exp(-d2_comp / (2.0 * h2_c[:, None]))               # [Nu,comp_k]
    denom_c= w_c.sum(dim=1, keepdim=True).clamp_min(eps)               # [Nu,1]
    X_nb_c = x_anchor[idx_comp]                                        # [Nu,comp_k,D]
    mu_c   = (w_c[..., None] * X_nb_c).sum(1) / denom_c                # [Nu,D]
    diff_c = X_nb_c - mu_c[:, None, :]                                  # [Nu,comp_k,D]
    var_c  = (w_c[..., None] * (diff_c*diff_c)).sum(1) / denom_c       # [Nu,D]
    tau_c  = ridge_frac * var_c.mean(dim=1, keepdim=True)              # [Nu,1]
    varr_c = (var_c + tau_c).clamp_min(eps)                            # [Nu,D]
    logdet_c = torch.sum(torch.log(2.0 * math.pi * varr_c), dim=1)     # [Nu]

    # map global anchor id -> row in the subset arrays
    pos_map = torch.full((Na,), -1, device=device, dtype=torch.long)
    pos_map[uniq_idx] = torch.arange(Nu, device=device, dtype=torch.long)
    pos = pos_map[idx_qk]                                              # [Nq,k_mix]

    # gather per-query component params
    mu_sel  = mu_c[pos]                                                # [Nq,k_mix,D]
    var_sel = varr_c[pos]                                              # [Nq,k_mix,D]
    logdsel = logdet_c[pos]                                            # [Nq,k_mix]

    dX   = x_query[:, None, :] - mu_sel                                # [Nq,k_mix,D]
    quad = torch.sum(dX * (dX / var_sel), dim=2)                       # [Nq,k_mix]
    logw = -0.5 * (quad + logdsel)                                     # [Nq,k_mix]

    m = torch.max(logw, dim=1, keepdim=True).values                    # [Nq,1]
    W = torch.exp(logw - m)                                            # [Nq,k_mix]
    comp_score = -(dX / var_sel)                                       # [Nq,k_mix,D]
    num = torch.einsum('nk,nkd->nd', W, comp_score)                    # [Nq,D]
    den = W.sum(dim=1).clamp_min(eps)                                  # [Nq]
    return num / den[:, None]





@torch.no_grad()
def precompute_proxy_scores(dataset_cpu: torch.Tensor,
                            anchor_pool: torch.Tensor,
                            *,
                            batch: int = PRECOMP_BATCH,
                            k: int | None = None,
                            ref_chunk: int = PROXY_REF_CHUNK,
                            recompute: bool = PROXY_RECOMPUTE,
                            gmm_batch: int = PROXY_GMM_BATCH,   # unused now (k-mix)
                            recompute_max_anchors: int = PROXY_RECOMPUTE_MAX_ANCHORS
                            ) -> torch.Tensor:
    """
    Precompute proxy scores for the entire dataset and return them on CPU.
    If recompute=True, uses the k-mix path (mix over query's KNN anchors only).
    """
    N, D = dataset_cpu.shape
    out = torch.empty(N, D, dtype=torch.float32)

    for start in range(0, N, batch):
        end = min(start + batch, N)
        q = dataset_cpu[start:end].to(DEVICE, non_blocking=True)

        s = kernel_proxy_score_local_lrsvd(
            q, anchor_pool,
            k=k,
            rank=min(PROXY_RANK, q.shape[1]),
            ref_chunk=ref_chunk,
            ridge_mode=PROXY_RIDGE_MODE,
            tail_trim_q=PROXY_TAU_TRIM_Q,
            ridge_scale=PROXY_TAU_SCALE,
            ridge_floor=PROXY_TAU_FLOOR,
            lam_clip_mult=PROXY_LAM_CLIP_MULT,
            alpha=PROXY_ALPHA,
            use_cpu_svd=True,
            enforce_dtype=torch.float32,
            diag_tail=PROXY_DIAG_TAIL,
            recompute=recompute,
            k_mix=PROXY_K_MIX,
            comp_k=(PROXY_COMP_K if PROXY_COMP_K is not None else None),
            ridge_frac=PROXY_RIDGE_FRAC
        )
        out[start:end] = s.detach().to('cpu', non_blocking=True)

    return out




# ----------------- Nets ----------------------
class LambdaNet(nn.Module):
    """Small MLP → scalar λ∈(0,1)."""
    def __init__(self, dim: int, hidden: int = 128):
        super().__init__()
        self.f = nn.Sequential(
            nn.Linear(dim + 1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden),  nn.SiLU(),
            nn.Linear(hidden, 1),
            nn.Sigmoid()
        )
    def forward(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        return self.f(torch.cat([x, torch.log(t)], dim=1)).squeeze(-1)

class CriticNet(nn.Module):
    """Outputs σ_t-scaled score estimate (we unscale at eval)."""
    def __init__(self, dim: int, hidden: int = 512):
        super().__init__()
        self.f = nn.Sequential(
            nn.Linear(dim + 1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden),  nn.SiLU(),
            nn.Linear(hidden, dim),
        )
    def forward(self, y: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        return self.f(torch.cat([y, torch.log(t)], dim=1))

def update_ema(target: nn.Module, src: nn.Module, decay: float):
    with torch.no_grad():
        for p_t, p_s in zip(target.parameters(), src.parameters()):
            p_t.data.mul_(decay).add_(p_s.data, alpha=(1.0 - decay))

# ----------------- Build targets -----------------
params, sampler_func, score_func = get_gmm_funcs(NUM_C, k_dim=K_DIM, variant=VARIANT, comp_std=STD,
                                                 overall_scale=SCALE, seed=0, embedding_mode = EMBED_MODE)[:3]
means0_cp, stds0_cp, w0_cp = map(cp.asarray, params)
means0_np, stds0_np, w0_np = params
means0 = torch.as_tensor(means0_np, device=DEVICE).float().contiguous()
stds0  = torch.as_tensor(stds0_np,  device=DEVICE).float().contiguous()
w0     = torch.as_tensor(w0_np,     device=DEVICE).float().contiguous(); w0 /= w0.sum()


def true_score_func_cp(y_cp, t_scalar):
    return calculate_true_score_at_t(
        y_cp, t_scalar, means0_cp, stds0_cp, w0_cp, batch_size=4096
    )


# --- Build a large anchor pool (prior samples) for proxy s0 ---
with torch.no_grad():
    proxy_pool = torch_sample_gmm(PROXY_N, means0, stds0, w0)  # [PROXY_N, D]


with torch.no_grad():
    x_ref = torch_sample_gmm(N_REF, means0, stds0, w0)
    s0_ref = kernel_proxy_score_local_lrsvd(
        x_ref, proxy_pool,
        rank=min(PROXY_RANK, x_ref.shape[1]),
        ridge_mode=PROXY_RIDGE_MODE,
        tail_trim_q=PROXY_TAU_TRIM_Q,
        ridge_scale=PROXY_TAU_SCALE,
        ridge_floor=PROXY_TAU_FLOOR,
        lam_clip_mult=PROXY_LAM_CLIP_MULT,
        alpha=PROXY_ALPHA,
        use_cpu_svd=True,
        enforce_dtype=torch.float32,
        diag_tail=PROXY_DIAG_TAIL,
        recompute=PROXY_RECOMPUTE,           # <-- use k-mix recompute if enabled
        k_mix=PROXY_K_MIX,
        comp_k=(PROXY_COMP_K if PROXY_COMP_K is not None else None),
        ridge_frac=PROXY_RIDGE_FRAC
    )

D = means0.shape[1]

# ------- Gate (λ-net) -------
lam_net     = LambdaNet(D, hidden=HIDDEM_LAM).to(DEVICE)
lam_net_ema = LambdaNet(D, hidden=HIDDEM_LAM).to(DEVICE)
lam_net_ema.load_state_dict(lam_net.state_dict())
print(f"[INFO] Gate init: random (D={D}).")

# ------- Critic -------
critic      = CriticNet(D, hidden=HIDDEN).to(DEVICE)
critic_ema  = CriticNet(D, hidden=HIDDEN).to(DEVICE)
critic_ema.load_state_dict(critic.state_dict())

# Unscaled EMA score from the critic.
# During training the critic target was:
#   q_scaled = sigma_t * ((1-λ) * a + λ * b)  ≈  sigma_t * s_theta(x_t, t)
# so at inference we simply undo the scale:
#   s_theta(x_t, t) ≈ q_scaled / sigma_t


# optional compile
try:
    if DEVICE == "cuda":
        lam_net     = torch.compile(lam_net)
        critic      = torch.compile(critic)
        lam_net_ema = torch.compile(lam_net_ema)
        critic_ema  = torch.compile(critic_ema)
except Exception:
    pass

opt_lam    = optim.Adam(lam_net.parameters(), lr=LR_INIT*LR_LAM_SCALE, weight_decay=WD_LAM)
opt_crit   = optim.Adam(critic.parameters(),  lr=LR_INIT)
sched_lam  = torch.optim.lr_scheduler.CosineAnnealingLR(opt_lam,  T_max=STEPS, eta_min=LR_MIN*LR_LAM_SCALE)
sched_crit = torch.optim.lr_scheduler.CosineAnnealingLR(opt_crit, T_max=STEPS, eta_min=LR_MIN)
scaler_crit = torch.amp.GradScaler('cuda', enabled=USE_AMP)
scaler_lam  = torch.amp.GradScaler('cuda', enabled=USE_AMP)
mse        = nn.MSELoss()

def update_all_ema():
    update_ema(lam_net_ema, lam_net, EMA_DECAY)
    update_ema(critic_ema, critic, EMA_DECAY)

# ----------------- Fixed training dataset (CPU pinned, deterministic) -----------------
TRAIN_SEED  = int(globals().get("TRAIN_SEED", globals().get("SEED", 0)))
_gen_cpu    = torch.Generator(device='cpu').manual_seed(TRAIN_SEED)

def _sample_gmm_cpu_with_gen(n, means_cpu, stds_cpu, w_cpu, gen):
    K, D = means_cpu.shape
    idx   = torch.multinomial(w_cpu, num_samples=n, replacement=True, generator=gen)
    noise = torch.randn(n, D, generator=gen, dtype=means_cpu.dtype)
    return means_cpu[idx] + noise * stds_cpu[idx].unsqueeze(1)

with torch.no_grad():
    _means_cpu = means0.detach().cpu().float()
    _stds_cpu  = stds0.detach().cpu().float()
    _w_cpu     = (w0 / w0.sum()).detach().cpu().float()

    x0_train_cpu  = _sample_gmm_cpu_with_gen(N_Train, _means_cpu, _stds_cpu, _w_cpu, _gen_cpu).pin_memory()
    u_train_cpu   = torch.rand(N_Train, 1, generator=_gen_cpu, dtype=torch.float32).pin_memory()
    eps_train_cpu = torch.randn(N_Train, _means_cpu.shape[1], generator=_gen_cpu, dtype=torch.float32).pin_memory()

# --- NEW: Optional precompute s0 proxies for training pool (aligned with indices) ---
S0_TRAIN_CPU = None
if PRECOMPUTE_TRAIN_SCORES:
    print(f"[INFO] Precomputing learned s0 proxies for training pool of size {N_Train}..."
          + (" (recompute/KDE mode)" if PROXY_RECOMPUTE else ""))
    S0_TRAIN_CPU = precompute_proxy_scores(x0_train_cpu, proxy_pool, batch=PRECOMP_BATCH).pin_memory()
    print("[INFO] Done precomputing s0 proxies.")

_train_ptr  = 0
_train_perm = torch.randperm(N_Train, generator=_gen_cpu)

def _next_batch_indices(B):
    global _train_ptr, _train_perm
    if _train_ptr + B <= N_Train:
        idx = _train_perm[_train_ptr:_train_ptr+B]
        _train_ptr += B
        return idx
    tail = _train_perm[_train_ptr:]
    _train_perm = torch.randperm(N_Train, generator=_gen_cpu)
    _train_ptr  = 0
    need = B - tail.numel()
    head = _train_perm[_train_ptr:_train_ptr+need]
    _train_ptr += need
    return torch.cat([tail, head], dim=0)

# ----------------- NP teachers using proxy s0_ref -----------------
@torch.no_grad()
def kss_score_torch(y: torch.Tensor, t: torch.Tensor, x_ref: torch.Tensor, s0_ref: torch.Tensor) -> torch.Tensor:
    B, D = y.shape
    N    = x_ref.shape[0]
    et   = torch.exp(-t); et2 = torch.exp(-2.0 * t)
    var  = (1.0 - et2).clamp_min(1e-12); invv = 1.0 / var
    mu   = et.view(B,1,1) * x_ref.view(1,N,D)
    diff = y.view(B,1,D) - mu
    d2   = (diff**2).sum(-1)
    logw = -0.5 * invv.view(B,1) * d2
    w    = torch.softmax(logw, dim=1)
    return torch.exp(t).view(B,1) * (w @ s0_ref)

@torch.no_grad()
def tweedie_score_torch(y: torch.Tensor, t: torch.Tensor, x_ref: torch.Tensor) -> torch.Tensor:
    B, D = y.shape
    N    = x_ref.shape[0]
    et   = torch.exp(-t); et2 = torch.exp(-2.0 * t)
    var  = (1.0 - et2).clamp_min(1e-12); invv = 1.0 / var
    mu   = et.view(B,1,1) * x_ref.view(1,N,D)
    diff = y.view(B,1,D) - mu
    d2   = (diff**2).sum(-1)
    logw = -0.5 * invv.view(B,1) * d2
    w    = torch.softmax(logw, dim=1)
    mu_bar = (w.unsqueeze(-1) * mu).sum(dim=1)
    return -invv.view(B,1) * (y - mu_bar)

@torch.no_grad()
def snis_lambda_torch(
    y, t, x_ref, s0_ref,
    s_kss=None, s_twd=None, eps: float = 1e-12,
    max_ref: int | None = 10_000, chunk: int = 4096
):
    """Streaming SNIS λ*: (Var[â]-Cov[â,b̂])/(Var[â]+Var[b̂]-2Cov[â,b̂])."""
    B, D = y.shape
    N = x_ref.shape[0]
    if (max_ref is not None) and (N > max_ref):
        idx = torch.randperm(N, device=x_ref.device)[:max_ref]
        x_ref  = x_ref[idx]
        s0_ref = s0_ref[idx]
        N = x_ref.shape[0]

    et   = torch.exp(-t)                     # [B,1]
    et2  = torch.exp(-2.0 * t)
    var  = (1.0 - et2).clamp_min(1e-12)      # [B,1]
    invv = 1.0 / var

    # Pass 1
    m = torch.full((B,), -float("inf"), device=y.device, dtype=y.dtype)
    s = torch.zeros(B, device=y.device, dtype=y.dtype)
    acc_a = torch.zeros(B, D, device=y.device, dtype=y.dtype)
    acc_b = torch.zeros(B, D, device=y.device, dtype=y.dtype)

    for start in range(0, N, chunk):
        end = min(start + chunk, N)
        xr = x_ref[start:end]
        s0 = s0_ref[start:end]
        mu   = et.view(B,1,1) * xr.view(1,-1,D)
        diff = y.view(B,1,D) - mu
        d2   = (diff**2).sum(-1)
        L    = -0.5 * invv.view(B,1) * d2
        m_new = torch.maximum(m, L.max(dim=1).values)
        scale = torch.exp(m - m_new)
        wtil  = torch.exp(L - m_new[:,None])

        a_i   = torch.exp(t).view(B,1,1) * s0.view(1,-1,D)
        b_i   = -(invv.view(B,1,1)) * (y.view(B,1,D) - mu)
        s     = s * scale + wtil.sum(1)
        acc_a = acc_a * scale[:,None] + torch.einsum('bn,bnd->bd', wtil, a_i)
        acc_b = acc_b * scale[:,None] + torch.einsum('bn,bnd->bd', wtil, b_i)
        m = m_new

    mu_a = acc_a / s[:,None]
    mu_b = acc_b / s[:,None]

    # Pass 2
    S0    = torch.zeros(B, device=y.device, dtype=y.dtype)
    Vknum = torch.zeros(B, device=y.device, dtype=y.dtype)
    Vtnum = torch.zeros(B, device=y.device, dtype=y.dtype)
    Cnum  = torch.zeros(B, device=y.device, dtype=y.dtype)
    logZ  = torch.log(s) + m

    for start in range(0, N, chunk):
        end = min(start + chunk, N)
        xr = x_ref[start:end]
        s0 = s0_ref[start:end]
        mu   = et.view(B,1,1) * xr.view(1,-1,D)
        diff = y.view(B,1,D) - mu
        d2   = (diff**2).sum(-1)
        L    = -0.5 * invv.view(B,1) * d2
        w    = torch.exp(L - logZ[:,None])
        w2   = w * w
        S0  += w2.sum(1)

        a_i  = torch.exp(t).view(B,1,1) * s0.view(1,-1,D)
        b_i  = -(invv.view(B,1,1)) * (y.view(B,1,D) - mu)
        ac   = a_i - mu_a[:,None,:]
        bc   = b_i - mu_b[:,None,:]

        Vknum += (w2[:,:,None] * (ac*ac)).sum((1,2))
        Vtnum += (w2[:,:,None] * (bc*bc)).sum((1,2))
        Cnum  += (w2[:,:,None] * (ac*bc)).sum((1,2))

    den = (1.0 - S0).clamp_min(1e-10)
    Vk  = Vknum / den
    Vt  = Vtnum / den
    C   =  Cnum / den
    lam = ((Vk - C) / (Vk + Vt - 2.0*C + eps)).clamp(0.0, 1.0)
    return lam

# ----------------- Batch maker (log-uniform t) -----------------
_train_indices_last = None  # for exposing the last indices to caller
@torch.no_grad()
def make_batch(B, *, ref_chunk=None, max_ref=None, t_mode='log', return_idx=True):
    global _train_indices_last
    if ref_chunk is None:
        ref_chunk = min(4096, int(globals().get("N_REF", 4096)))
    if max_ref is None:
        max_ref = int(globals().get("N_REF", 10_000))

    # fixed pool slicing
    idx = _next_batch_indices(B)
    _train_indices_last = idx  # store
    x0  = x0_train_cpu[idx].to(DEVICE, non_blocking=True)
    u   = u_train_cpu[idx].to(DEVICE, non_blocking=True)
    eps = eps_train_cpu[idx].to(DEVICE, non_blocking=True)

    if t_mode == 'log':
        logTmin = math.log(T_MIN); logTmax = math.log(T_MAX)
        t = torch.exp(torch.tensor(logTmin, device=DEVICE) + u * (logTmax - logTmin))
    elif t_mode == 'uniform':
        t = torch.tensor(T_MIN, device=DEVICE) + u * (T_MAX - T_MIN)
    else:
        raise ValueError(f"Unknown t_mode: {t_mode}")

    et    = torch.exp(-t)
    sigma = torch.sqrt((1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12))
    x_t   = et * x0 + sigma * eps

    # streamed NP teachers (for metrics and SNIS λ)
    x_ref_src, s0_ref_src = x_ref, s0_ref
    N_total = x_ref_src.shape[0]
    if (max_ref is not None) and (N_total > max_ref):
        sel = torch.randperm(N_total, generator=_gen_cpu, device='cpu')[:max_ref]
        x_ref_src  = x_ref_src[sel]
        s0_ref_src = s0_ref_src[sel]
        N_total    = max_ref

    Bsz, D = x_t.shape
    y      = x_t
    invv   = 1.0 / (1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12)
    y2     = (y*y).sum(-1)

    m      = torch.full((Bsz,), -float("inf"), device=DEVICE, dtype=y.dtype)
    s      = torch.zeros(Bsz, device=DEVICE, dtype=y.dtype)
    num_x  = torch.zeros(Bsz, D, device=DEVICE, dtype=y.dtype)
    num_s0 = torch.zeros(Bsz, D, device=DEVICE, dtype=y.dtype)

    for start in range(0, N_total, ref_chunk):
        end = min(start + ref_chunk, N_total)
        xr  = x_ref_src[start:end].to(DEVICE, non_blocking=True).contiguous()
        s0  = s0_ref_src[start:end].to(DEVICE, non_blocking=True).contiguous()

        xr2   = (xr*xr).sum(-1)
        cross = y @ xr.T
        d2    = y2[:,None] + (et*et) * xr2[None,:] - 2.0 * et * cross
        L     = -0.5 * invv * d2

        m_new = torch.maximum(m, L.max(dim=1).values)
        scale = torch.exp(m - m_new)
        wtil  = torch.exp(L - m_new[:,None])

        s      = s * scale + wtil.sum(1)
        num_x  = num_x  * scale[:,None] + torch.einsum('bn,nd->bd', wtil, xr)
        num_s0 = num_s0 * scale[:,None] + torch.einsum('bn,nd->bd', wtil, s0)
        m      = m_new

    x_bar  = num_x  / (s[:,None] + 1e-30)
    s0_bar = num_s0 / (s[:,None] + 1e-30)

    s_kss  = torch.exp(t) * s0_bar
    s_twd  = -invv * (y - torch.exp(-t) * x_bar)
    s_true = true_score_xt_torch(x_t, t, means0, stds0, w0)

    if return_idx:
        return x_t, t, sigma, s_kss, s_twd, s_true, x0, idx
    else:
        return x_t, t, sigma, s_kss, s_twd, s_true, x0

@torch.no_grad()
def sample_evolved_gmm(n: int, t_scalar: float,
                       means0: torch.Tensor, stds0: torch.Tensor, w0: torch.Tensor) -> torch.Tensor:
    # Draw x_t directly from the OU-evolved mixture (means_t, stds_t).
    t  = torch.tensor(t_scalar, device=means0.device, dtype=means0.dtype)
    et = torch.exp(-t); et2 = torch.exp(-2.0 * t)
    means_t = means0 * et                               # [K,D]
    var_t   = stds0**2 * et2 + (1.0 - et2)             # [K,D]
    stds_t  = torch.sqrt(var_t.clamp_min(1e-12))       # [K,D]
    return torch_sample_gmm(n, means_t, stds_t, w0)

# ----------------- Eval (γ-grid) -----------------
@torch.no_grad()
def eval_gamma_grid_rmse(n_t: int = EVAL_N_T, per_t: int = EVAL_PER_T):
    t_grid = make_gamma_times(n_t=n_t)
    sums = {k: 0.0 for k in [
        "KSSNP_TRUE","TWDNP_TRUE","BLNDNP_TRUE_NP","BLNDNP_TRUE_NN",
        "LAM_ABS_DIFF","LAM_PEARSON","Critic_RMSE"
    ]}
    cnt_total = 0
    sum_np = 0.0; sum_nn = 0.0
    sum_np2 = 0.0; sum_nn2 = 0.0; sum_prod = 0.0

    for t in t_grid.tolist():
        x_eval = sample_evolved_gmm(per_t, t, means0, stds0, w0)
        tcol   = torch.full((per_t,1), t, device=DEVICE, dtype=x_eval.dtype)
        sigma  = torch.sqrt((1.0 - torch.exp(-2.0*tcol)).clamp_min(1e-12))

        s_true = true_score_xt_torch(x_eval, tcol, means0, stds0, w0)
        s_kss  = kss_score_torch(x_eval, tcol, x_ref, s0_ref)       # uses proxy s0_ref
        s_twd  = tweedie_score_torch(x_eval, tcol, x_ref)

        pred_crit = critic_ema(x_eval, tcol) / sigma

        lam_np = snis_lambda_torch(x_eval, tcol, x_ref, s0_ref, s_kss, s_twd).view(-1,1)
        lam_nn = lam_net_ema(x_eval, tcol).view(-1,1)

        s_bl_np_np = (1 - lam_np)*s_kss + lam_np*s_twd
        s_bl_np_nn = (1 - lam_nn)*s_kss + lam_nn*s_twd

        sums["KSSNP_TRUE"]     += rmse_t(s_kss,   s_true).item()
        sums["TWDNP_TRUE"]     += rmse_t(s_twd,   s_true).item()
        sums["BLNDNP_TRUE_NP"] += rmse_t(s_bl_np_np, s_true).item()
        sums["BLNDNP_TRUE_NN"] += rmse_t(s_bl_np_nn, s_true).item()
        sums["Critic_RMSE"]    += rmse_t(pred_crit, s_true).item()

        d = (lam_nn - lam_np).abs().mean().item()
        sums["LAM_ABS_DIFF"]   += d

        ln = lam_nn.view(-1); lp = lam_np.view(-1)
        cnt_total += ln.numel()
        sum_nn  += ln.sum().item(); sum_np  += lp.sum().item()
        sum_nn2 += (ln*ln).sum().item(); sum_np2 += (lp*lp).sum().item()
        sum_prod+= (ln*lp).sum().item()

    for k in sums:
        if k != "LAM_PEARSON":
            sums[k] /= len(t_grid)

    mean_nn = sum_nn / cnt_total; mean_np = sum_np / cnt_total
    var_nn  = max(0.0, sum_nn2 / cnt_total - mean_nn**2)
    var_np  = max(0.0, sum_np2 / cnt_total - mean_np**2)
    cov     = (sum_prod / cnt_total) - mean_nn*mean_np
    den = max(1e-12, math.sqrt(max(var_nn, 0.0) * max(var_np, 0.0)))
    sums["LAM_PEARSON"] = (cov / den) if den > 0 else 0.0
    return sums


# ===== add once, above your training loop =====
import numpy as np, cupy as cp, torch
@torch.no_grad()
def ema_score_xt_torch(x_t: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    if t.dim() == 1: t = t[:, None]
    sigma_t  = torch.sqrt((1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12))
    q_scaled = critic_ema(x_t, t)
    s = q_scaled / sigma_t
    return torch.nan_to_num(s, nan=0.0, posinf=0.0, neginf=0.0)

def make_custom_score_from_torch(score_xt_torch, device):
    def score_cp(y_cp, t_scalar):
        y_np = cp.asnumpy(y_cp)
        x_t  = torch.as_tensor(y_np, device=device, dtype=torch.float32)
        t    = torch.full((x_t.shape[0], 1), float(t_scalar), device=device, dtype=x_t.dtype)
        s    = score_xt_torch(x_t, t).detach().cpu().numpy().astype(np.float64)
        return cp.asarray(s)
    return score_cp

# --- your EXACT blackbox sampler ---
def blackbox_sampler(
    N_part, time_pts, custom_score, sampler_func, *,
    mode="heun_hop", h_coeff=0.5, true_score_func=None,
    p_prune = 0, likelyhood_func = None, loglik_grad_fn = None):
    xp  = cp
    dim = int(sampler_func(5).shape[1])           # match model/data dim
    y   = xp.random.randn(N_part, dim).astype(xp.float64)

    rmse_list = []                                # collect RMSE vs oracle

    for k in range(len(time_pts)-1):
        t_cur, t_prev = time_pts[k], time_pts[k+1]
        dt    = t_cur - t_prev
        noise = xp.random.randn(N_part, dim).astype(xp.float64)

        S_cur  = custom_score(y, t_cur)           # [N_part, dim]
        y_hat  = y + dt*(y + 2*S_cur) + xp.sqrt(2.0*dt)*noise
        S_prev = custom_score(y_hat, t_prev)

        if mode == "heun_hop":
            A = xp.exp(dt); B = xp.exp(2*dt) - 1.0; C = xp.sqrt(B); D = B / A
            y = A*y + h_coeff*B*S_prev + (1-h_coeff)*D*S_cur + C*noise
        elif mode == "ou_sde":
            y = y_hat
        elif mode == "pf_ode":
            y = y + dt*(y + S_cur)
        elif mode == "heun_pc":
            drift_avg = 0.5*(y + 2*S_cur) + 0.5*(y_hat + 2*S_prev)
            y = y + dt*drift_avg + xp.sqrt(2.0 * dt) * noise
        else:
            raise ValueError(mode)

    if true_score_func is not None:
        S_cur_true  = true_score_func(y, t_cur)
        S_prev_true = true_score_func(y_hat, t_prev)

        S_cur_safe,  cur_idx_safe  = prune_cp_arr(S_cur,  p_percent = p_prune)
        S_prev_safe, prev_idx_safe = prune_cp_arr(S_prev, p_percent = p_prune)

        RMSE_cur  = float(xp_rmse(S_cur_safe,  S_cur_true[cur_idx_safe]))
        RMSE_prev = float(xp_rmse(S_prev_safe, S_prev_true[prev_idx_safe]))

        rmse_list.append(RMSE_cur)
        rmse_list.append(RMSE_prev)
        return y, rmse_list
    else:
        return y

# build once and reuse
_CUSTOM_SCORE = make_custom_score_from_torch(ema_score_xt_torch, DEVICE)

def probe_with_run_comparison(epoch_or_step: int, N_ref = 2000, N_part = 2000,
                              nsteps: int = 8 , trials = 3, p_prune = 50):
    # force run_comparison to use THIS blackbox sampler
    run_comparison.__globals__['blackbox_sampler'] = blackbox_sampler

    samplers = [
        ('heun_pc', 'custom', 'NN-Critic-EMA', _CUSTOM_SCORE),
    ]
    _ = run_comparison(
        samplers           = samplers,
        steps_list         = [16],
        true_sampler_func  = sampler_func,
        prior_sampler_func = sampler_func,
        prior_score_func   = score_func,
        true_score_func    = true_score_func_cp,
        true_init_score    = score_func,
        track_score_rmse   = True,
        N_ref              = N_ref,
        N_part             = N_part,
        trials             = trials,
        nrows              = 3,
        mean_bins          = 80,
        time_split         = 'power',
        T_end              = T_end,
        T_target           = T_target,
        div                = 'M_KSD',
        plot_hists         = True,
        hist_mode          = 'pca',
        display_mode       = 'min_label',
        trial_name         = f'NN_hist_compare_e{epoch_or_step}',
        p_prune            = p_prune,
        ref_seed           = 1,
        plot_res           = False,
        hist_norm          = 2.0,
        plot_prior         = True,
        save_tag           = f"run_data/SVDNN_epoch_{epoch_or_step}",
    )


# ----------------- Training -------------------------
SUBSTEPS = int(globals().get("SUBSTEPS", 1))
REF_CHUNK_TRAIN = min(4096, int(globals().get("N_REF", 4096)))
MAX_REF_TRAIN   = int(globals().get("N_REF", 10_000))


@torch.no_grad()
def _s0_local_runtime(x: torch.Tensor) -> torch.Tensor:
    """
    Compute the low-rank local proxy score that matches Script A (recompute=False):
      s(x) = Σ^{-1}(μ - x) with Σ = τ I + V diag(λ) V^T,
    using your anchors in `proxy_pool`.
    """
    return kernel_proxy_score_local_lrsvd(
        x, proxy_pool,
        k=None,                                 # uses Script-A-matched adaptive-k inside
        rank=PROXY_RANK,
        ref_chunk=PROXY_REF_CHUNK,
        ridge_mode=PROXY_RIDGE_MODE,
        tail_trim_q=0.1,
        ridge_scale=PROXY_TAU_SCALE,
        ridge_floor=PROXY_TAU_FLOOR,
        lam_clip_mult=PROXY_LAM_CLIP_MULT,
        alpha=PROXY_ALPHA,
        use_cpu_svd=True,
        enforce_dtype=x.dtype
    )


def _s0_proxy_for_batch(x0_batch, idx_in_train):
    if (S0_TRAIN_CPU is not None) and (idx_in_train is not None):
        return S0_TRAIN_CPU[idx_in_train].to(DEVICE, non_blocking=True)

    if USE_LOCAL_LRSVD_FOR_S0_BATCH:  # new flag
        return kernel_proxy_score_local_lrsvd(x0_batch, proxy_pool, rank=min(PROXY_RANK, x0_batch.shape[1]), ridge_mode="tail_mean",
                                                 tail_trim_q=0.1,ridge_scale=1.0,ridge_floor=1e-6,lam_clip_mult=1e3,
                                                alpha=0.4,use_cpu_svd=True,enforce_dtype=torch.float32)
    return kernel_proxy_score_torch(x0_batch, proxy_pool, recompute=False,
                                    ref_chunk=PROXY_REF_CHUNK, ridge_frac=PROXY_RIDGE_FRAC)


for step in range(1, STEPS+1):
    for _ in range(SUBSTEPS):
        for _ in range(CRIT_STEPS_PER_LAM):
              x_t, t, sigma_t, s_kss_np, s_twd_np, s_true, x0, idx_train = make_batch(
                  BATCH, ref_chunk=REF_CHUNK_TRAIN, max_ref=MAX_REF_TRAIN, t_mode='log', return_idx=True
              )
              #######
              with torch.no_grad():
                  et   = torch.exp(-t)
                  invv = 1.0 / (1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12)

                  # No resample: USE PRECOMPUTED if available (S0_TRAIN_CPU), else fallback
                  s0_base    = _s0_proxy_for_batch(x0, idx_train)
                  z_base_twd = x0

                  a = torch.exp(t) * s0_base
                  b = -invv * (x_t - et * z_base_twd)

                  lam_frozen = lam_net_ema(x_t, t).unsqueeze(-1).clamp(LAM_EPS, 1.0 - LAM_EPS)
                  z_scaled   = sigma_t * ((1 - lam_frozen) * a + lam_frozen * b)

              opt_crit.zero_grad(set_to_none=True)
              with torch.amp.autocast('cuda', enabled=USE_AMP):
                  q_scaled = critic(x_t, t)
                  loss_crit = mse(q_scaled, z_scaled)

              scaler_crit.scale(loss_crit).backward()
              scaler_crit.unscale_(opt_crit)
              torch.nn.utils.clip_grad_norm_(critic.parameters(), 1.0)
              scaler_crit.step(opt_crit)
              scaler_crit.update()
              sched_crit.step()
              update_ema(critic_ema, critic, EMA_DECAY)

    # ---------------- Gate: 1 step against frozen critic from EMA ---------------
    x_t, t, sigma_t, s_kss_np, s_twd_np, s_true, x0, idx_train = make_batch(
        BATCH, ref_chunk=REF_CHUNK_TRAIN, max_ref=MAX_REF_TRAIN, t_mode='log', return_idx=True
    )
    #####
    with torch.no_grad():
        et   = torch.exp(-t)
        invv = 1.0 / (1.0 - torch.exp(-2.0 * t)).clamp_min(1e-12)

        s0_base  = _s0_proxy_for_batch(x0, idx_train)
        z_base_twd = x0

        a = torch.exp(t) * s0_base
        b = -invv * (x_t - et * z_base_twd)
        q_scaled_frozen = critic_ema(x_t, t)


    opt_lam.zero_grad(set_to_none=True)
    with torch.amp.autocast('cuda', enabled=USE_AMP):
        lam_pred = lam_net(x_t, t).unsqueeze(-1).clamp(LAM_EPS, 1.0 - LAM_EPS)
        z_scaled = sigma_t * ((1 - lam_pred) * a + lam_pred * b)
        loss_lam = mse(z_scaled, q_scaled_frozen)

    if step > WARMUP_LAM_STEPS:
        scaler_lam.scale(loss_lam).backward()
        scaler_lam.unscale_(opt_lam)
        torch.nn.utils.clip_grad_norm_(lam_net.parameters(), 1.0)
        scaler_lam.step(opt_lam)
        scaler_lam.update()
        sched_lam.step()
        update_ema(lam_net_ema, lam_net, EMA_DECAY)
    else:
        sched_lam.step()

# ---------------- Logging every PRINT_EVERY steps ----------------
    if step % PRINT_EVERY == 0:
        metrics = eval_gamma_grid_rmse(n_t=EVAL_N_T, per_t=EVAL_PER_T)
        lr_c = sched_crit.get_last_lr()[0]; lr_l = sched_lam.get_last_lr()[0]
        lam_mean = lam_net_ema(x_t, t).mean().item()
        lam_std  = lam_net_ema(x_t, t).std(unbiased=False).item()
        print(f"[{step:5d}/{STEPS}] lr(crit):{lr_c:.2e} lr(λ):{lr_l:.2e} | "
              f"loss(crit):{loss_crit.item():.3e}"
              + (f" loss(λ):{loss_lam.item():.3e}" if step>WARMUP_LAM_STEPS else " (λ warmup)"))
        print(f"  λ(EMA) mean={lam_mean:.3f} std={lam_std:.3f}")
        print("  TRUE(γ): "
              f"Critic RMSE:{metrics['Critic_RMSE']:.3e}  "
              f"KNP:{metrics['KSSNP_TRUE']:.3e}  "
              f"TNP:{metrics['TWDNP_TRUE']:.3e}  "
              f"BNP[SNIS-λ]:{metrics['BLNDNP_TRUE_NP']:.3e}  "
              f"BNP[NN-λ]: {metrics['BLNDNP_TRUE_NN']:.3e}  |  ")
        if SHOW_LAMBDA_STATS:
            print(f"  λ agreement (net vs SNIS): mean|Δ|={metrics['LAM_ABS_DIFF']:.3e}  "
                  f"pearson={metrics['LAM_PEARSON']:.3f}\n")

        # --- end-to-end probe through EXACT blackbox path (Heun-PC, 10 steps) ---
        try:
            probe_with_run_comparison(step, nsteps=8, trials = 1)
        except Exception as e:
            print(f"[WARN] run_comparison probe failed: {e}\n")



# === Save EMA versions ===
def unwrap_module(m):
    if hasattr(m, "module"):   m = m.module
    if hasattr(m, "_orig_mod"): m = m._orig_mod
    return m


torch.save(
    {
        "lam_net": unwrap_module(lam_net_ema).state_dict(),
        "critic":  unwrap_module(critic_ema).state_dict(),
        "dim":     D,
        "hidden":  HIDDEN,
        "proxy_meta": {
                      "PROXY_N": PROXY_N,
                      "PROXY_RIDGE_FRAC": PROXY_RIDGE_FRAC,
                      "PROXY_ALPHA": PROXY_ALPHA,
                      "PROXY_REF_CHUNK": PROXY_REF_CHUNK,
                      "PRECOMPUTE_TRAIN_SCORES": PRECOMPUTE_TRAIN_SCORES,
                      "PROXY_RECOMPUTE": PROXY_RECOMPUTE,
                      "PROXY_GMM_BATCH": PROXY_GMM_BATCH,
                      "PROXY_RECOMPUTE_MAX_ANCHORS": PROXY_RECOMPUTE_MAX_ANCHORS,
                      "PROXY_RANK": PROXY_RANK,
                      "PROXY_V_DTYPE": str(PROXY_V_DTYPE),
                      "PROXY_TAU_MODE": PROXY_TAU_MODE,
                      "PROXY_TAU_TRIM_Q": PROXY_TAU_TRIM_Q,
                      "PROXY_TAU_SCALE": PROXY_TAU_SCALE,
                      "PROXY_TAU_FLOOR": PROXY_TAU_FLOOR,
                      "PROXY_LAM_CLIP_MULT": PROXY_LAM_CLIP_MULT,
}
    },
    SAVE_PATH,
)
print(f"[INFO] EMA networks written to {SAVE_PATH}")

In [ ]:
# ================================================================
# Amortized vs Stochastic (Unamortized) vs Oracle-s0 Critic-Gate vs NN-DSM
# Full comparison script
# - Loads:
#     * kss/twd/λ bundle:              "kss_twd_lambda_ema.pt"
#     * critic (proxy s0):             "learned_score_critic_gate_ema.pt"
#     * critic (amortized s0):         "amort_learned_score_critic_gate_ema.pt"
#     * ORIGINAL critic-gate (oracle): "critic_gate_ema.pt"
# - Assumes the following utilities are importable:
#     get_gmm_funcs, OU_evolve_samples, calculate_true_score_at_t,
#     tweedie_score_t, kss_score_t, snis_blend, snis_blend_oracle, run_comparison
# ================================================================

import os
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:256")

import torch, cupy as cp, math, gc
from torch import nn

DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
DTORCH  = torch.float32

# -------------------- VRAM hygiene --------------------
def clear_gpu_mem():
    try:
        import cupy as cp
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()
    except Exception:
        pass
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        for dev in range(torch.cuda.device_count()):
            torch.cuda.set_device(dev)
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()
clear_gpu_mem()


# -------------------- Model defs --------------------
class ScoreNet(nn.Module):
    def __init__(self, dim, hidden=512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim+1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, dim),
        )
    def forward(self, x, t):
        return self.net(torch.cat([x, torch.log(t)], dim=1))

class LambdaNet(nn.Module):
    def __init__(self, dim, hidden=128):
        super().__init__()
        self.f = nn.Sequential(
            nn.Linear(dim+1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, 1), nn.Sigmoid()
        )
    def forward(self, x, t):
        return self.f(torch.cat([x, torch.log(t)], dim=1)).squeeze(-1)

class CriticNet(nn.Module):
    def __init__(self, dim, hidden=512):
        super().__init__()
        self.f = nn.Sequential(
            nn.Linear(dim+1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, dim),
        )
    def forward(self, y, t):
        # predicts σ_t * s*(y,t)
        return self.f(torch.cat([y, torch.log(t)], dim=1))

# -------------------- Infer dims/hidden from state dict --------------------
def _infer_hidden_from_state_dict(sd: dict, first_layer_prefix_options=("net.0.weight", "f.0.weight")):
    key = None
    for k in first_layer_prefix_options:
        if k in sd:
            key = k
            break
    if key is None:
        for k in sd.keys():
            if k.endswith("0.weight"):
                key = k
                break
    if key is None:
        raise KeyError("Could not infer first layer weight key ('.0.weight') in state dict.")
    w = sd[key]   # [hidden, dim_in]
    hidden = w.shape[0]
    dim_plus_one = w.shape[1]
    dim = dim_plus_one - 1  # we append log(t)
    return hidden, dim



# -------------------- Load critics --------------------


# 0) Scalar Critic Gate trained with exact s0
#critic_ckpt = torch.load("critic_gate_ema.pt", map_location=DEVICE)
#crit_sd = critic_ckpt.get("critic_net") or critic_ckpt.get("critic")
#if crit_sd is None:
#    raise KeyError("Critic weights not found in learned_score_critic_gate_ema.pt.")
#H_crit, D_crit = _infer_hidden_from_state_dict(crit_sd)
#critic_net = CriticNet(D_crit, hidden=H_crit).to(DEVICE).eval()
#critic_net.load_state_dict(crit_sd)


# 1) Scalar Critic Gate trained with learned proxy s0
#critic_proxy_ckpt = torch.load("learned_score_critic_gate_ema.pt", map_location=DEVICE)
#crit_proxy_sd = critic_proxy_ckpt.get("critic_net") or critic_proxy_ckpt.get("critic")
#if crit_proxy_sd is None:
#    raise KeyError("Critic weights not found in learned_score_critic_gate_ema.pt.")
#H_crit_proxy, D_crit_proxy = _infer_hidden_from_state_dict(crit_proxy_sd)
#critic_proxy_net = CriticNet(D_crit_proxy, hidden=H_crit_proxy).to(DEVICE).eval()
#critic_proxy_net.load_state_dict(crit_proxy_sd)


# 2) Tweedie-only DSM (NN-DSM)
twd_ckpt = torch.load("tweedie_distill_ema.pt", map_location=DEVICE)
twd_sd   = twd_ckpt.get("net_twd") or twd_ckpt.get("net") or twd_ckpt
H_twd, D_twd = _infer_hidden_from_state_dict(
    twd_sd, first_layer_prefix_options=("net.0.weight",)
)
net_twd = ScoreNet(D_twd, hidden=H_twd).to(DEVICE).eval()
net_twd.load_state_dict(twd_sd)


# 3) Scalar Critic Gate trained with low rank learned proxy s0
#lr_critic_proxy_ckpt = torch.load("learned_lr_score_critic_gate_ema2.pt", map_location=DEVICE)
#lr_crit_proxy_sd = lr_critic_proxy_ckpt.get("critic_net") or lr_critic_proxy_ckpt.get("critic")
#if lr_crit_proxy_sd is None:
#    raise KeyError("Critic weights not found in learned_lr_score_critic_gate_ema.pt.")
#H_lr_crit_proxy, D_lr_crit_proxy = _infer_hidden_from_state_dict(lr_crit_proxy_sd)
#lr_critic_proxy_net = CriticNet(D_lr_crit_proxy, hidden=H_lr_crit_proxy).to(DEVICE).eval()
#lr_critic_proxy_net.load_state_dict(lr_crit_proxy_sd)

# sanity: match dims with critic you already loaded
#if "D_crit_proxy" in globals():
#    assert D_twd == D_crit_proxy, f"Dim mismatch: DSM D={D_twd}, critic D={D_crit_proxy}"
#    assert D_lr_crit_proxy == D_crit_proxy, f"Dim mismatch: lr-critic D={D_lr_crit_proxy}, critic D={D_crit_proxy}"
# Use a single expected data dimension inferred from loaded nets
#D_expected = D_twd  # (same as D_crit_proxy; asserted above)


# -------------------- Load critics --------------------
# 4) Scalar Critic Gate trained with recomputed learned proxy s0

recomp_critic_proxy_ckpt = torch.load("learned_score_critic_gate_recomp_ema.pt", map_location=DEVICE)
recomp_crit_proxy_sd = recomp_critic_proxy_ckpt.get("critic_net") or recomp_critic_proxy_ckpt.get("critic")
if recomp_crit_proxy_sd is None:
    raise KeyError("Critic weights not found in learned_score_critic_gate_recomp_ema.pt.")
H_recomp_crit_proxy, D_recomp_crit_proxy = _infer_hidden_from_state_dict(recomp_crit_proxy_sd)
recomp_critic_proxy_net = CriticNet(D_recomp_crit_proxy, hidden=H_recomp_crit_proxy).to(DEVICE).eval()
recomp_critic_proxy_net.load_state_dict(recomp_crit_proxy_sd)


# -------------------- Distribution (match training) --------------------
NUM_C   = globals().get("NUM_C",   2000)
K_DIM   = globals().get("K_DIM",   12)
VARIANT = globals().get("VARIANT", "helix")
STD     = globals().get("STD",     0.12)
SCALE   = globals().get("SCALE",   3.0)
EMBED_MODE = globals().get("EMBED_MODE", "sine_wiggle")

T_target = globals().get("T_target", 1e-3)
T_end    = globals().get("T_end",    1.5)


SEED  = globals().get("SEED", 0)
N_ref_sampling = globals().get("N_ref_sampling", 5000)
ref_seed = SEED

params, sampler_func, _ = get_gmm_funcs(
    num_c=NUM_C, k_dim=K_DIM, variant=VARIANT,
    comp_std=STD, overall_scale=SCALE, seed=SEED, embedding_mode = EMBED_MODE)[:3]
means0_cp, stds0_cp, w0_cp = map(cp.asarray, params)

# -------------------- CuPy/Torch bridges --------------------
from torch.utils.dlpack import from_dlpack as torch_from_dlpack
from torch.utils.dlpack import to_dlpack as torch_to_dlpack

def cp_to_torch_float32(x_cp):
    x32 = cp.ascontiguousarray(x_cp.astype(cp.float32))
    return torch_from_dlpack(x32.toDlpack()).to(DEVICE)

def torch_to_cp_float64(x_t):
    x_t = x_t.detach().contiguous().to('cpu' if DEVICE=='cpu' else DEVICE)
    x_cp = cp.fromDlpack(torch_to_dlpack(x_t))
    return x_cp.astype(cp.float64)

# -------------------- True score helper (CuPy) --------------------
def true_score_func_cp(y_cp, t_scalar):
    return calculate_true_score_at_t(
        y_cp, t_scalar, means0_cp, stds0_cp, w0_cp, batch_size=4096
    )

# For final-time metric:
score_func = lambda y_cp: true_score_func_cp(y_cp, 1e-6)
evol_sampler_func = lambda n: sampler_func(n)

# -------------------- Dim check --------------------
D_expected = D
def _ensure_2d_y(y_t: torch.Tensor, D_expected: int) -> torch.Tensor:
    if y_t.ndim == 1:
        raise ValueError(f"Got 1D input of shape {tuple(y_t.shape)}; expected (B,{D_expected}).")
    if y_t.shape[1] != D_expected:
        raise ValueError(f"Dim mismatch: got (B,{y_t.shape[1]}), expected (B,{D_expected}).")
    return y_t

# -------------------- NN score callables → CuPy (float64) --------------------

@torch.no_grad()
def nn_twd_score_cp(y_cp, t_scalar):
    y_t = cp_to_torch_float32(y_cp); y_t = _ensure_2d_y(y_t, D_expected)
    t_t = torch.full((y_t.shape[0], 1), float(t_scalar), device=DEVICE, dtype=DTORCH)
    sigma = torch.sqrt((1.0 - torch.exp(-2.0*t_t)).clamp_min(1e-12))
    s = net_twd(y_t, t_t) / sigma
    return torch_to_cp_float64(s)

@torch.no_grad()
def lam_net_cp(y_cp, t_scalar):
    y_t = cp_to_torch_float32(y_cp); y_t = _ensure_2d_y(y_t, D_expected)
    t_t = torch.full((y_t.shape[0], 1), float(t_scalar), device=DEVICE, dtype=DTORCH)
    lam = lam_net(y_t, t_t)
    return torch_to_cp_float64(lam)

def make_nn_blend_score(net_kss, net_twd, lam_net_ema=None, D_expected=None, device="cuda"):
    def score_nn(y_cp, t_scalar):
        y_t = torch.as_tensor(cp.asnumpy(y_cp), device=device, dtype=torch.float32)
        B, Ddim = y_t.shape
        if D_expected is not None:
            assert Ddim == D_expected, f"dim mismatch: x has {Ddim}, model expects {D_expected}"
        t = torch.full((B,1), float(t_scalar), device=device, dtype=y_t.dtype)
        sigma = torch.sqrt((1.0 - torch.exp(-2.0*t)).clamp_min(1e-12))
        s_kss = net_kss(y_t, t) / sigma
        s_twd = net_twd(y_t, t) / sigma
        lam = (0.5 * torch.ones((B,1), device=device, dtype=y_t.dtype)
               if lam_net_ema is None else lam_net_ema(y_t, t).unsqueeze(-1))
        s = (1 - lam) * s_kss + lam * s_twd
        return cp.asarray(s.detach().cpu().numpy(), dtype=cp.float64)
    return score_nn

def make_nn_critic_score_cp(critic_module: nn.Module, D_expected: int):
    @torch.no_grad()
    def f(y_cp, t_scalar):
        y_t = cp_to_torch_float32(y_cp); y_t = _ensure_2d_y(y_t, D_expected)
        t_t = torch.full((y_t.shape[0], 1), float(t_scalar), device=DEVICE, dtype=DTORCH)
        sigma = torch.sqrt((1.0 - torch.exp(-2.0*t_t)).clamp_min(1e-12))
        s = critic_module(y_t, t_t) / sigma
        return torch_to_cp_float64(s)
    return f

@torch.no_grad()
def nn_twd_score_cp(y_cp, t_scalar):
    y_t = cp_to_torch_float32(y_cp); _ensure_2d_y(y_t, D_expected)
    t_t = torch.full((y_t.shape[0], 1), float(t_scalar), device=DEVICE, dtype=DTORCH)
    sigma = torch.sqrt((1.0 - torch.exp(-2.0 * t_t)).clamp_min(1e-12))
    s = net_twd(y_t, t_t) / sigma   # DSM predicts sigma * s
    return torch_to_cp_float64(s)

# ---- critic callables ----
#nn_critic_score_cp = make_nn_blend_score(net_kss=critic_net, net_twd=net_twd)
#nn_critic_proxy_score_cp = make_nn_critic_score_cp(critic_proxy_net, D_crit_proxy)
#nn_critic_lr_proxy_score_cp = make_nn_critic_score_cp(lr_critic_proxy_net, D_lr_crit_proxy)
nn_critic_recomp_score_cp = make_nn_critic_score_cp(recomp_critic_proxy_net, D_recomp_crit_proxy)


# === NP reference at final time ===
x0_ref = sampler_func(N_ref_sampling, seed=(ref_seed if ref_seed!='rand' else None))
s0_ref = score_func(x0_ref)

# -------------------- Build estimator dictionary --------------------
def build_estimators(x0_ref, s0_ref, true_score_func_cp):
    def np_twd(y_cp, t): return tweedie_score_t(y_cp, t, x0_ref)
    def np_kss(y_cp, t): return kss_score_t(y_cp, t, x0_ref, s0_ref)
    def np_blend(y_cp, t):
        lam = snis_blend(y_cp, x0_ref, s0_ref, t)
        sk, st = np_kss(y_cp, t), np_twd(y_cp, t)
        return (1.0 - lam)[:,None]*sk + lam[:,None]*st
    return {
      "NP-Tweedie"               : np_twd,
      "NP-KSS"                   : np_kss,
      "NP-Blend(SNIS)"           : np_blend,
      "True"                     : true_score_func_cp,
      #"NN-Critic"                 : nn_critic_score_cp,
      #"NN-Critic-proxy-score"    : nn_critic_proxy_score_cp,
      #"NN-Critic-lr-proxy-score" : nn_critic_lr_proxy_score_cp,
      "NN-Critic-recomp-score"   : nn_critic_recomp_score_cp,
      "NN-Tweedie"               : nn_twd_score_cp,   # <-- add this
}
estimators = build_estimators(x0_ref, s0_ref, true_score_func_cp)

# -------------------- Main experiment --------------------
def main():
    sampler = 'heun_pc'
    samplers = [
        #(sampler, 'custom', 'NN-Critic-proxy-score',    estimators['NN-Critic-proxy-score']),
        #(sampler, 'custom', 'NN-Critic',         estimators['NN-Critic']),
        (sampler, 'custom', "Critic Gate", estimators["NN-Critic-recomp-score"]),
        (sampler, 'custom', 'DSM',               estimators['NN-Tweedie']),
        #(sampler, 'custom', 'NN-Critic-lr-proxy-score', estimators['NN-Critic-lr-proxy-score'])
        ]

    results = run_comparison(
        samplers        = samplers,
        steps_list      = [10],
        true_sampler_func = sampler_func,
        prior_sampler_func    = sampler_func,
        prior_score_func      = score_func,
        true_score_func = true_score_func_cp,
        true_init_score = score_func,
        track_score_rmse= True,
        N_ref           = 5000,
        N_part          = 5000,
        trials          = 20,
        nrows           = 4,
        mean_bins       = 120,
        time_split      = 'power',
        T_end           = T_end,
        T_target        = T_target,
        div             = 'OT_MMD',
        plot_hists      = True,
        hist_mode       = 'rand',
        display_mode    = 'min_label',
        trial_name      = 'NN_hist_compare',
        p_prune         = 1,
        ref_seed        = 1,
        plot_res = False,
        hist_norm= 3.45,
        plot_prior = True,
        rand_seed = False,
        self_normalize_hists = False,
    )

if __name__ == "__main__":
    main()

/tmp/ipython-input-2644331221.py:192: DeprecationWarning: This function is deprecated in favor of cupy.from_dlpack
  x_cp = cp.fromDlpack(torch_to_dlpack(x_t))


10 steps, Critic Gate: OT_MMD =3.400e-02, OT_MMD_floor = 1.900e-02, score_RMSE=1.216e+32, time=74.7s
 
10 steps, DSM: OT_MMD =3.864e-02, OT_MMD_floor = 1.936e-02, score_RMSE=3.262e+01, time=74.6s
 


In [ ]:
!rm run_data/*.png